# Enriching data by joining to LOOKUP tables
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

Introductory paragraph - for example:

This tutorial demonstrates how to work with [feature](link to feature doc). In this tutorial you perform the following tasks:

- Task 1
- Task 2
- Task 3
- etc



## Table of contents

- [Prerequisites](#Prerequisites)
- [Initalization](#Initalization)
- [Next section](#Nextsection)
- etc

## Prerequisites

This tutorial works with Druid XX.0.0 or later.

#### Run with Docker

<!-- Profiles are:
`druid-jupyter` - just Jupyter and Druid
`all-services` - includes Jupyter, Druid, and Kafka
 -->

Launch this tutorial and all prerequisites using the ....... profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).
   
#### Run without Docker

If you do not use the Docker Compose environment, you need the following:

* A running Apache Druid instance, with a `DRUID_HOST` local environment variable containing the server name of your Druid router
* [druidapi](https://github.com/apache/druid/blob/master/examples/quickstart/jupyter-notebooks/druidapi/README.md), a Python client for Apache Druid. Follow the instructions in the Install section of the README file.

 <!-- Remove as needed -->
* A running Apache Kafka instance, with a `KAFKA_HOST` local environment variable containing the broker server name.
* [matplotlib](https://matplotlib.org/), a library for creating visualizations in Python.
* [pandas](https://pandas.pydata.org/), a data analysis and manipulation tool.

### Initialization

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

<!-- Include these cells if you're relying on someone ingesting example data through the console -->

### Load example data

Once your Druid environment is up and running, ingest the sample data for this tutorial.

Run the following cell to create a table called `example-flights-lookup`. Notice {the use of X as a timestamp | only required columns are ingested | WHERE / expressions / GROUP BY are front-loaded | partitions on X period and clusters by Y}.

When completed, you'll see a description of the final table.

<!--

Replace `example-flights-lookup` with a unique table name for this notebook.

- Always prefix your table name with `example-`
- If using the standard example datasets, use the following standard values for `dataset`:

    wikipedia       wikipedia
    koalas          KoalasToTheMax one day
    koalanest       KoalasToTheMax one day (nested)
    nyctaxi3        NYC Taxi cabs (3 files)
    nyctaxi         NYC Taxi cabs (all files)
    flights         FlightCarrierOnTime (1 month)

-->

Monitor the ingestion task process in the Druid console.

In [ ]:
sql='''
REPLACE INTO "example-flights-lookup" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/flight_on_time/flights/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11.csv.zip"]}',
    '{"type":"csv","findColumnsFromHeader":true}'
  )
) EXTEND ("depaturetime" VARCHAR, "arrivalime" VARCHAR, "Year" BIGINT, "Quarter" BIGINT, "Month" BIGINT, "DayofMonth" BIGINT, "DayOfWeek" BIGINT, "FlightDate" VARCHAR, "Reporting_Airline" VARCHAR, "DOT_ID_Reporting_Airline" BIGINT, "IATA_CODE_Reporting_Airline" VARCHAR, "Tail_Number" VARCHAR, "Flight_Number_Reporting_Airline" BIGINT, "OriginAirportID" BIGINT, "OriginAirportSeqID" BIGINT, "OriginCityMarketID" BIGINT, "Origin" VARCHAR, "OriginCityName" VARCHAR, "OriginState" VARCHAR, "OriginStateFips" BIGINT, "OriginStateName" VARCHAR, "OriginWac" BIGINT, "DestAirportID" BIGINT, "DestAirportSeqID" BIGINT, "DestCityMarketID" BIGINT, "Dest" VARCHAR, "DestCityName" VARCHAR, "DestState" VARCHAR, "DestStateFips" BIGINT, "DestStateName" VARCHAR, "DestWac" BIGINT, "CRSDepTime" BIGINT, "DepTime" BIGINT, "DepDelay" BIGINT, "DepDelayMinutes" BIGINT, "DepDel15" BIGINT, "DepartureDelayGroups" BIGINT, "DepTimeBlk" VARCHAR, "TaxiOut" BIGINT, "WheelsOff" BIGINT, "WheelsOn" BIGINT, "TaxiIn" BIGINT, "CRSArrTime" BIGINT, "ArrTime" BIGINT, "ArrDelay" BIGINT, "ArrDelayMinutes" BIGINT, "ArrDel15" BIGINT, "ArrivalDelayGroups" BIGINT, "ArrTimeBlk" VARCHAR, "Cancelled" BIGINT, "CancellationCode" VARCHAR, "Diverted" BIGINT, "CRSElapsedTime" BIGINT, "ActualElapsedTime" BIGINT, "AirTime" BIGINT, "Flights" BIGINT, "Distance" BIGINT, "DistanceGroup" BIGINT, "CarrierDelay" BIGINT, "WeatherDelay" BIGINT, "NASDelay" BIGINT, "SecurityDelay" BIGINT, "LateAircraftDelay" BIGINT, "FirstDepTime" VARCHAR, "TotalAddGTime" VARCHAR, "LongestAddGTime" VARCHAR, "DivAirportLandings" VARCHAR, "DivReachedDest" VARCHAR, "DivActualElapsedTime" VARCHAR, "DivArrDelay" VARCHAR, "DivDistance" VARCHAR, "Div1Airport" VARCHAR, "Div1AirportID" VARCHAR, "Div1AirportSeqID" VARCHAR, "Div1WheelsOn" VARCHAR, "Div1TotalGTime" VARCHAR, "Div1LongestGTime" VARCHAR, "Div1WheelsOff" VARCHAR, "Div1TailNum" VARCHAR, "Div2Airport" VARCHAR, "Div2AirportID" VARCHAR, "Div2AirportSeqID" VARCHAR, "Div2WheelsOn" VARCHAR, "Div2TotalGTime" VARCHAR, "Div2LongestGTime" VARCHAR, "Div2WheelsOff" VARCHAR, "Div2TailNum" VARCHAR, "Div3Airport" VARCHAR, "Div3AirportID" VARCHAR, "Div3AirportSeqID" VARCHAR, "Div3WheelsOn" VARCHAR, "Div3TotalGTime" VARCHAR, "Div3LongestGTime" VARCHAR, "Div3WheelsOff" VARCHAR, "Div3TailNum" VARCHAR, "Div4Airport" VARCHAR, "Div4AirportID" VARCHAR, "Div4AirportSeqID" VARCHAR, "Div4WheelsOn" VARCHAR, "Div4TotalGTime" VARCHAR, "Div4LongestGTime" VARCHAR, "Div4WheelsOff" VARCHAR, "Div4TailNum" VARCHAR, "Div5Airport" VARCHAR, "Div5AirportID" VARCHAR, "Div5AirportSeqID" VARCHAR, "Div5WheelsOn" VARCHAR, "Div5TotalGTime" VARCHAR, "Div5LongestGTime" VARCHAR, "Div5WheelsOff" VARCHAR, "Div5TailNum" VARCHAR, "Unnamed: 109" VARCHAR))
SELECT
  TIME_PARSE("depaturetime") AS "__time",
  "Reporting_Airline",
  "Tail_Number",
  "Distance",
  "Origin",
  "OriginAirportID",
  "Dest",
  "DestAirportID",
  "Flight_Number_Reporting_Airline"
FROM "ext"
PARTITIONED BY DAY
'''

sql_client.run_task(sql)
sql_client.wait_until_ready('example-flights-lookup')
display.table('example-flights-lookup')

Run the following cell to import additional Python modules that you will use to X, Y, Z.

In [ ]:
# Add your modules here, remembering to align this with the prerequisites section

import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import datetime
import requests

The JSON object we will send to the API contains this information:

* lookup _tier_
* lookup _name_
* lookup data _version_
* the lookup _data_

Run the following cell to put these values into variables we will use later.

It calculates a _version_ for the lookup. This will be incorporated into the JSON POST so that, when we next need to update the `LOOKUP` data, we can provide a later version number or - in this case - datetime.

In [ ]:
lookup_tier = "__default"
lookup_name = "example-flights-airportcodes"
lookup_version = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(lookup_version)

Run the next cell to create a dict object that we will use in the post.

In [ ]:
lookup_definition = {
    lookup_tier: {
        lookup_name: {
            "version": lookup_version,  
            "lookupExtractorFactory": {
                "type": "map",
                    "map": {
                        "03N": "small_airport",
                        "07FA": "small_airport",
                        "0AK": "small_airport",
                        "0CO2": "small_airport",
                        "0TE7": "small_airport",
                        "13MA": "small_airport",
                        "13Z": "seaplane_base",
                        "16A": "small_airport",
                        "16K": "seaplane_base",
                        "19AK": "small_airport",
                        "19P": "seaplane_base",
                        "1KC": "small_airport",
                        "1O6": "small_airport",
                        "1XA2": "small_airport",
                        "1Z1": "small_airport",
                        "1Z9": "seaplane_base",
                        "2AK": "small_airport",
                        "2AK6": "small_airport",
                        "2IG4": "small_airport",
                        "2K5": "small_airport",
                        "2TE0": "small_airport",
                        "2Z1": "seaplane_base",
                        "2Z6": "seaplane_base",
                        "38WA": "small_airport",
                        "3AK5": "small_airport",
                        "3IS8": "small_airport",
                        "3VS": "small_airport",
                        "3Z8": "seaplane_base",
                        "4A2": "small_airport",
                        "4AK": "small_airport",
                        "4K0": "small_airport",
                        "4K5": "small_airport",
                        "4KA": "small_airport",
                        "4TE8": "closed",
                        "4Z7": "seaplane_base",
                        "51Z": "small_airport",
                        "57A": "seaplane_base",
                        "5A8": "medium_airport",
                        "5KE": "seaplane_base",
                        "5NK": "small_airport",
                        "5Z9": "seaplane_base",
                        "65LA": "small_airport",
                        "6CA3": "heliport",
                        "6N5": "heliport",
                        "6N7": "seaplane_base",
                        "6OK6": "small_airport",
                        "78K": "seaplane_base",
                        "78WA": "small_airport",
                        "7KA": "small_airport",
                        "7NC2": "small_airport",
                        "7WA5": "small_airport",
                        "82CL": "small_airport",
                        "83Q": "seaplane_base",
                        "84CL": "heliport",
                        "84K": "seaplane_base",
                        "89NY": "small_airport",
                        "8XS8": "small_airport",
                        "90WA": "small_airport",
                        "96Z": "seaplane_base",
                        "9A3": "small_airport",
                        "9A8": "small_airport",
                        "9B1": "small_airport",
                        "9N2": "seaplane_base",
                        "9Z8": "small_airport",
                        "A23": "seaplane_base",
                        "A61": "small_airport",
                        "A63": "small_airport",
                        "A79": "small_airport",
                        "AAD": "small_airport",
                        "ABP": "small_airport",
                        "ABW": "closed",
                        "AD-ALV": "heliport",
                        "ADC": "small_airport",
                        "ADV": "small_airport",
                        "AE-0006": "seaplane_base",
                        "AEE": "small_airport",
                        "AEI": "heliport",
                        "AEK": "small_airport",
                        "AEQ": "small_airport",
                        "AF10": "small_airport",
                        "AF11": "small_airport",
                        "AFK": "seaplane_base",
                        "AFR": "small_airport",
                        "AGAF": "small_airport",
                        "AGAR": "small_airport",
                        "AGAT": "small_airport",
                        "AGBA": "small_airport",
                        "AGEV": "small_airport",
                        "AGG": "small_airport",
                        "AGGA": "small_airport",
                        "AGGB": "small_airport",
                        "AGGC": "small_airport",
                        "AGGE": "small_airport",
                        "AGGF": "small_airport",
                        "AGGH": "medium_airport",
                        "AGGI": "small_airport",
                        "AGGJ": "small_airport",
                        "AGGK": "small_airport",
                        "AGGL": "small_airport",
                        "AGGM": "medium_airport",
                        "AGGN": "small_airport",
                        "AGGO": "small_airport",
                        "AGGP": "small_airport",
                        "AGGQ": "closed",
                        "AGGR": "small_airport",
                        "AGGS": "small_airport",
                        "AGGT": "small_airport",
                        "AGGU": "small_airport",
                        "AGGV": "small_airport",
                        "AGGY": "small_airport",
                        "AGK": "small_airport",
                        "AGKG": "small_airport",
                        "AGKU": "small_airport",
                        "AGKW": "small_airport",
                        "AGL": "small_airport",
                        "AGNA": "small_airport",
                        "AGRC": "small_airport",
                        "AGRM": "small_airport",
                        "AGTI": "heliport",
                        "AGY": "small_airport",
                        "AHJ": "medium_airport",
                        "AHT": "closed",
                        "AHY": "small_airport",
                        "AIE": "small_airport",
                        "AIH": "small_airport",
                        "AIP": "small_airport",
                        "AK13": "small_airport",
                        "AK26": "small_airport",
                        "AK33": "small_airport",
                        "AK49": "small_airport",
                        "AK56": "seaplane_base",
                        "AK62": "seaplane_base",
                        "AK71": "small_airport",
                        "AK75": "small_airport",
                        "AK81": "seaplane_base",
                        "AK96": "small_airport",
                        "AK97": "small_airport",
                        "AKM": "small_airport",
                        "AL73": "small_airport",
                        "ALZ": "seaplane_base",
                        "AMC": "small_airport",
                        "AME": "small_airport",
                        "AMF": "small_airport",
                        "AMU": "small_airport",
                        "AMY": "small_airport",
                        "ANH": "closed",
                        "ANYN": "medium_airport",
                        "ANZ": "small_airport",
                        "AO-0005": "small_airport",
                        "AO-CNZ": "small_airport",
                        "AO-DRC": "small_airport",
                        "AO-GGC": "small_airport",
                        "AO-JMB": "small_airport",
                        "AO-KNP": "small_airport",
                        "AO-NDF": "small_airport",
                        "AOA": "closed",
                        "AOB": "small_airport",
                        "AOD": "small_airport",
                        "APP": "small_airport",
                        "APR": "small_airport",
                        "AQ-0002": "small_airport",
                        "AQY": "small_airport",
                        "AR-0005": "small_airport",
                        "AR-0007": "small_airport",
                        "AR-0009": "small_airport",
                        "AR-0010": "small_airport",
                        "AR-0011": "small_airport",
                        "AR-0015": "closed",
                        "AR-0017": "small_airport",
                        "AR-0346": "small_airport",
                        "AR-0390": "small_airport",
                        "AR-0462": "small_airport",
                        "AR-0494": "small_airport",
                        "ARP": "small_airport",
                        "ARX": "closed",
                        "AS-TAV": "small_airport",
                        "ASZ": "small_airport",
                        "ATN": "small_airport",
                        "ATP": "small_airport",
                        "AU-0081": "small_airport",
                        "AU-AGW": "small_airport",
                        "AU-AYD": "small_airport",
                        "AU-BYX": "small_airport",
                        "AU-COB": "small_airport",
                        "AU-CRJ": "small_airport",
                        "AU-CRY": "small_airport",
                        "AU-CSD": "small_airport",
                        "AU-DYM": "small_airport",
                        "AU-HIS": "seaplane_base",
                        "AU-HLV": "small_airport",
                        "AU-KBD": "small_airport",
                        "AU-KGR": "small_airport",
                        "AU-MWY": "small_airport",
                        "AU-MYO": "small_airport",
                        "AU-OKB": "small_airport",
                        "AU-PEP": "small_airport",
                        "AU-RDA": "small_airport",
                        "AU-SSK": "small_airport",
                        "AU-SWB": "small_airport",
                        "AU-TPR": "small_airport",
                        "AU-TWP": "small_airport",
                        "AU-ZVG": "small_airport",
                        "AUI": "small_airport",
                        "AUJ": "small_airport",
                        "AUP": "small_airport",
                        "AUV": "small_airport",
                        "AWE": "small_airport",
                        "AWR": "closed",
                        "AXF": "medium_airport",
                        "AYAQ": "small_airport",
                        "AYBK": "medium_airport",
                        "AYBR": "small_airport",
                        "AYCH": "medium_airport",
                        "AYDK": "small_airport",
                        "AYDL": "small_airport",
                        "AYDN": "small_airport",
                        "AYDO": "small_airport",
                        "AYDU": "medium_airport",
                        "AYEB": "small_airport",
                        "AYED": "small_airport",
                        "AYEL": "small_airport",
                        "AYEN": "small_airport",
                        "AYEO": "small_airport",
                        "AYER": "small_airport",
                        "AYES": "small_airport",
                        "AYFE": "small_airport",
                        "AYFR": "small_airport",
                        "AYFU": "small_airport",
                        "AYGA": "medium_airport",
                        "AYGF": "small_airport",
                        "AYGG": "small_airport",
                        "AYGN": "medium_airport",
                        "AYGP": "small_airport",
                        "AYGR": "medium_airport",
                        "AYGS": "small_airport",
                        "AYHB": "small_airport",
                        "AYHE": "small_airport",
                        "AYHH": "small_airport",
                        "AYHK": "medium_airport",
                        "AYHO": "small_airport",
                        "AYII": "small_airport",
                        "AYIM": "small_airport",
                        "AYIO": "small_airport",
                        "AYIU": "small_airport",
                        "AYIX": "small_airport",
                        "AYJO": "small_airport",
                        "AYKA": "small_airport",
                        "AYKG": "small_airport",
                        "AYKI": "small_airport",
                        "AYKJ": "small_airport",
                        "AYKK": "small_airport",
                        "AYKM": "medium_airport",
                        "AYKR": "small_airport",
                        "AYKT": "small_airport",
                        "AYKU": "small_airport",
                        "AYKV": "medium_airport",
                        "AYKY": "small_airport",
                        "AYLB": "small_airport",
                        "AYLO": "small_airport",
                        "AYLP": "small_airport",
                        "AYLS": "small_airport",
                        "AYLX": "small_airport",
                        "AYM": "seaplane_base",
                        "AYMA": "small_airport",
                        "AYMB": "small_airport",
                        "AYMD": "medium_airport",
                        "AYMH": "medium_airport",
                        "AYML": "small_airport",
                        "AYMN": "medium_airport",
                        "AYMO": "medium_airport",
                        "AYMR": "small_airport",
                        "AYMS": "small_airport",
                        "AYMW": "small_airport",
                        "AYNA": "small_airport",
                        "AYNE": "small_airport",
                        "AYNG": "small_airport",
                        "AYNM": "small_airport",
                        "AYNO": "small_airport",
                        "AYNY": "small_airport",
                        "AYNZ": "medium_airport",
                        "AYOE": "small_airport",
                        "AYOJ": "small_airport",
                        "AYOK": "small_airport",
                        "AYOQ": "small_airport",
                        "AYOW": "small_airport",
                        "AYOY": "small_airport",
                        "AYPC": "small_airport",
                        "AYPG": "small_airport",
                        "AYPO": "small_airport",
                        "AYPQ": "small_airport",
                        "AYPY": "large_airport",
                        "AYQO": "small_airport",
                        "AYQS": "small_airport",
                        "AYRG": "small_airport",
                        "AYRI": "small_airport",
                        "AYRM": "small_airport",
                        "AYRV": "small_airport",
                        "AYSA": "small_airport",
                        "AYSE": "small_airport",
                        "AYSG": "small_airport",
                        "AYSH": "small_airport",
                        "AYSJ": "small_airport",
                        "AYSO": "small_airport",
                        "AYSQ": "small_airport",
                        "AYSV": "small_airport",
                        "AYSX": "small_airport",
                        "AYTA": "small_airport",
                        "AYTB": "small_airport",
                        "AYTI": "small_airport",
                        "AYTK": "medium_airport",
                        "AYTN": "small_airport",
                        "AYTP": "small_airport",
                        "AYTS": "small_airport",
                        "AYTT": "small_airport",
                        "AYTW": "small_airport",
                        "AYTZ": "small_airport",
                        "AYU": "small_airport",
                        "AYUC": "small_airport",
                        "AYUE": "small_airport",
                        "AYUR": "small_airport",
                        "AYUZ": "small_airport",
                        "AYVL": "small_airport",
                        "AYVM": "small_airport",
                        "AYVN": "medium_airport",
                        "AYVO": "small_airport",
                        "AYWB": "small_airport",
                        "AYWD": "medium_airport",
                        "AYWF": "small_airport",
                        "AYWK": "medium_airport",
                        "AYWO": "small_airport",
                        "AYWQ": "small_airport",
                        "AYWS": "small_airport",
                        "AYWT": "small_airport",
                        "AYWU": "small_airport",
                        "AYXE": "small_airport",
                        "AYXI": "small_airport",
                        "AYXW": "small_airport",
                        "AYY": "seaplane_base",
                        "AYYE": "small_airport",
                        "AYYR": "small_airport",
                        "AYZ": "closed",
                        "AYZM": "small_airport",
                        "AZ-0001": "medium_airport",
                        "AZ15": "heliport",
                        "AZB": "small_airport",
                        "BAJ": "small_airport",
                        "BAP": "closed",
                        "BBG": "small_airport",
                        "BCJ": "closed",
                        "BCP": "small_airport",
                        "BCW": "medium_airport",
                        "BCZ": "small_airport",
                        "BDH": "small_airport",
                        "BDZ": "small_airport",
                        "BEA": "closed",
                        "BER": "closed",
                        "BG-HKV": "small_airport",
                        "BG-JAM": "medium_airport",
                        "BGAA": "medium_airport",
                        "BGAG": "heliport",
                        "BGAM": "heliport",
                        "BGAP": "heliport",
                        "BGAT": "heliport",
                        "BGBW": "medium_airport",
                        "BGCH": "heliport",
                        "BGCO": "small_airport",
                        "BGDU": "closed",
                        "BGET": "heliport",
                        "BGGH": "medium_airport",
                        "BGGN": "heliport",
                        "BGIG": "heliport",
                        "BGIK": "heliport",
                        "BGIL": "heliport",
                        "BGIN": "heliport",
                        "BGIS": "heliport",
                        "BGIT": "heliport",
                        "BGJH": "heliport",
                        "BGJN": "small_airport",
                        "BGKA": "heliport",
                        "BGKK": "small_airport",
                        "BGKM": "heliport",
                        "BGKQ": "heliport",
                        "BGKS": "heliport",
                        "BGKT": "heliport",
                        "BGMQ": "small_airport",
                        "BGNN": "heliport",
                        "BGNS": "heliport",
                        "BGNT": "heliport",
                        "BGNU": "heliport",
                        "BGP": "small_airport",
                        "BGPT": "small_airport",
                        "BGQE": "heliport",
                        "BGQQ": "small_airport",
                        "BGSC": "heliport",
                        "BGSF": "medium_airport",
                        "BGSI": "heliport",
                        "BGSO": "heliport",
                        "BGSS": "medium_airport",
                        "BGSV": "heliport",
                        "BGTA": "heliport",
                        "BGTL": "medium_airport",
                        "BGTN": "heliport",
                        "BGUK": "small_airport",
                        "BGUM": "heliport",
                        "BGUQ": "small_airport",
                        "BGUT": "heliport",
                        "BHL": "small_airport",
                        "BHT": "small_airport",
                        "BIAR": "medium_airport",
                        "BIBD": "small_airport",
                        "BIBF": "small_airport",
                        "BIBK": "small_airport",
                        "BIBL": "small_airport",
                        "BIBR": "small_airport",
                        "BIBV": "small_airport",
                        "BIDV": "small_airport",
                        "BIEG": "medium_airport",
                        "BIFF": "small_airport",
                        "BIFM": "small_airport",
                        "BIGF": "small_airport",
                        "BIGJ": "small_airport",
                        "BIGR": "small_airport",
                        "BIHK": "small_airport",
                        "BIHN": "medium_airport",
                        "BIHT": "small_airport",
                        "BIHU": "medium_airport",
                        "BIHV": "small_airport",
                        "BIID": "small_airport",
                        "BIIS": "medium_airport",
                        "BIJ": "closed",
                        "BIKF": "large_airport",
                        "BIKP": "small_airport",
                        "BIKR": "small_airport",
                        "BINF": "small_airport",
                        "BIOF": "small_airport",
                        "BIPA": "small_airport",
                        "BIRE": "small_airport",
                        "BIRF": "small_airport",
                        "BIRG": "small_airport",
                        "BIRK": "medium_airport",
                        "BIRL": "small_airport",
                        "BISI": "medium_airport",
                        "BIST": "small_airport",
                        "BITE": "small_airport",
                        "BITN": "small_airport",
                        "BIVM": "medium_airport",
                        "BIVO": "small_airport",
                        "BIZ": "closed",
                        "BJE": "small_airport",
                        "BJQ": "small_airport",
                        "BJT": "seaplane_base",
                        "BKPR": "large_airport",
                        "BMH": "small_airport",
                        "BMQ": "small_airport",
                        "BMZ": "small_airport",
                        "BNF": "seaplane_base",
                        "BNM": "small_airport",
                        "BNQ": "closed",
                        "BNT": "small_airport",
                        "BNV": "closed",
                        "BNZ": "closed",
                        "BO-0001": "small_airport",
                        "BO-BVL": "small_airport",
                        "BOF": "heliport",
                        "BOK": "small_airport",
                        "BOQ": "closed",
                        "BOT": "small_airport",
                        "BOV": "small_airport",
                        "BPF": "small_airport",
                        "BQV": "seaplane_base",
                        "BR-0015": "closed",
                        "BR-ALT": "small_airport",
                        "BR-SWM": "closed",
                        "BRG": "closed",
                        "BS-NSB": "seaplane_base",
                        "BSP": "small_airport",
                        "BSV": "small_airport",
                        "BUL": "small_airport",
                        "BVP": "closed",
                        "BVR": "small_airport",
                        "BW-HUK": "small_airport",
                        "BWJ": "small_airport",
                        "BWP": "small_airport",
                        "BXL": "seaplane_base",
                        "BXZ": "small_airport",
                        "BYA": "small_airport",
                        "BYL": "small_airport",
                        "BYV": "seaplane_base",
                        "BZ-BCV": "small_airport",
                        "BZ-BGK": "medium_airport",
                        "BZ-CUK": "medium_airport",
                        "BZ-CYC": "medium_airport",
                        "BZ-CZH": "medium_airport",
                        "BZ-DGA": "medium_airport",
                        "BZ-INB": "small_airport",
                        "BZ-MDB": "small_airport",
                        "BZ-ORZ": "small_airport",
                        "BZ-PLJ": "medium_airport",
                        "BZ-PND": "medium_airport",
                        "BZ-SJX": "medium_airport",
                        "BZ-SPR": "medium_airport",
                        "BZ-SQS": "small_airport",
                        "BZ-STU": "small_airport",
                        "BZ-SVK": "small_airport",
                        "BZ-TZA": "medium_airport",
                        "BZB": "medium_airport",
                        "BZM": "small_airport",
                        "BZS": "closed",
                        "CA-0053": "closed",
                        "CA-0095": "closed",
                        "CA-0112": "closed",
                        "CA-0201": "closed",
                        "CA-0213": "closed",
                        "CA-0241": "closed",
                        "CA-0245": "closed",
                        "CA-0274": "closed",
                        "CA-0313": "closed",
                        "CA-0322": "closed",
                        "CA-0378": "closed",
                        "CA-0412": "closed",
                        "CA-0446": "small_airport",
                        "CA-0504": "small_airport",
                        "CAB3": "seaplane_base",
                        "CAB4": "seaplane_base",
                        "CAB5": "closed",
                        "CAC3": "closed",
                        "CAC8": "seaplane_base",
                        "CAD4": "small_airport",
                        "CAD5": "small_airport",
                        "CAE3": "seaplane_base",
                        "CAE5": "seaplane_base",
                        "CAE9": "seaplane_base",
                        "CAF6": "seaplane_base",
                        "CAG3": "small_airport",
                        "CAG8": "seaplane_base",
                        "CAH2": "seaplane_base",
                        "CAH3": "small_airport",
                        "CAJ3": "small_airport",
                        "CAJ4": "small_airport",
                        "CAL3": "small_airport",
                        "CAL4": "small_airport",
                        "CAL9": "seaplane_base",
                        "CAM3": "small_airport",
                        "CAN8": "closed",
                        "CAP3": "small_airport",
                        "CAP7": "seaplane_base",
                        "CAQ8": "seaplane_base",
                        "CAT4": "small_airport",
                        "CAT5": "small_airport",
                        "CAU8": "seaplane_base",
                        "CAV5": "seaplane_base",
                        "CAV7": "seaplane_base",
                        "CAW7": "closed",
                        "CAW8": "seaplane_base",
                        "CAX6": "seaplane_base",
                        "CAY4": "seaplane_base",
                        "CAZ5": "small_airport",
                        "CBBC": "small_airport",
                        "CBC": "small_airport",
                        "CBM5": "closed",
                        "CBS8": "small_airport",
                        "CBW4": "small_airport",
                        "CBX5": "small_airport",
                        "CBX7": "small_airport",
                        "CCA6": "small_airport",
                        "CCD4": "small_airport",
                        "CCE4": "small_airport",
                        "CCK4": "small_airport",
                        "CCP4": "small_airport",
                        "CCZ2": "small_airport",
                        "CDK2": "small_airport",
                        "CDL7": "small_airport",
                        "CEB5": "small_airport",
                        "CEC4": "small_airport",
                        "CEM3": "small_airport",
                        "CEQ5": "small_airport",
                        "CER3": "small_airport",
                        "CER4": "small_airport",
                        "CET2": "small_airport",
                        "CEW3": "small_airport",
                        "CFA7": "small_airport",
                        "CFC4": "small_airport",
                        "CFF4": "small_airport",
                        "CFG6": "small_airport",
                        "CFJ2": "small_airport",
                        "CFM4": "small_airport",
                        "CFQ4": "small_airport",
                        "CG-OKG": "small_airport",
                        "CGA": "seaplane_base",
                        "CGC": "small_airport",
                        "CGG": "small_airport",
                        "CGK2": "small_airport",
                        "CGT": "closed",
                        "CHP": "small_airport",
                        "CIV": "seaplane_base",
                        "CJC8": "small_airport",
                        "CJL2": "small_airport",
                        "CJL8": "small_airport",
                        "CJN8": "seaplane_base",
                        "CJS3": "small_airport",
                        "CJT3": "small_airport",
                        "CJV7": "small_airport",
                        "CJX": "small_airport",
                        "CJY3": "small_airport",
                        "CKB6": "small_airport",
                        "CKL3": "small_airport",
                        "CKM8": "small_airport",
                        "CKQ3": "small_airport",
                        "CKR": "small_airport",
                        "CKU": "small_airport",
                        "CKV4": "small_airport",
                        "CKX": "small_airport",
                        "CLC": "closed",
                        "CLG": "closed",
                        "CMT": "small_airport",
                        "CMZ": "small_airport",
                        "CN-0005": "small_airport",
                        "CN-0010": "medium_airport",
                        "CN-0011": "medium_airport",
                        "CN-0012": "medium_airport",
                        "CN-0024": "medium_airport",
                        "CN-0027": "medium_airport",
                        "CN-0034": "small_airport",
                        "CN-0040": "medium_airport",
                        "CN-0046": "medium_airport",
                        "CN-0050": "medium_airport",
                        "CN-0051": "small_airport",
                        "CN-0061": "medium_airport",
                        "CN-0062": "small_airport",
                        "CN-0067": "medium_airport",
                        "CN-0068": "medium_airport",
                        "CN-0079": "medium_airport",
                        "CN-0081": "medium_airport",
                        "CN-0082": "medium_airport",
                        "CN-0091": "medium_airport",
                        "CN-0104": "medium_airport",
                        "CN-0113": "medium_airport",
                        "CN-0130": "medium_airport",
                        "CN-0143": "medium_airport",
                        "CN-0189": "medium_airport",
                        "CN-NTG": "small_airport",
                        "CNE3": "small_airport",
                        "CNH 2.00": "small_airport",
                        "CNJ8": "seaplane_base",
                        "CNK4": "small_airport",
                        "CNL3": "small_airport",
                        "CNM5": "small_airport",
                        "CNT3": "small_airport",
                        "CO-0039": "heliport",
                        "CO-ARQ": "small_airport",
                        "CO-LCR": "small_airport",
                        "CO-SNT": "small_airport",
                        "CO-TCD": "small_airport",
                        "CPF2": "small_airport",
                        "CPI": "small_airport",
                        "CPS9": "seaplane_base",
                        "CPV7": "small_airport",
                        "CPV8": "small_airport",
                        "CSB2": "small_airport",
                        "CSH4": "small_airport",
                        "CSK 6.00": "small_airport",
                        "CSR8": "small_airport",
                        "CSU2": "small_airport",
                        "CT88": "heliport",
                        "CTB6": "small_airport",
                        "CTK6": "small_airport",
                        "CTM3": "seaplane_base",
                        "CTP9": "small_airport",
                        "CTR7": "seaplane_base",
                        "CTT5": "small_airport",
                        "CTU2": "small_airport",
                        "CTU4": "closed",
                        "CTU5": "small_airport",
                        "CU-0003": "small_airport",
                        "CUJ": "small_airport",
                        "CVH": "small_airport",
                        "CVL": "small_airport",
                        "CVR": "closed",
                        "CXC": "small_airport",
                        "CY-0002": "medium_airport",
                        "CY-NIC": "closed",
                        "CYAB": "small_airport",
                        "CYAC": "medium_airport",
                        "CYAD": "small_airport",
                        "CYAG": "medium_airport",
                        "CYAH": "medium_airport",
                        "CYAL": "medium_airport",
                        "CYAM": "medium_airport",
                        "CYAQ": "medium_airport",
                        "CYAS": "medium_airport",
                        "CYAT": "medium_airport",
                        "CYAY": "medium_airport",
                        "CYAZ": "medium_airport",
                        "CYBA": "small_airport",
                        "CYBB": "medium_airport",
                        "CYBC": "medium_airport",
                        "CYBD": "medium_airport",
                        "CYBE": "medium_airport",
                        "CYBF": "medium_airport",
                        "CYBG": "medium_airport",
                        "CYBK": "medium_airport",
                        "CYBL": "medium_airport",
                        "CYBQ": "medium_airport",
                        "CYBR": "medium_airport",
                        "CYBT": "medium_airport",
                        "CYBV": "medium_airport",
                        "CYBX": "medium_airport",
                        "CYCA": "medium_airport",
                        "CYCB": "medium_airport",
                        "CYCC": "medium_airport",
                        "CYCD": "medium_airport",
                        "CYCE": "medium_airport",
                        "CYCG": "medium_airport",
                        "CYCH": "medium_airport",
                        "CYCK": "small_airport",
                        "CYCL": "medium_airport",
                        "CYCN": "medium_airport",
                        "CYCO": "medium_airport",
                        "CYCQ": "medium_airport",
                        "CYCR": "medium_airport",
                        "CYCS": "medium_airport",
                        "CYCT": "small_airport",
                        "CYCV": "closed",
                        "CYCW": "small_airport",
                        "CYCY": "medium_airport",
                        "CYCZ": "small_airport",
                        "CYDA": "medium_airport",
                        "CYDB": "medium_airport",
                        "CYDF": "medium_airport",
                        "CYDL": "small_airport",
                        "CYDM": "medium_airport",
                        "CYDN": "medium_airport",
                        "CYDO": "medium_airport",
                        "CYDP": "medium_airport",
                        "CYDQ": "medium_airport",
                        "CYEG": "large_airport",
                        "CYEK": "medium_airport",
                        "CYEL": "medium_airport",
                        "CYEM": "medium_airport",
                        "CYEN": "medium_airport",
                        "CYER": "medium_airport",
                        "CYET": "medium_airport",
                        "CYEU": "small_airport",
                        "CYEV": "medium_airport",
                        "CYEY": "medium_airport",
                        "CYFA": "medium_airport",
                        "CYFB": "medium_airport",
                        "CYFC": "medium_airport",
                        "CYFE": "medium_airport",
                        "CYFH": "medium_airport",
                        "CYFJ": "medium_airport",
                        "CYFO": "medium_airport",
                        "CYFR": "medium_airport",
                        "CYFS": "medium_airport",
                        "CYFT": "medium_airport",
                        "CYGB": "medium_airport",
                        "CYGH": "medium_airport",
                        "CYGK": "medium_airport",
                        "CYGL": "medium_airport",
                        "CYGM": "medium_airport",
                        "CYGO": "medium_airport",
                        "CYGP": "medium_airport",
                        "CYGQ": "medium_airport",
                        "CYGR": "medium_airport",
                        "CYGT": "medium_airport",
                        "CYGV": "medium_airport",
                        "CYGW": "medium_airport",
                        "CYGX": "medium_airport",
                        "CYGZ": "medium_airport",
                        "CYHA": "medium_airport",
                        "CYHB": "small_airport",
                        "CYHC": "seaplane_base",
                        "CYHD": "medium_airport",
                        "CYHE": "medium_airport",
                        "CYHF": "medium_airport",
                        "CYHH": "medium_airport",
                        "CYHI": "medium_airport",
                        "CYHK": "medium_airport",
                        "CYHM": "medium_airport",
                        "CYHN": "medium_airport",
                        "CYHO": "medium_airport",
                        "CYHR": "medium_airport",
                        "CYHT": "medium_airport",
                        "CYHU": "medium_airport",
                        "CYHY": "medium_airport",
                        "CYHZ": "large_airport",
                        "CYIB": "medium_airport",
                        "CYID": "medium_airport",
                        "CYIF": "medium_airport",
                        "CYIK": "medium_airport",
                        "CYIO": "medium_airport",
                        "CYIV": "medium_airport",
                        "CYJA": "small_airport",
                        "CYJF": "medium_airport",
                        "CYJN": "medium_airport",
                        "CYJT": "medium_airport",
                        "CYKA": "medium_airport",
                        "CYKC": "small_airport",
                        "CYKD": "medium_airport",
                        "CYKF": "medium_airport",
                        "CYKG": "medium_airport",
                        "CYKJ": "medium_airport",
                        "CYKL": "medium_airport",
                        "CYKM": "small_airport",
                        "CYKO": "medium_airport",
                        "CYKQ": "medium_airport",
                        "CYKX": "medium_airport",
                        "CYKY": "medium_airport",
                        "CYKZ": "medium_airport",
                        "CYLA": "medium_airport",
                        "CYLB": "small_airport",
                        "CYLC": "medium_airport",
                        "CYLD": "medium_airport",
                        "CYLH": "medium_airport",
                        "CYLJ": "medium_airport",
                        "CYLK": "medium_airport",
                        "CYLL": "medium_airport",
                        "CYLQ": "small_airport",
                        "CYLR": "medium_airport",
                        "CYLS": "small_airport",
                        "CYLT": "medium_airport",
                        "CYLU": "medium_airport",
                        "CYLW": "medium_airport",
                        "CYM": "seaplane_base",
                        "CYMA": "medium_airport",
                        "CYME": "medium_airport",
                        "CYMG": "medium_airport",
                        "CYMH": "medium_airport",
                        "CYMJ": "medium_airport",
                        "CYML": "medium_airport",
                        "CYMM": "medium_airport",
                        "CYMO": "medium_airport",
                        "CYMT": "medium_airport",
                        "CYMU": "medium_airport",
                        "CYMW": "small_airport",
                        "CYMX": "medium_airport",
                        "CYNA": "medium_airport",
                        "CYNC": "medium_airport",
                        "CYND": "medium_airport",
                        "CYNE": "medium_airport",
                        "CYNH": "small_airport",
                        "CYNJ": "small_airport",
                        "CYNL": "medium_airport",
                        "CYNM": "medium_airport",
                        "CYNR": "small_airport",
                        "CYOA": "medium_airport",
                        "CYOC": "medium_airport",
                        "CYOD": "medium_airport",
                        "CYOH": "medium_airport",
                        "CYOJ": "medium_airport",
                        "CYOO": "medium_airport",
                        "CYOP": "medium_airport",
                        "CYOS": "medium_airport",
                        "CYOW": "large_airport",
                        "CYOY": "heliport",
                        "CYPA": "medium_airport",
                        "CYPC": "medium_airport",
                        "CYPD": "medium_airport",
                        "CYPE": "medium_airport",
                        "CYPG": "medium_airport",
                        "CYPH": "medium_airport",
                        "CYPL": "medium_airport",
                        "CYPM": "medium_airport",
                        "CYPN": "medium_airport",
                        "CYPO": "medium_airport",
                        "CYPQ": "medium_airport",
                        "CYPR": "medium_airport",
                        "CYPW": "medium_airport",
                        "CYPX": "medium_airport",
                        "CYPY": "medium_airport",
                        "CYPZ": "small_airport",
                        "CYQA": "medium_airport",
                        "CYQB": "medium_airport",
                        "CYQD": "medium_airport",
                        "CYQF": "medium_airport",
                        "CYQG": "medium_airport",
                        "CYQH": "medium_airport",
                        "CYQI": "medium_airport",
                        "CYQK": "medium_airport",
                        "CYQL": "medium_airport",
                        "CYQM": "medium_airport",
                        "CYQN": "medium_airport",
                        "CYQQ": "medium_airport",
                        "CYQR": "medium_airport",
                        "CYQS": "medium_airport",
                        "CYQT": "medium_airport",
                        "CYQU": "medium_airport",
                        "CYQV": "medium_airport",
                        "CYQW": "medium_airport",
                        "CYQX": "medium_airport",
                        "CYQY": "medium_airport",
                        "CYQZ": "medium_airport",
                        "CYRA": "medium_airport",
                        "CYRB": "medium_airport",
                        "CYRI": "medium_airport",
                        "CYRJ": "medium_airport",
                        "CYRL": "medium_airport",
                        "CYRM": "small_airport",
                        "CYRO": "medium_airport",
                        "CYRQ": "medium_airport",
                        "CYRS": "medium_airport",
                        "CYRT": "medium_airport",
                        "CYRV": "medium_airport",
                        "CYSB": "medium_airport",
                        "CYSC": "medium_airport",
                        "CYSD": "heliport",
                        "CYSE": "small_airport",
                        "CYSF": "medium_airport",
                        "CYSH": "medium_airport",
                        "CYSJ": "medium_airport",
                        "CYSK": "medium_airport",
                        "CYSL": "medium_airport",
                        "CYSM": "medium_airport",
                        "CYSN": "medium_airport",
                        "CYSP": "medium_airport",
                        "CYST": "medium_airport",
                        "CYSU": "medium_airport",
                        "CYSY": "medium_airport",
                        "CYTA": "medium_airport",
                        "CYTE": "medium_airport",
                        "CYTF": "medium_airport",
                        "CYTH": "medium_airport",
                        "CYTL": "medium_airport",
                        "CYTQ": "medium_airport",
                        "CYTR": "medium_airport",
                        "CYTS": "medium_airport",
                        "CYTZ": "medium_airport",
                        "CYUB": "medium_airport",
                        "CYUL": "large_airport",
                        "CYUT": "medium_airport",
                        "CYUX": "medium_airport",
                        "CYUY": "medium_airport",
                        "CYVB": "medium_airport",
                        "CYVC": "medium_airport",
                        "CYVG": "small_airport",
                        "CYVK": "medium_airport",
                        "CYVL": "small_airport",
                        "CYVM": "medium_airport",
                        "CYVO": "medium_airport",
                        "CYVP": "medium_airport",
                        "CYVQ": "medium_airport",
                        "CYVR": "large_airport",
                        "CYVT": "small_airport",
                        "CYVV": "medium_airport",
                        "CYVZ": "medium_airport",
                        "CYWA": "medium_airport",
                        "CYWG": "large_airport",
                        "CYWH": "seaplane_base",
                        "CYWJ": "medium_airport",
                        "CYWK": "medium_airport",
                        "CYWL": "medium_airport",
                        "CYWP": "medium_airport",
                        "CYWY": "medium_airport",
                        "CYXC": "medium_airport",
                        "CYXD": "closed",
                        "CYXE": "medium_airport",
                        "CYXH": "medium_airport",
                        "CYXI": "closed",
                        "CYXJ": "medium_airport",
                        "CYXK": "medium_airport",
                        "CYXL": "medium_airport",
                        "CYXN": "medium_airport",
                        "CYXP": "medium_airport",
                        "CYXQ": "medium_airport",
                        "CYXR": "medium_airport",
                        "CYXS": "medium_airport",
                        "CYXT": "medium_airport",
                        "CYXU": "medium_airport",
                        "CYXX": "medium_airport",
                        "CYXY": "medium_airport",
                        "CYXZ": "medium_airport",
                        "CYYB": "medium_airport",
                        "CYYC": "large_airport",
                        "CYYD": "medium_airport",
                        "CYYE": "medium_airport",
                        "CYYF": "medium_airport",
                        "CYYG": "medium_airport",
                        "CYYH": "medium_airport",
                        "CYYJ": "large_airport",
                        "CYYL": "medium_airport",
                        "CYYM": "small_airport",
                        "CYYN": "medium_airport",
                        "CYYQ": "medium_airport",
                        "CYYR": "medium_airport",
                        "CYYT": "large_airport",
                        "CYYU": "medium_airport",
                        "CYYW": "medium_airport",
                        "CYYY": "medium_airport",
                        "CYYZ": "large_airport",
                        "CYZE": "medium_airport",
                        "CYZF": "medium_airport",
                        "CYZG": "medium_airport",
                        "CYZH": "medium_airport",
                        "CYZP": "medium_airport",
                        "CYZR": "medium_airport",
                        "CYZS": "medium_airport",
                        "CYZT": "medium_airport",
                        "CYZU": "medium_airport",
                        "CYZV": "medium_airport",
                        "CYZW": "medium_airport",
                        "CYZX": "medium_airport",
                        "CZAC": "medium_airport",
                        "CZAM": "medium_airport",
                        "CZBB": "medium_airport",
                        "CZBD": "medium_airport",
                        "CZBF": "medium_airport",
                        "CZBM": "medium_airport",
                        "CZEE": "medium_airport",
                        "CZEM": "medium_airport",
                        "CZFA": "medium_airport",
                        "CZFD": "medium_airport",
                        "CZFG": "medium_airport",
                        "CZFM": "medium_airport",
                        "CZFN": "medium_airport",
                        "CZGF": "medium_airport",
                        "CZGI": "medium_airport",
                        "CZGR": "medium_airport",
                        "CZHP": "small_airport",
                        "CZJ": "small_airport",
                        "CZJG": "medium_airport",
                        "CZJN": "medium_airport",
                        "CZK": "small_airport",
                        "CZKE": "medium_airport",
                        "CZLQ": "medium_airport",
                        "CZMD": "medium_airport",
                        "CZML": "small_airport",
                        "CZMN": "medium_airport",
                        "CZMT": "medium_airport",
                        "CZN": "small_airport",
                        "CZNG": "medium_airport",
                        "CZO": "small_airport",
                        "CZPB": "medium_airport",
                        "CZPC": "medium_airport",
                        "CZPO": "small_airport",
                        "CZRJ": "medium_airport",
                        "CZSJ": "medium_airport",
                        "CZSN": "medium_airport",
                        "CZST": "medium_airport",
                        "CZSW": "seaplane_base",
                        "CZTA": "small_airport",
                        "CZTM": "medium_airport",
                        "CZUC": "medium_airport",
                        "CZUM": "small_airport",
                        "CZWH": "medium_airport",
                        "CZWL": "medium_airport",
                        "D38": "small_airport",
                        "D66": "small_airport",
                        "DA14": "small_airport",
                        "DAAB": "medium_airport",
                        "DAAD": "medium_airport",
                        "DAAE": "medium_airport",
                        "DAAG": "large_airport",
                        "DAAJ": "medium_airport",
                        "DAAP": "medium_airport",
                        "DAAS": "medium_airport",
                        "DAAT": "medium_airport",
                        "DAAV": "medium_airport",
                        "DAAY": "medium_airport",
                        "DABB": "medium_airport",
                        "DABC": "medium_airport",
                        "DABO": "small_airport",
                        "DABP": "closed",
                        "DABS": "medium_airport",
                        "DABT": "medium_airport",
                        "DAF": "small_airport",
                        "DAFH": "medium_airport",
                        "DAFI": "small_airport",
                        "DAO": "small_airport",
                        "DAOB": "medium_airport",
                        "DAOF": "medium_airport",
                        "DAOI": "medium_airport",
                        "DAOL": "medium_airport",
                        "DAON": "medium_airport",
                        "DAOO": "medium_airport",
                        "DAOR": "medium_airport",
                        "DAOS": "small_airport",
                        "DAOV": "medium_airport",
                        "DAOY": "small_airport",
                        "DATG": "small_airport",
                        "DATM": "small_airport",
                        "DAUA": "medium_airport",
                        "DAUB": "medium_airport",
                        "DAUE": "medium_airport",
                        "DAUG": "medium_airport",
                        "DAUH": "medium_airport",
                        "DAUI": "medium_airport",
                        "DAUK": "medium_airport",
                        "DAUL": "medium_airport",
                        "DAUO": "medium_airport",
                        "DAUT": "medium_airport",
                        "DAUU": "medium_airport",
                        "DAUZ": "medium_airport",
                        "DBBB": "medium_airport",
                        "DBBD": "small_airport",
                        "DBBK": "small_airport",
                        "DBBN": "small_airport",
                        "DBBP": "small_airport",
                        "DBBS": "small_airport",
                        "DBC": "medium_airport",
                        "DBK": "seaplane_base",
                        "DBP": "small_airport",
                        "DBU": "seaplane_base",
                        "DCG": "seaplane_base",
                        "DCK": "small_airport",
                        "DCR": "closed",
                        "DDM": "small_airport",
                        "DE-0419": "closed",
                        "DE-0428": "closed",
                        "DE-0429": "closed",
                        "DER": "small_airport",
                        "DEX": "small_airport",
                        "DFCA": "small_airport",
                        "DFCC": "small_airport",
                        "DFCJ": "small_airport",
                        "DFCL": "small_airport",
                        "DFCP": "small_airport",
                        "DFEA": "small_airport",
                        "DFEB": "small_airport",
                        "DFED": "small_airport",
                        "DFEE": "small_airport",
                        "DFEF": "small_airport",
                        "DFEG": "small_airport",
                        "DFEL": "small_airport",
                        "DFEM": "small_airport",
                        "DFEP": "small_airport",
                        "DFER": "small_airport",
                        "DFES": "small_airport",
                        "DFET": "small_airport",
                        "DFEZ": "small_airport",
                        "DFFD": "large_airport",
                        "DFOB": "small_airport",
                        "DFOD": "small_airport",
                        "DFOG": "small_airport",
                        "DFON": "small_airport",
                        "DFOO": "medium_airport",
                        "DFOT": "small_airport",
                        "DFOU": "small_airport",
                        "DFOY": "small_airport",
                        "DGAA": "large_airport",
                        "DGG": "closed",
                        "DGLE": "medium_airport",
                        "DGM": "seaplane_base",
                        "DGSI": "medium_airport",
                        "DGSN": "medium_airport",
                        "DGTK": "medium_airport",
                        "DHB": "seaplane_base",
                        "DHG": "small_airport",
                        "DIAO": "small_airport",
                        "DIAP": "medium_airport",
                        "DIAU": "small_airport",
                        "DIBI": "small_airport",
                        "DIBK": "medium_airport",
                        "DIBN": "small_airport",
                        "DIBU": "small_airport",
                        "DIDK": "small_airport",
                        "DIDL": "medium_airport",
                        "DIFK": "small_airport",
                        "DIGA": "small_airport",
                        "DIGL": "small_airport",
                        "DIGN": "small_airport",
                        "DIKO": "medium_airport",
                        "DIMN": "medium_airport",
                        "DIOD": "small_airport",
                        "DIOF": "small_airport",
                        "DISG": "small_airport",
                        "DISP": "medium_airport",
                        "DISS": "small_airport",
                        "DITB": "small_airport",
                        "DITN": "small_airport",
                        "DIW": "seaplane_base",
                        "DIYO": "medium_airport",
                        "DLR": "small_airport",
                        "DN57": "small_airport",
                        "DNAA": "large_airport",
                        "DNAI": "large_airport",
                        "DNAK": "medium_airport",
                        "DNAS": "medium_airport",
                        "DNBC": "medium_airport",
                        "DNBE": "medium_airport",
                        "DNCA": "medium_airport",
                        "DNEN": "medium_airport",
                        "DNIB": "medium_airport",
                        "DNIL": "medium_airport",
                        "DNIM": "medium_airport",
                        "DNJO": "medium_airport",
                        "DNKA": "medium_airport",
                        "DNKN": "large_airport",
                        "DNMA": "medium_airport",
                        "DNMK": "medium_airport",
                        "DNMM": "large_airport",
                        "DNMN": "medium_airport",
                        "DNPO": "medium_airport",
                        "DNSO": "medium_airport",
                        "DNYO": "medium_airport",
                        "DNZA": "medium_airport",
                        "DOO": "small_airport",
                        "DOS": "closed",
                        "DPK": "closed",
                        "DPT": "small_airport",
                        "DPU": "closed",
                        "DQA": "small_airport",
                        "DRRM": "medium_airport",
                        "DRRN": "large_airport",
                        "DRRT": "medium_airport",
                        "DRZA": "medium_airport",
                        "DRZL": "small_airport",
                        "DRZR": "medium_airport",
                        "DSG": "small_airport",
                        "DTKA": "medium_airport",
                        "DTMB": "medium_airport",
                        "DTTA": "large_airport",
                        "DTTF": "medium_airport",
                        "DTTG": "medium_airport",
                        "DTTJ": "medium_airport",
                        "DTTR": "medium_airport",
                        "DTTX": "medium_airport",
                        "DTTZ": "medium_airport",
                        "DVD": "small_airport",
                        "DWO": "seaplane_base",
                        "DWR": "small_airport",
                        "DWS": "closed",
                        "DXNG": "medium_airport",
                        "DXXX": "medium_airport",
                        "EAL": "small_airport",
                        "EBAW": "medium_airport",
                        "EBBR": "large_airport",
                        "EBCI": "large_airport",
                        "EBKT": "medium_airport",
                        "EBLG": "large_airport",
                        "EBO": "small_airport",
                        "EBOS": "medium_airport",
                        "EBZR": "small_airport",
                        "EC-MZD": "small_airport",
                        "EDA": "seaplane_base",
                        "EDAC": "medium_airport",
                        "EDAH": "medium_airport",
                        "EDAU": "small_airport",
                        "EDAX": "small_airport",
                        "EDBC": "small_airport",
                        "EDBH": "small_airport",
                        "EDBM": "medium_airport",
                        "EDBN": "medium_airport",
                        "EDCD": "closed",
                        "EDCG": "small_airport",
                        "EDCK": "small_airport",
                        "EDCP": "small_airport",
                        "EDDB": "large_airport",
                        "EDDC": "large_airport",
                        "EDDE": "medium_airport",
                        "EDDF": "large_airport",
                        "EDDG": "large_airport",
                        "EDDH": "large_airport",
                        "EDDI": "closed",
                        "EDDK": "large_airport",
                        "EDDL": "large_airport",
                        "EDDM": "large_airport",
                        "EDDN": "large_airport",
                        "EDDP": "large_airport",
                        "EDDR": "medium_airport",
                        "EDDS": "large_airport",
                        "EDDT": "large_airport",
                        "EDDV": "large_airport",
                        "EDDW": "large_airport",
                        "EDFH": "medium_airport",
                        "EDFM": "medium_airport",
                        "EDGE": "medium_airport",
                        "EDGS": "medium_airport",
                        "EDHI": "medium_airport",
                        "EDHK": "medium_airport",
                        "EDHL": "medium_airport",
                        "EDHN": "small_airport",
                        "EDJA": "medium_airport",
                        "EDKA": "small_airport",
                        "EDKB": "small_airport",
                        "EDLE": "small_airport",
                        "EDLI": "small_airport",
                        "EDLN": "medium_airport",
                        "EDLP": "medium_airport",
                        "EDLV": "medium_airport",
                        "EDLW": "large_airport",
                        "EDMA": "medium_airport",
                        "EDMO": "medium_airport",
                        "EDMS": "small_airport",
                        "EDNY": "medium_airport",
                        "EDOP": "medium_airport",
                        "EDQD": "medium_airport",
                        "EDQE": "small_airport",
                        "EDQG": "small_airport",
                        "EDQM": "medium_airport",
                        "EDRB": "small_airport",
                        "EDRZ": "small_airport",
                        "EDSB": "large_airport",
                        "EDTD": "medium_airport",
                        "EDTL": "medium_airport",
                        "EDVE": "medium_airport",
                        "EDVK": "medium_airport",
                        "EDWB": "closed",
                        "EDWD": "closed",
                        "EDWE": "small_airport",
                        "EDWG": "small_airport",
                        "EDWI": "small_airport",
                        "EDWJ": "small_airport",
                        "EDWL": "small_airport",
                        "EDWR": "small_airport",
                        "EDWS": "small_airport",
                        "EDWU": "small_airport",
                        "EDWY": "small_airport",
                        "EDWZ": "small_airport",
                        "EDXB": "small_airport",
                        "EDXF": "small_airport",
                        "EDXH": "small_airport",
                        "EDXJ": "small_airport",
                        "EDXO": "small_airport",
                        "EDXW": "medium_airport",
                        "EDXY": "small_airport",
                        "EEKA": "medium_airport",
                        "EEKE": "medium_airport",
                        "EEPU": "medium_airport",
                        "EETN": "large_airport",
                        "EETU": "medium_airport",
                        "EFET": "medium_airport",
                        "EFG": "small_airport",
                        "EFHA": "medium_airport",
                        "EFHF": "medium_airport",
                        "EFHK": "large_airport",
                        "EFHV": "small_airport",
                        "EFIT": "medium_airport",
                        "EFIV": "medium_airport",
                        "EFJO": "medium_airport",
                        "EFJY": "medium_airport",
                        "EFKA": "medium_airport",
                        "EFKE": "medium_airport",
                        "EFKI": "medium_airport",
                        "EFKJ": "medium_airport",
                        "EFKK": "medium_airport",
                        "EFKS": "medium_airport",
                        "EFKT": "medium_airport",
                        "EFKU": "medium_airport",
                        "EFLA": "small_airport",
                        "EFLP": "medium_airport",
                        "EFMA": "medium_airport",
                        "EFMI": "medium_airport",
                        "EFOU": "medium_airport",
                        "EFPO": "medium_airport",
                        "EFRO": "medium_airport",
                        "EFSA": "medium_airport",
                        "EFSI": "medium_airport",
                        "EFSO": "medium_airport",
                        "EFTP": "medium_airport",
                        "EFTU": "medium_airport",
                        "EFUT": "medium_airport",
                        "EFVA": "medium_airport",
                        "EFVR": "medium_airport",
                        "EFYL": "medium_airport",
                        "EG-AUE": "small_airport",
                        "EGAA": "large_airport",
                        "EGAB": "medium_airport",
                        "EGAC": "large_airport",
                        "EGAE": "medium_airport",
                        "EGBB": "large_airport",
                        "EGBE": "medium_airport",
                        "EGBJ": "medium_airport",
                        "EGBK": "small_airport",
                        "EGBN": "medium_airport",
                        "EGBP": "small_airport",
                        "EGCC": "large_airport",
                        "EGCN": "large_airport",
                        "EGDJ": "small_airport",
                        "EGDL": "closed",
                        "EGDY": "medium_airport",
                        "EGEC": "medium_airport",
                        "EGED": "medium_airport",
                        "EGEF": "medium_airport",
                        "EGEH": "small_airport",
                        "EGEL": "small_airport",
                        "EGEN": "small_airport",
                        "EGEO": "small_airport",
                        "EGEP": "small_airport",
                        "EGER": "small_airport",
                        "EGES": "small_airport",
                        "EGET": "small_airport",
                        "EGEW": "small_airport",
                        "EGEY": "small_airport",
                        "EGFE": "medium_airport",
                        "EGFF": "large_airport",
                        "EGFH": "medium_airport",
                        "EGGD": "large_airport",
                        "EGGP": "large_airport",
                        "EGGW": "large_airport",
                        "EGHC": "medium_airport",
                        "EGHD": "closed",
                        "EGHE": "medium_airport",
                        "EGHH": "large_airport",
                        "EGHI": "large_airport",
                        "EGHJ": "small_airport",
                        "EGHK": "closed",
                        "EGHQ": "medium_airport",
                        "EGHR": "small_airport",
                        "EGJA": "medium_airport",
                        "EGJB": "medium_airport",
                        "EGJJ": "medium_airport",
                        "EGKA": "medium_airport",
                        "EGKB": "medium_airport",
                        "EGKK": "large_airport",
                        "EGKM": "closed",
                        "EGKR": "small_airport",
                        "EGLC": "medium_airport",
                        "EGLF": "medium_airport",
                        "EGLK": "medium_airport",
                        "EGLL": "large_airport",
                        "EGMC": "medium_airport",
                        "EGMD": "medium_airport",
                        "EGMH": "closed",
                        "EGMK": "closed",
                        "EGNC": "medium_airport",
                        "EGNH": "medium_airport",
                        "EGNJ": "medium_airport",
                        "EGNL": "medium_airport",
                        "EGNM": "large_airport",
                        "EGNO": "medium_airport",
                        "EGNR": "medium_airport",
                        "EGNS": "medium_airport",
                        "EGNT": "large_airport",
                        "EGNV": "medium_airport",
                        "EGNX": "large_airport",
                        "EGOV": "medium_airport",
                        "EGPA": "medium_airport",
                        "EGPB": "medium_airport",
                        "EGPC": "medium_airport",
                        "EGPD": "large_airport",
                        "EGPE": "medium_airport",
                        "EGPF": "large_airport",
                        "EGPH": "large_airport",
                        "EGPI": "medium_airport",
                        "EGPK": "medium_airport",
                        "EGPL": "medium_airport",
                        "EGPM": "medium_airport",
                        "EGPN": "medium_airport",
                        "EGPO": "medium_airport",
                        "EGPR": "medium_airport",
                        "EGPT": "small_airport",
                        "EGPU": "medium_airport",
                        "EGPW": "small_airport",
                        "EGQK": "medium_airport",
                        "EGQL": "medium_airport",
                        "EGQS": "medium_airport",
                        "EGSC": "medium_airport",
                        "EGSH": "large_airport",
                        "EGSS": "large_airport",
                        "EGTB": "small_airport",
                        "EGTE": "large_airport",
                        "EGTG": "closed",
                        "EGTK": "medium_airport",
                        "EGTO": "small_airport",
                        "EGUA": "closed",
                        "EGUB": "medium_airport",
                        "EGUL": "large_airport",
                        "EGUN": "large_airport",
                        "EGUY": "medium_airport",
                        "EGVA": "large_airport",
                        "EGVF": "heliport",
                        "EGVJ": "closed",
                        "EGVN": "large_airport",
                        "EGVO": "medium_airport",
                        "EGWU": "medium_airport",
                        "EGWZ": "closed",
                        "EGXB": "closed",
                        "EGXC": "medium_airport",
                        "EGXH": "medium_airport",
                        "EGXJ": "closed",
                        "EGXP": "medium_airport",
                        "EGXU": "medium_airport",
                        "EGXW": "medium_airport",
                        "EGYC": "closed",
                        "EGYM": "medium_airport",
                        "EGYP": "medium_airport",
                        "EHAM": "large_airport",
                        "EHBK": "medium_airport",
                        "EHEH": "large_airport",
                        "EHGG": "medium_airport",
                        "EHGR": "medium_airport",
                        "EHKD": "medium_airport",
                        "EHLE": "medium_airport",
                        "EHLW": "medium_airport",
                        "EHRD": "medium_airport",
                        "EHSB": "closed",
                        "EHTW": "medium_airport",
                        "EHVB": "closed",
                        "EHWO": "medium_airport",
                        "EIBN": "small_airport",
                        "EIBT": "small_airport",
                        "EICA": "small_airport",
                        "EICB": "closed",
                        "EICK": "large_airport",
                        "EICM": "medium_airport",
                        "EIDL": "medium_airport",
                        "EIDW": "large_airport",
                        "EIIM": "small_airport",
                        "EIIR": "small_airport",
                        "EIKK": "small_airport",
                        "EIKN": "medium_airport",
                        "EIKY": "medium_airport",
                        "EILT": "small_airport",
                        "EIMN": "small_airport",
                        "EINN": "large_airport",
                        "EISG": "medium_airport",
                        "EIWF": "medium_airport",
                        "EJN": "small_airport",
                        "EJT": "small_airport",
                        "EKAH": "medium_airport",
                        "EKBI": "large_airport",
                        "EKCH": "large_airport",
                        "EKEB": "medium_airport",
                        "EKKA": "medium_airport",
                        "EKLS": "small_airport",
                        "EKMB": "medium_airport",
                        "EKOD": "medium_airport",
                        "EKRK": "medium_airport",
                        "EKRN": "medium_airport",
                        "EKSB": "medium_airport",
                        "EKSN": "medium_airport",
                        "EKSP": "medium_airport",
                        "EKSV": "medium_airport",
                        "EKTS": "medium_airport",
                        "EKVG": "medium_airport",
                        "EKVJ": "medium_airport",
                        "EKYT": "large_airport",
                        "ELLX": "large_airport",
                        "EMR": "heliport",
                        "ENAL": "medium_airport",
                        "ENAN": "medium_airport",
                        "ENAT": "medium_airport",
                        "ENBL": "small_airport",
                        "ENBN": "medium_airport",
                        "ENBO": "large_airport",
                        "ENBR": "large_airport",
                        "ENBS": "medium_airport",
                        "ENBV": "medium_airport",
                        "ENCN": "medium_airport",
                        "ENDI": "small_airport",
                        "ENDU": "medium_airport",
                        "ENEV": "medium_airport",
                        "ENFB": "closed",
                        "ENFG": "medium_airport",
                        "ENFL": "medium_airport",
                        "ENGM": "large_airport",
                        "ENHA": "small_airport",
                        "ENHD": "medium_airport",
                        "ENHF": "medium_airport",
                        "ENHK": "small_airport",
                        "ENHV": "medium_airport",
                        "ENKA": "small_airport",
                        "ENKB": "medium_airport",
                        "ENKL": "small_airport",
                        "ENKR": "medium_airport",
                        "ENLI": "small_airport",
                        "ENLK": "medium_airport",
                        "ENMH": "medium_airport",
                        "ENML": "medium_airport",
                        "ENMS": "medium_airport",
                        "ENNA": "medium_airport",
                        "ENNK": "medium_airport",
                        "ENNM": "small_airport",
                        "ENNO": "medium_airport",
                        "ENOL": "medium_airport",
                        "ENOV": "medium_airport",
                        "ENRA": "medium_airport",
                        "ENRM": "medium_airport",
                        "ENRO": "medium_airport",
                        "ENRS": "small_airport",
                        "ENRY": "medium_airport",
                        "ENSB": "medium_airport",
                        "ENSD": "small_airport",
                        "ENSG": "small_airport",
                        "ENSH": "medium_airport",
                        "ENSK": "medium_airport",
                        "ENSN": "medium_airport",
                        "ENSO": "medium_airport",
                        "ENSR": "medium_airport",
                        "ENSS": "medium_airport",
                        "ENST": "medium_airport",
                        "ENTC": "large_airport",
                        "ENTO": "medium_airport",
                        "ENVA": "large_airport",
                        "ENVD": "medium_airport",
                        "ENVR": "heliport",
                        "ENZV": "large_airport",
                        "EPBP": "medium_airport",
                        "EPBY": "medium_airport",
                        "EPCH": "small_airport",
                        "EPGD": "large_airport",
                        "EPKK": "large_airport",
                        "EPKO": "small_airport",
                        "EPKT": "large_airport",
                        "EPKW": "small_airport",
                        "EPLL": "medium_airport",
                        "EPMO": "large_airport",
                        "EPOK": "medium_airport",
                        "EPPO": "large_airport",
                        "EPRA": "medium_airport",
                        "EPRZ": "medium_airport",
                        "EPSC": "medium_airport",
                        "EPSK": "closed",
                        "EPSY": "medium_airport",
                        "EPWA": "large_airport",
                        "EPWR": "large_airport",
                        "EPZG": "medium_airport",
                        "ERT": "small_airport",
                        "ESDF": "medium_airport",
                        "ESGG": "large_airport",
                        "ESGJ": "medium_airport",
                        "ESGL": "small_airport",
                        "ESGP": "medium_airport",
                        "ESGR": "medium_airport",
                        "ESGT": "medium_airport",
                        "ESKK": "medium_airport",
                        "ESKM": "medium_airport",
                        "ESKN": "medium_airport",
                        "ESMK": "medium_airport",
                        "ESMO": "small_airport",
                        "ESMQ": "medium_airport",
                        "ESMS": "large_airport",
                        "ESMT": "medium_airport",
                        "ESMX": "medium_airport",
                        "ESND": "medium_airport",
                        "ESNG": "medium_airport",
                        "ESNH": "closed",
                        "ESNK": "medium_airport",
                        "ESNL": "medium_airport",
                        "ESNN": "medium_airport",
                        "ESNO": "medium_airport",
                        "ESNQ": "medium_airport",
                        "ESNS": "medium_airport",
                        "ESNU": "medium_airport",
                        "ESNV": "medium_airport",
                        "ESNX": "medium_airport",
                        "ESNY": "medium_airport",
                        "ESNZ": "medium_airport",
                        "ESOE": "medium_airport",
                        "ESOH": "medium_airport",
                        "ESOK": "medium_airport",
                        "ESOW": "medium_airport",
                        "ESP": "closed",
                        "ESPA": "large_airport",
                        "ESSA": "large_airport",
                        "ESSB": "medium_airport",
                        "ESSD": "medium_airport",
                        "ESSF": "small_airport",
                        "ESSK": "medium_airport",
                        "ESSL": "medium_airport",
                        "ESSP": "medium_airport",
                        "ESST": "medium_airport",
                        "ESSU": "medium_airport",
                        "ESSV": "medium_airport",
                        "ESSW": "small_airport",
                        "ESTA": "medium_airport",
                        "ESUD": "medium_airport",
                        "ESUE": "small_airport",
                        "ESUP": "medium_airport",
                        "ESUT": "medium_airport",
                        "ET-GLC": "small_airport",
                        "ET-SHC": "small_airport",
                        "ETAD": "medium_airport",
                        "ETAR": "large_airport",
                        "ETAS": "closed",
                        "ETHF": "medium_airport",
                        "ETIK": "heliport",
                        "ETIN": "small_airport",
                        "ETMN": "medium_airport",
                        "ETNG": "medium_airport",
                        "ETNL": "medium_airport",
                        "ETNS": "medium_airport",
                        "ETOU": "medium_airport",
                        "ETSF": "closed",
                        "ETSI": "medium_airport",
                        "ETUO": "closed",
                        "EVDA": "medium_airport",
                        "EVLA": "medium_airport",
                        "EVRA": "large_airport",
                        "EVVA": "medium_airport",
                        "EWY": "closed",
                        "EXI": "seaplane_base",
                        "EYKA": "medium_airport",
                        "EYPA": "medium_airport",
                        "EYPP": "medium_airport",
                        "EYSA": "medium_airport",
                        "EYVI": "large_airport",
                        "F23": "small_airport",
                        "FAAB": "medium_airport",
                        "FAAG": "medium_airport",
                        "FAAL": "small_airport",
                        "FABE": "medium_airport",
                        "FABL": "medium_airport",
                        "FABU": "small_airport",
                        "FACC": "small_airport",
                        "FACD": "small_airport",
                        "FACT": "large_airport",
                        "FADK": "small_airport",
                        "FADQ": "small_airport",
                        "FAEL": "medium_airport",
                        "FAEM": "small_airport",
                        "FAER": "small_airport",
                        "FAFB": "medium_airport",
                        "FAGC": "medium_airport",
                        "FAGG": "large_airport",
                        "FAGI": "small_airport",
                        "FAGM": "medium_airport",
                        "FAHL": "small_airport",
                        "FAHR": "medium_airport",
                        "FAHS": "small_airport",
                        "FAKD": "medium_airport",
                        "FAKM": "medium_airport",
                        "FAKN": "medium_airport",
                        "FAKP": "small_airport",
                        "FAKU": "medium_airport",
                        "FAKZ": "medium_airport",
                        "FAL": "closed",
                        "FALA": "medium_airport",
                        "FALC": "medium_airport",
                        "FALD": "small_airport",
                        "FALE": "large_airport",
                        "FALK": "small_airport",
                        "FALO": "small_airport",
                        "FALW": "medium_airport",
                        "FALY": "medium_airport",
                        "FAMD": "medium_airport",
                        "FAMG": "medium_airport",
                        "FAMH": "small_airport",
                        "FAMM": "medium_airport",
                        "FAMN": "medium_airport",
                        "FAMO": "small_airport",
                        "FAMU": "medium_airport",
                        "FANC": "medium_airport",
                        "FANG": "small_airport",
                        "FANS": "small_airport",
                        "FAOB": "small_airport",
                        "FAOH": "medium_airport",
                        "FAOR": "large_airport",
                        "FAPA": "small_airport",
                        "FAPE": "medium_airport",
                        "FAPG": "medium_airport",
                        "FAPH": "medium_airport",
                        "FAPJ": "medium_airport",
                        "FAPK": "small_airport",
                        "FAPM": "medium_airport",
                        "FAPN": "small_airport",
                        "FAPP": "medium_airport",
                        "FAPS": "medium_airport",
                        "FAQT": "medium_airport",
                        "FARB": "medium_airport",
                        "FARI": "small_airport",
                        "FARS": "medium_airport",
                        "FASB": "medium_airport",
                        "FASC": "medium_airport",
                        "FASE": "small_airport",
                        "FASS": "medium_airport",
                        "FASZ": "medium_airport",
                        "FATH": "small_airport",
                        "FATN": "small_airport",
                        "FATZ": "medium_airport",
                        "FAUL": "medium_airport",
                        "FAUP": "medium_airport",
                        "FAUT": "medium_airport",
                        "FAVB": "medium_airport",
                        "FAVG": "medium_airport",
                        "FAVR": "medium_airport",
                        "FAVY": "small_airport",
                        "FAWB": "medium_airport",
                        "FAWK": "medium_airport",
                        "FBFT": "medium_airport",
                        "FBGZ": "small_airport",
                        "FBJW": "medium_airport",
                        "FBKE": "medium_airport",
                        "FBKR": "small_airport",
                        "FBLO": "small_airport",
                        "FBMN": "medium_airport",
                        "FBOR": "small_airport",
                        "FBPY": "small_airport",
                        "FBSK": "large_airport",
                        "FBSN": "medium_airport",
                        "FBSP": "medium_airport",
                        "FBSV": "small_airport",
                        "FBSW": "small_airport",
                        "FBTL": "small_airport",
                        "FBTS": "small_airport",
                        "FCBB": "medium_airport",
                        "FCBD": "small_airport",
                        "FCBK": "small_airport",
                        "FCBL": "small_airport",
                        "FCBM": "small_airport",
                        "FCBS": "small_airport",
                        "FCBY": "small_airport",
                        "FCBZ": "small_airport",
                        "FCMM": "small_airport",
                        "FCOB": "small_airport",
                        "FCOD": "medium_airport",
                        "FCOE": "small_airport",
                        "FCOG": "small_airport",
                        "FCOI": "small_airport",
                        "FCOK": "small_airport",
                        "FCOM": "small_airport",
                        "FCOO": "medium_airport",
                        "FCOS": "small_airport",
                        "FCOT": "small_airport",
                        "FCOU": "medium_airport",
                        "FCPA": "small_airport",
                        "FCPL": "medium_airport",
                        "FCPP": "medium_airport",
                        "FDMS": "medium_airport",
                        "FDSK": "large_airport",
                        "FEA": "small_airport",
                        "FEFC": "small_airport",
                        "FEFF": "medium_airport",
                        "FEFG": "small_airport",
                        "FEFI": "small_airport",
                        "FEFM": "small_airport",
                        "FEFN": "small_airport",
                        "FEFO": "small_airport",
                        "FEFR": "small_airport",
                        "FEFS": "small_airport",
                        "FEFT": "medium_airport",
                        "FEFW": "small_airport",
                        "FEFY": "small_airport",
                        "FEFZ": "small_airport",
                        "FEGE": "small_airport",
                        "FEGF": "small_airport",
                        "FEGL": "small_airport",
                        "FEGM": "small_airport",
                        "FEGO": "small_airport",
                        "FEGR": "small_airport",
                        "FEGU": "small_airport",
                        "FEGZ": "small_airport",
                        "FGAB": "small_airport",
                        "FGBT": "medium_airport",
                        "FGMY": "small_airport",
                        "FGSL": "medium_airport",
                        "FHAW": "medium_airport",
                        "FHSH": "small_airport",
                        "FIMP": "large_airport",
                        "FIMR": "medium_airport",
                        "FIN": "small_airport",
                        "FJDG": "medium_airport",
                        "FKAN": "small_airport",
                        "FKKB": "small_airport",
                        "FKKC": "medium_airport",
                        "FKKD": "medium_airport",
                        "FKKF": "small_airport",
                        "FKKG": "small_airport",
                        "FKKH": "small_airport",
                        "FKKI": "small_airport",
                        "FKKJ": "small_airport",
                        "FKKL": "medium_airport",
                        "FKKM": "medium_airport",
                        "FKKN": "medium_airport",
                        "FKKO": "small_airport",
                        "FKKR": "medium_airport",
                        "FKKS": "small_airport",
                        "FKKU": "medium_airport",
                        "FKKV": "medium_airport",
                        "FKKW": "small_airport",
                        "FKKY": "medium_airport",
                        "FKYS": "medium_airport",
                        "FLBA": "small_airport",
                        "FLCP": "small_airport",
                        "FLE": "closed",
                        "FLJK": "small_airport",
                        "FLKE": "medium_airport",
                        "FLKL": "small_airport",
                        "FLKO": "small_airport",
                        "FLKS": "small_airport",
                        "FLKY": "small_airport",
                        "FLLI": "medium_airport",
                        "FLLK": "small_airport",
                        "FLLS": "large_airport",
                        "FLMA": "small_airport",
                        "FLMF": "medium_airport",
                        "FLMG": "medium_airport",
                        "FLNA": "small_airport",
                        "FLND": "medium_airport",
                        "FLRZ": "small_airport",
                        "FLSN": "small_airport",
                        "FLSO": "medium_airport",
                        "FLSS": "small_airport",
                        "FLSW": "small_airport",
                        "FLT": "small_airport",
                        "FLZB": "small_airport",
                        "FM-ULI": "small_airport",
                        "FMCH": "medium_airport",
                        "FMCI": "medium_airport",
                        "FMCN": "small_airport",
                        "FMCV": "medium_airport",
                        "FMCZ": "medium_airport",
                        "FMEE": "medium_airport",
                        "FMEP": "medium_airport",
                        "FMMC": "small_airport",
                        "FMME": "small_airport",
                        "FMMG": "small_airport",
                        "FMMH": "small_airport",
                        "FMMI": "large_airport",
                        "FMMK": "small_airport",
                        "FMML": "small_airport",
                        "FMMN": "medium_airport",
                        "FMMO": "small_airport",
                        "FMMQ": "small_airport",
                        "FMMR": "small_airport",
                        "FMMS": "medium_airport",
                        "FMMT": "medium_airport",
                        "FMMU": "small_airport",
                        "FMMV": "medium_airport",
                        "FMMX": "small_airport",
                        "FMMY": "small_airport",
                        "FMMZ": "small_airport",
                        "FMNA": "medium_airport",
                        "FMNB": "small_airport",
                        "FMNC": "medium_airport",
                        "FMND": "medium_airport",
                        "FMNE": "small_airport",
                        "FMNF": "small_airport",
                        "FMNG": "small_airport",
                        "FMNH": "medium_airport",
                        "FMNL": "medium_airport",
                        "FMNM": "medium_airport",
                        "FMNN": "medium_airport",
                        "FMNO": "small_airport",
                        "FMNP": "small_airport",
                        "FMNQ": "medium_airport",
                        "FMNR": "medium_airport",
                        "FMNS": "medium_airport",
                        "FMNT": "small_airport",
                        "FMNV": "medium_airport",
                        "FMNW": "medium_airport",
                        "FMNX": "small_airport",
                        "FMNZ": "small_airport",
                        "FMSB": "small_airport",
                        "FMSC": "small_airport",
                        "FMSD": "medium_airport",
                        "FMSF": "medium_airport",
                        "FMSG": "small_airport",
                        "FMSI": "small_airport",
                        "FMSJ": "small_airport",
                        "FMSK": "medium_airport",
                        "FMSL": "small_airport",
                        "FMSM": "medium_airport",
                        "FMSN": "small_airport",
                        "FMSR": "medium_airport",
                        "FMST": "medium_airport",
                        "FMSU": "small_airport",
                        "FMSV": "small_airport",
                        "FMSY": "small_airport",
                        "FMSZ": "small_airport",
                        "FNAM": "small_airport",
                        "FNBC": "medium_airport",
                        "FNBG": "medium_airport",
                        "FNBL": "small_airport",
                        "FNCA": "medium_airport",
                        "FNCF": "small_airport",
                        "FNCH": "small_airport",
                        "FNCT": "medium_airport",
                        "FNCV": "small_airport",
                        "FNCX": "small_airport",
                        "FNCZ": "small_airport",
                        "FNDU": "medium_airport",
                        "FNE": "small_airport",
                        "FNGI": "medium_airport",
                        "FNHU": "medium_airport",
                        "FNKU": "medium_airport",
                        "FNLK": "small_airport",
                        "FNLU": "large_airport",
                        "FNLZ": "small_airport",
                        "FNMA": "medium_airport",
                        "FNME": "medium_airport",
                        "FNMO": "medium_airport",
                        "FNNG": "medium_airport",
                        "FNPA": "medium_airport",
                        "FNSA": "medium_airport",
                        "FNSO": "medium_airport",
                        "FNSU": "medium_airport",
                        "FNUA": "small_airport",
                        "FNUB": "medium_airport",
                        "FNUE": "medium_airport",
                        "FNUG": "medium_airport",
                        "FNWK": "small_airport",
                        "FNXA": "medium_airport",
                        "FNZE": "small_airport",
                        "FNZG": "small_airport",
                        "FOGB": "small_airport",
                        "FOGE": "small_airport",
                        "FOGF": "small_airport",
                        "FOGG": "small_airport",
                        "FOGI": "small_airport",
                        "FOGJ": "small_airport",
                        "FOGK": "medium_airport",
                        "FOGM": "medium_airport",
                        "FOGO": "medium_airport",
                        "FOGQ": "medium_airport",
                        "FOGR": "medium_airport",
                        "FOGV": "small_airport",
                        "FOOB": "medium_airport",
                        "FOOD": "small_airport",
                        "FOOE": "small_airport",
                        "FOOG": "medium_airport",
                        "FOOH": "medium_airport",
                        "FOOI": "small_airport",
                        "FOOK": "medium_airport",
                        "FOOL": "medium_airport",
                        "FOOM": "small_airport",
                        "FOON": "medium_airport",
                        "FOOR": "small_airport",
                        "FOOS": "small_airport",
                        "FOOT": "small_airport",
                        "FOOY": "small_airport",
                        "FOY": "small_airport",
                        "FPPR": "medium_airport",
                        "FPST": "medium_airport",
                        "FQAG": "small_airport",
                        "FQBR": "medium_airport",
                        "FQCB": "small_airport",
                        "FQCH": "medium_airport",
                        "FQIA": "small_airport",
                        "FQIN": "medium_airport",
                        "FQLC": "medium_airport",
                        "FQLU": "small_airport",
                        "FQMA": "large_airport",
                        "FQMD": "medium_airport",
                        "FQMP": "medium_airport",
                        "FQNC": "small_airport",
                        "FQNP": "medium_airport",
                        "FQPB": "medium_airport",
                        "FQPO": "small_airport",
                        "FQQL": "medium_airport",
                        "FQTT": "medium_airport",
                        "FQVL": "medium_airport",
                        "FQXA": "small_airport",
                        "FR-JCA": "heliport",
                        "FSDR": "small_airport",
                        "FSIA": "large_airport",
                        "FSL": "small_airport",
                        "FSN": "closed",
                        "FSPP": "medium_airport",
                        "FSSB": "small_airport",
                        "FSSD": "small_airport",
                        "FSSF": "small_airport",
                        "FTTA": "small_airport",
                        "FTTB": "small_airport",
                        "FTTC": "medium_airport",
                        "FTTD": "medium_airport",
                        "FTTH": "small_airport",
                        "FTTI": "small_airport",
                        "FTTJ": "large_airport",
                        "FTTK": "small_airport",
                        "FTTL": "small_airport",
                        "FTTM": "small_airport",
                        "FTTN": "small_airport",
                        "FTTP": "small_airport",
                        "FTTS": "small_airport",
                        "FTTU": "small_airport",
                        "FTTY": "medium_airport",
                        "FUB": "small_airport",
                        "FVBM": "small_airport",
                        "FVBU": "medium_airport",
                        "FVCH": "small_airport",
                        "FVCZ": "medium_airport",
                        "FVFA": "medium_airport",
                        "FVHA": "large_airport",
                        "FVKB": "medium_airport",
                        "FVMH": "small_airport",
                        "FVMU": "small_airport",
                        "FVMV": "medium_airport",
                        "FVTL": "medium_airport",
                        "FVWN": "medium_airport",
                        "FVWT": "small_airport",
                        "FWCD": "small_airport",
                        "FWCL": "medium_airport",
                        "FWCM": "small_airport",
                        "FWDW": "medium_airport",
                        "FWKA": "medium_airport",
                        "FWKG": "small_airport",
                        "FWKI": "medium_airport",
                        "FWLK": "small_airport",
                        "FWMG": "medium_airport",
                        "FWMY": "small_airport",
                        "FWSM": "small_airport",
                        "FWUU": "medium_airport",
                        "FXLK": "small_airport",
                        "FXLR": "small_airport",
                        "FXLS": "small_airport",
                        "FXMF": "small_airport",
                        "FXMK": "small_airport",
                        "FXMM": "medium_airport",
                        "FXNK": "small_airport",
                        "FXPG": "small_airport",
                        "FXQG": "small_airport",
                        "FXQN": "small_airport",
                        "FXSH": "small_airport",
                        "FXSK": "small_airport",
                        "FXSM": "small_airport",
                        "FXSS": "small_airport",
                        "FXTA": "small_airport",
                        "FXTK": "small_airport",
                        "FYAA": "small_airport",
                        "FYAR": "medium_airport",
                        "FYGB": "small_airport",
                        "FYGF": "medium_airport",
                        "FYHI": "small_airport",
                        "FYKB": "small_airport",
                        "FYKM": "small_airport",
                        "FYKT": "medium_airport",
                        "FYLS": "small_airport",
                        "FYLZ": "medium_airport",
                        "FYME": "small_airport",
                        "FYMG": "small_airport",
                        "FYMO": "small_airport",
                        "FYNA": "small_airport",
                        "FYOA": "medium_airport",
                        "FYOE": "small_airport",
                        "FYOG": "medium_airport",
                        "FYOO": "small_airport",
                        "FYOP": "small_airport",
                        "FYOS": "small_airport",
                        "FYOW": "small_airport",
                        "FYRU": "medium_airport",
                        "FYSA": "small_airport",
                        "FYSM": "small_airport",
                        "FYSS": "small_airport",
                        "FYTE": "small_airport",
                        "FYTM": "medium_airport",
                        "FYWB": "medium_airport",
                        "FYWE": "medium_airport",
                        "FYWH": "large_airport",
                        "FZAA": "large_airport",
                        "FZAB": "medium_airport",
                        "FZAG": "small_airport",
                        "FZAJ": "small_airport",
                        "FZAL": "small_airport",
                        "FZAM": "small_airport",
                        "FZAR": "small_airport",
                        "FZBA": "small_airport",
                        "FZBI": "small_airport",
                        "FZBO": "medium_airport",
                        "FZBT": "small_airport",
                        "FZCA": "medium_airport",
                        "FZCB": "small_airport",
                        "FZCE": "small_airport",
                        "FZCV": "small_airport",
                        "FZEA": "medium_airport",
                        "FZEN": "small_airport",
                        "FZFA": "small_airport",
                        "FZFD": "medium_airport",
                        "FZFK": "medium_airport",
                        "FZFP": "small_airport",
                        "FZFU": "small_airport",
                        "FZGA": "medium_airport",
                        "FZGN": "small_airport",
                        "FZGV": "small_airport",
                        "FZIC": "medium_airport",
                        "FZIR": "small_airport",
                        "FZJH": "medium_airport",
                        "FZKA": "medium_airport",
                        "FZKJ": "medium_airport",
                        "FZMA": "medium_airport",
                        "FZMB": "small_airport",
                        "FZNA": "medium_airport",
                        "FZNP": "small_airport",
                        "FZOA": "medium_airport",
                        "FZOD": "small_airport",
                        "FZOP": "small_airport",
                        "FZQA": "medium_airport",
                        "FZQC": "small_airport",
                        "FZQG": "small_airport",
                        "FZQM": "medium_airport",
                        "FZRA": "small_airport",
                        "FZRB": "small_airport",
                        "FZRF": "medium_airport",
                        "FZRM": "small_airport",
                        "FZRQ": "small_airport",
                        "FZSA": "medium_airport",
                        "FZSK": "small_airport",
                        "FZTK": "small_airport",
                        "FZUA": "medium_airport",
                        "FZUG": "small_airport",
                        "FZUK": "small_airport",
                        "FZVA": "small_airport",
                        "FZVI": "small_airport",
                        "FZVM": "small_airport",
                        "FZVR": "small_airport",
                        "FZVS": "small_airport",
                        "FZWA": "medium_airport",
                        "FZWC": "small_airport",
                        "FZWT": "small_airport",
                        "GA-AKE": "small_airport",
                        "GA-GAX": "small_airport",
                        "GAB": "small_airport",
                        "GABS": "large_airport",
                        "GAGM": "small_airport",
                        "GAGO": "medium_airport",
                        "GAKA": "small_airport",
                        "GAKO": "small_airport",
                        "GAKY": "medium_airport",
                        "GAMB": "medium_airport",
                        "GANK": "small_airport",
                        "GANR": "small_airport",
                        "GASK": "small_airport",
                        "GATB": "medium_airport",
                        "GAYE": "small_airport",
                        "GAZ": "small_airport",
                        "GB-0070": "closed",
                        "GB-DOC": "small_airport",
                        "GB-FLH": "small_airport",
                        "GB-FOA": "small_airport",
                        "GB-OUK": "medium_airport",
                        "GB-PSV": "small_airport",
                        "GBM": "small_airport",
                        "GBYD": "medium_airport",
                        "GCFV": "medium_airport",
                        "GCGM": "small_airport",
                        "GCHI": "medium_airport",
                        "GCLA": "large_airport",
                        "GCLP": "large_airport",
                        "GCRR": "medium_airport",
                        "GCTS": "large_airport",
                        "GCV": "small_airport",
                        "GCXO": "large_airport",
                        "GDA": "closed",
                        "GECE": "heliport",
                        "GEML": "medium_airport",
                        "GEW": "small_airport",
                        "GFBN": "medium_airport",
                        "GFBO": "medium_airport",
                        "GFD": "small_airport",
                        "GFGK": "small_airport",
                        "GFHA": "small_airport",
                        "GFKB": "small_airport",
                        "GFKE": "medium_airport",
                        "GFLL": "large_airport",
                        "GFYE": "medium_airport",
                        "GGBU": "small_airport",
                        "GGOV": "medium_airport",
                        "GHE": "small_airport",
                        "GL-JGR": "heliport",
                        "GL-QCU": "heliport",
                        "GLBU": "small_airport",
                        "GLCP": "small_airport",
                        "GLGE": "small_airport",
                        "GLMR": "medium_airport",
                        "GLNA": "small_airport",
                        "GLP": "small_airport",
                        "GLRB": "large_airport",
                        "GLST": "small_airport",
                        "GLTN": "small_airport",
                        "GLVA": "small_airport",
                        "GMAD": "medium_airport",
                        "GMAT": "medium_airport",
                        "GMAZ": "small_airport",
                        "GMFB": "medium_airport",
                        "GMFF": "medium_airport",
                        "GMFK": "medium_airport",
                        "GMFM": "medium_airport",
                        "GMFO": "medium_airport",
                        "GMMA": "medium_airport",
                        "GMMB": "small_airport",
                        "GMMC": "closed",
                        "GMMD": "small_airport",
                        "GMME": "medium_airport",
                        "GMMF": "small_airport",
                        "GMMH": "medium_airport",
                        "GMMI": "small_airport",
                        "GMML": "medium_airport",
                        "GMMN": "large_airport",
                        "GMMS": "small_airport",
                        "GMMW": "medium_airport",
                        "GMMX": "medium_airport",
                        "GMMY": "medium_airport",
                        "GMMZ": "medium_airport",
                        "GMQ": "medium_airport",
                        "GMTA": "medium_airport",
                        "GMTN": "medium_airport",
                        "GMTT": "medium_airport",
                        "GNE": "closed",
                        "GNU": "small_airport",
                        "GOBD": "large_airport",
                        "GOC": "small_airport",
                        "GODK": "small_airport",
                        "GOGG": "medium_airport",
                        "GOGS": "medium_airport",
                        "GOOK": "medium_airport",
                        "GOOY": "large_airport",
                        "GOSM": "medium_airport",
                        "GOSP": "small_airport",
                        "GOSR": "small_airport",
                        "GOSS": "medium_airport",
                        "GOTB": "medium_airport",
                        "GOTK": "medium_airport",
                        "GOTS": "small_airport",
                        "GOTT": "medium_airport",
                        "GQNA": "small_airport",
                        "GQNB": "small_airport",
                        "GQNC": "small_airport",
                        "GQND": "small_airport",
                        "GQNE": "small_airport",
                        "GQNF": "small_airport",
                        "GQNH": "small_airport",
                        "GQNI": "small_airport",
                        "GQNJ": "small_airport",
                        "GQNK": "small_airport",
                        "GQNL": "small_airport",
                        "GQNO": "large_airport",
                        "GQNS": "small_airport",
                        "GQNT": "small_airport",
                        "GQPA": "medium_airport",
                        "GQPF": "small_airport",
                        "GQPP": "medium_airport",
                        "GQPZ": "small_airport",
                        "GRC": "small_airport",
                        "GRH": "closed",
                        "GSZ": "small_airport",
                        "GT-0001": "small_airport",
                        "GT-0002": "small_airport",
                        "GT-0003": "small_airport",
                        "GT-0004": "small_airport",
                        "GT-0006": "small_airport",
                        "GT-0007": "small_airport",
                        "GT-PKJ": "small_airport",
                        "GTZ": "small_airport",
                        "GUCY": "medium_airport",
                        "GUE": "small_airport",
                        "GUFA": "small_airport",
                        "GUFH": "small_airport",
                        "GUKU": "small_airport",
                        "GULB": "small_airport",
                        "GUMA": "small_airport",
                        "GUNZ": "small_airport",
                        "GUOK": "small_airport",
                        "GUSB": "small_airport",
                        "GUSI": "small_airport",
                        "GUXN": "small_airport",
                        "GVAC": "large_airport",
                        "GVAN": "small_airport",
                        "GVBA": "medium_airport",
                        "GVE": "small_airport",
                        "GVMA": "medium_airport",
                        "GVMT": "small_airport",
                        "GVNP": "medium_airport",
                        "GVSF": "small_airport",
                        "GVSN": "medium_airport",
                        "GVSV": "medium_airport",
                        "GWN": "closed",
                        "GWW": "closed",
                        "GY-BCG": "small_airport",
                        "GY-BTO": "small_airport",
                        "GY-DOE": "small_airport",
                        "GY-LDO": "small_airport",
                        "GY-WSO": "small_airport",
                        "HAAB": "large_airport",
                        "HAAM": "medium_airport",
                        "HAAX": "medium_airport",
                        "HABC": "small_airport",
                        "HABD": "medium_airport",
                        "HABE": "small_airport",
                        "HADB": "closed",
                        "HADC": "small_airport",
                        "HADD": "small_airport",
                        "HADM": "small_airport",
                        "HADR": "medium_airport",
                        "HADT": "small_airport",
                        "HAFN": "small_airport",
                        "HAGB": "small_airport",
                        "HAGH": "small_airport",
                        "HAGM": "medium_airport",
                        "HAGN": "medium_airport",
                        "HAGO": "medium_airport",
                        "HAGR": "small_airport",
                        "HAHM": "medium_airport",
                        "HAHU": "small_airport",
                        "HAJJ": "medium_airport",
                        "HAJM": "medium_airport",
                        "HAKD": "small_airport",
                        "HAKL": "small_airport",
                        "HALA": "small_airport",
                        "HALL": "small_airport",
                        "HAMA": "small_airport",
                        "HAMK": "medium_airport",
                        "HAMM": "small_airport",
                        "HAMN": "small_airport",
                        "HAMR": "small_airport",
                        "HAMT": "small_airport",
                        "HANG": "small_airport",
                        "HANJ": "small_airport",
                        "HANK": "small_airport",
                        "HAPW": "small_airport",
                        "HASD": "small_airport",
                        "HASK": "small_airport",
                        "HASM": "small_airport",
                        "HASO": "medium_airport",
                        "HATP": "small_airport",
                        "HAWC": "small_airport",
                        "HAWR": "small_airport",
                        "HAY": "small_airport",
                        "HAZ": "small_airport",
                        "HBBA": "medium_airport",
                        "HBBE": "small_airport",
                        "HBBO": "small_airport",
                        "HBQ": "medium_airport",
                        "HBT": "seaplane_base",
                        "HCMA": "small_airport",
                        "HCMB": "small_airport",
                        "HCMC": "small_airport",
                        "HCMD": "small_airport",
                        "HCME": "small_airport",
                        "HCMF": "small_airport",
                        "HCMG": "small_airport",
                        "HCMH": "large_airport",
                        "HCMI": "medium_airport",
                        "HCMJ": "small_airport",
                        "HCMK": "medium_airport",
                        "HCMM": "medium_airport",
                        "HCMN": "small_airport",
                        "HCMO": "small_airport",
                        "HCMR": "small_airport",
                        "HCMS": "small_airport",
                        "HCMU": "small_airport",
                        "HCMV": "small_airport",
                        "HCMW": "small_airport",
                        "HDAM": "medium_airport",
                        "HDAS": "small_airport",
                        "HDMO": "small_airport",
                        "HDOB": "small_airport",
                        "HDTJ": "small_airport",
                        "HE13": "medium_airport",
                        "HE24": "small_airport",
                        "HE25": "small_airport",
                        "HE40": "small_airport",
                        "HEAL": "medium_airport",
                        "HEAR": "medium_airport",
                        "HEAT": "medium_airport",
                        "HEAX": "medium_airport",
                        "HEBA": "medium_airport",
                        "HEBL": "medium_airport",
                        "HECA": "large_airport",
                        "HEDK": "small_airport",
                        "HEGN": "large_airport",
                        "HEGR": "medium_airport",
                        "HEKG": "small_airport",
                        "HELX": "large_airport",
                        "HEMA": "medium_airport",
                        "HEMK": "medium_airport",
                        "HEMM": "medium_airport",
                        "HEO": "small_airport",
                        "HEOW": "small_airport",
                        "HEPS": "medium_airport",
                        "HESC": "medium_airport",
                        "HESH": "medium_airport",
                        "HESN": "medium_airport",
                        "HESX": "medium_airport",
                        "HETB": "medium_airport",
                        "HETR": "small_airport",
                        "HEY": "heliport",
                        "HGT": "heliport",
                        "HHAS": "medium_airport",
                        "HHMS": "medium_airport",
                        "HHSB": "medium_airport",
                        "HHTS": "small_airport",
                        "HI01": "small_airport",
                        "HI07": "heliport",
                        "HIA": "medium_airport",
                        "HIL": "small_airport",
                        "HKAM": "medium_airport",
                        "HKB": "small_airport",
                        "HKEL": "medium_airport",
                        "HKES": "small_airport",
                        "HKFG": "small_airport",
                        "HKGA": "small_airport",
                        "HKHO": "small_airport",
                        "HKJK": "large_airport",
                        "HKKE": "small_airport",
                        "HKKG": "small_airport",
                        "HKKI": "medium_airport",
                        "HKKL": "small_airport",
                        "HKKR": "small_airport",
                        "HKKT": "medium_airport",
                        "HKLK": "medium_airport",
                        "HKLO": "medium_airport",
                        "HKLU": "medium_airport",
                        "HKLY": "small_airport",
                        "HKMA": "small_airport",
                        "HKMB": "small_airport",
                        "HKML": "medium_airport",
                        "HKMO": "large_airport",
                        "HKMS": "medium_airport",
                        "HKMY": "small_airport",
                        "HKNI": "small_airport",
                        "HKNK": "small_airport",
                        "HKNW": "medium_airport",
                        "HKNY": "medium_airport",
                        "HKSB": "small_airport",
                        "HKUK": "small_airport",
                        "HKWJ": "medium_airport",
                        "HLGD": "medium_airport",
                        "HLGN": "small_airport",
                        "HLGT": "medium_airport",
                        "HLKF": "medium_airport",
                        "HLLB": "medium_airport",
                        "HLLM": "medium_airport",
                        "HLLQ": "medium_airport",
                        "HLLS": "medium_airport",
                        "HLLT": "large_airport",
                        "HLMB": "medium_airport",
                        "HLNR": "small_airport",
                        "HLON": "small_airport",
                        "HLTD": "medium_airport",
                        "HLZN": "small_airport",
                        "HLZW": "small_airport",
                        "HN-MUG": "small_airport",
                        "HOO": "small_airport",
                        "HRC": "small_airport",
                        "HRYG": "medium_airport",
                        "HRYI": "small_airport",
                        "HRYR": "large_airport",
                        "HRYU": "small_airport",
                        "HRZA": "medium_airport",
                        "HSAT": "small_airport",
                        "HSDB": "small_airport",
                        "HSDN": "medium_airport",
                        "HSDZ": "small_airport",
                        "HSFA": "medium_airport",
                        "HSFS": "medium_airport",
                        "HSGF": "small_airport",
                        "HSGG": "small_airport",
                        "HSGN": "small_airport",
                        "HSHG": "small_airport",
                        "HSJ": "small_airport",
                        "HSKA": "medium_airport",
                        "HSKG": "small_airport",
                        "HSKI": "closed",
                        "HSLI": "small_airport",
                        "HSMK": "small_airport",
                        "HSMN": "medium_airport",
                        "HSNH": "small_airport",
                        "HSNN": "small_airport",
                        "HSNW": "small_airport",
                        "HSOB": "medium_airport",
                        "HSPN": "medium_airport",
                        "HSSJ": "large_airport",
                        "HSSM": "medium_airport",
                        "HSSS": "large_airport",
                        "HSSW": "small_airport",
                        "HSWD": "small_airport",
                        "HSWW": "medium_airport",
                        "HSZA": "small_airport",
                        "HTAR": "medium_airport",
                        "HTBU": "small_airport",
                        "HTDA": "large_airport",
                        "HTDO": "medium_airport",
                        "HTGW": "medium_airport",
                        "HTIR": "medium_airport",
                        "HTKA": "small_airport",
                        "HTKI": "small_airport",
                        "HTKJ": "medium_airport",
                        "HTLI": "small_airport",
                        "HTLM": "medium_airport",
                        "HTM": "small_airport",
                        "HTMA": "small_airport",
                        "HTMD": "small_airport",
                        "HTMI": "small_airport",
                        "HTMP": "small_airport",
                        "HTMS": "small_airport",
                        "HTMT": "medium_airport",
                        "HTMU": "small_airport",
                        "HTMW": "medium_airport",
                        "HTNA": "small_airport",
                        "HTNJ": "small_airport",
                        "HTPE": "medium_airport",
                        "HTSN": "small_airport",
                        "HTSO": "small_airport",
                        "HTSU": "small_airport",
                        "HTSY": "small_airport",
                        "HTTB": "small_airport",
                        "HTTG": "medium_airport",
                        "HTZA": "large_airport",
                        "HUAR": "medium_airport",
                        "HUEN": "large_airport",
                        "HUGU": "medium_airport",
                        "HUJI": "small_airport",
                        "HUKC": "closed",
                        "HUKF": "small_airport",
                        "HUKS": "small_airport",
                        "HUMA": "small_airport",
                        "HUMI": "small_airport",
                        "HUSO": "medium_airport",
                        "HUTO": "small_airport",
                        "HWA": "small_airport",
                        "HYL": "seaplane_base",
                        "IBI": "small_airport",
                        "IBL": "medium_airport",
                        "ICO": "small_airport",
                        "ID-0003": "small_airport",
                        "ID-0006": "small_airport",
                        "ID-0011": "small_airport",
                        "ID-0017": "large_airport",
                        "ID-0169": "small_airport",
                        "ID-AAS": "small_airport",
                        "ID-AGD": "small_airport",
                        "ID-AKQ": "small_airport",
                        "ID-AYW": "small_airport",
                        "ID-BJG": "small_airport",
                        "ID-BXM": "small_airport",
                        "ID-DRH": "small_airport",
                        "ID-ELR": "small_airport",
                        "ID-EWE": "small_airport",
                        "ID-FOO": "small_airport",
                        "ID-GAV": "small_airport",
                        "ID-IUL": "small_airport",
                        "ID-KBF": "small_airport",
                        "ID-KBX": "small_airport",
                        "ID-KCD": "small_airport",
                        "ID-KCI": "small_airport",
                        "ID-KEA": "small_airport",
                        "ID-KMM": "small_airport",
                        "ID-KOD": "small_airport",
                        "ID-KRC": "small_airport",
                        "ID-KWB": "small_airport",
                        "ID-LLN": "small_airport",
                        "ID-LWE": "small_airport",
                        "ID-LYK": "small_airport",
                        "ID-MJY": "small_airport",
                        "ID-MPT": "small_airport",
                        "ID-MSI": "small_airport",
                        "ID-MUF": "small_airport",
                        "ID-NAF": "small_airport",
                        "ID-OBD": "small_airport",
                        "ID-PUM": "small_airport",
                        "ID-PWL": "small_airport",
                        "ID-RAQ": "small_airport",
                        "ID-RKI": "small_airport",
                        "ID-RTI": "small_airport",
                        "ID-RUF": "small_airport",
                        "ID-RZS": "small_airport",
                        "ID-SAE": "small_airport",
                        "ID-TBM": "small_airport",
                        "ID-TMY": "small_airport",
                        "ID-ZEG": "small_airport",
                        "ID-ZGP": "medium_airport",
                        "IDN": "small_airport",
                        "ILX": "closed",
                        "IMA": "small_airport",
                        "IMG": "small_airport",
                        "IN-0070": "small_airport",
                        "IN-JGB": "small_airport",
                        "IN-NVY": "small_airport",
                        "IN-RJI": "small_airport",
                        "IN-TEI": "small_airport",
                        "INE": "small_airport",
                        "IOK": "small_airport",
                        "IOP": "small_airport",
                        "IR-0001": "small_airport",
                        "IR-GSM": "small_airport",
                        "IRU": "seaplane_base",
                        "ISG": "medium_airport",
                        "IT-TQR": "heliport",
                        "ITK": "small_airport",
                        "IUS": "closed",
                        "IVH": "closed",
                        "IVI": "small_airport",
                        "JGD": "medium_airport",
                        "JIC": "small_airport",
                        "JIO": "small_airport",
                        "JIQ": "medium_airport",
                        "JJD": "medium_airport",
                        "JLA": "small_airport",
                        "JOP": "small_airport",
                        "JPB": "closed",
                        "JPN": "heliport",
                        "JRA": "heliport",
                        "JRB": "heliport",
                        "JUH": "small_airport",
                        "K00C": "small_airport",
                        "K00F": "small_airport",
                        "K05U": "small_airport",
                        "K06U": "small_airport",
                        "K07": "small_airport",
                        "K0B8": "small_airport",
                        "K0K7": "small_airport",
                        "K0S9": "small_airport",
                        "K0V7": "small_airport",
                        "K1B1": "small_airport",
                        "K1F0": "small_airport",
                        "K1G4": "small_airport",
                        "K1O2": "small_airport",
                        "K1O5": "small_airport",
                        "K1V6": "small_airport",
                        "K21": "seaplane_base",
                        "K23": "small_airport",
                        "K29": "small_airport",
                        "K2F0": "small_airport",
                        "K2O1": "small_airport",
                        "K2S7": "small_airport",
                        "K2W6": "small_airport",
                        "K37V": "small_airport",
                        "K39N": "small_airport",
                        "K3C8": "small_airport",
                        "K3O9": "small_airport",
                        "K3S8": "small_airport",
                        "K3TR": "small_airport",
                        "K3W7": "small_airport",
                        "K40G": "small_airport",
                        "K40J": "small_airport",
                        "K41U": "small_airport",
                        "K43U": "closed",
                        "K44U": "small_airport",
                        "K47N": "small_airport",
                        "K4R7": "small_airport",
                        "K4S1": "small_airport",
                        "K50I": "small_airport",
                        "K5N2": "small_airport",
                        "K5T9": "small_airport",
                        "K61B": "small_airport",
                        "K67L": "small_airport",
                        "K6D9": "small_airport",
                        "K6S2": "small_airport",
                        "K74S": "small_airport",
                        "K74V": "small_airport",
                        "K7G9": "small_airport",
                        "K7V2": "small_airport",
                        "K80F": "small_airport",
                        "K8A6": "small_airport",
                        "K9U3": "small_airport",
                        "KAAF": "small_airport",
                        "KABE": "medium_airport",
                        "KABI": "medium_airport",
                        "KABQ": "large_airport",
                        "KABR": "medium_airport",
                        "KABY": "medium_airport",
                        "KACB": "small_airport",
                        "KACK": "medium_airport",
                        "KACT": "medium_airport",
                        "KACV": "medium_airport",
                        "KACY": "medium_airport",
                        "KADG": "small_airport",
                        "KADH": "small_airport",
                        "KADM": "small_airport",
                        "KADS": "small_airport",
                        "KADW": "large_airport",
                        "KAE": "seaplane_base",
                        "KAEL": "small_airport",
                        "KAEX": "medium_airport",
                        "KAF": "small_airport",
                        "KAFF": "small_airport",
                        "KAFJ": "small_airport",
                        "KAFN": "small_airport",
                        "KAFO": "small_airport",
                        "KAFW": "large_airport",
                        "KAGC": "medium_airport",
                        "KAGO": "small_airport",
                        "KAGS": "large_airport",
                        "KAHC": "small_airport",
                        "KAHH": "small_airport",
                        "KAHN": "medium_airport",
                        "KAIA": "small_airport",
                        "KAID": "small_airport",
                        "KAIK": "small_airport",
                        "KAIO": "small_airport",
                        "KAIV": "small_airport",
                        "KAIY": "closed",
                        "KAIZ": "small_airport",
                        "KAKO": "small_airport",
                        "KAKR": "medium_airport",
                        "KALB": "medium_airport",
                        "KALI": "medium_airport",
                        "KALM": "medium_airport",
                        "KALN": "medium_airport",
                        "KALO": "medium_airport",
                        "KALS": "medium_airport",
                        "KALW": "medium_airport",
                        "KALX": "small_airport",
                        "KAMA": "large_airport",
                        "KAMN": "small_airport",
                        "KAMW": "small_airport",
                        "KANB": "medium_airport",
                        "KAND": "medium_airport",
                        "KANK": "small_airport",
                        "KANP": "small_airport",
                        "KANQ": "small_airport",
                        "KANW": "small_airport",
                        "KANY": "small_airport",
                        "KAOH": "small_airport",
                        "KAOO": "medium_airport",
                        "KAPA": "medium_airport",
                        "KAPC": "small_airport",
                        "KAPF": "medium_airport",
                        "KAPG": "medium_airport",
                        "KAPH": "small_airport",
                        "KAPN": "medium_airport",
                        "KAPT": "small_airport",
                        "KAPV": "small_airport",
                        "KAQ": "small_airport",
                        "KARA": "medium_airport",
                        "KARB": "small_airport",
                        "KARG": "small_airport",
                        "KARM": "small_airport",
                        "KARR": "small_airport",
                        "KART": "medium_airport",
                        "KARV": "small_airport",
                        "KARW": "small_airport",
                        "KASE": "medium_airport",
                        "KASG": "small_airport",
                        "KASH": "small_airport",
                        "KASL": "small_airport",
                        "KASN": "small_airport",
                        "KAST": "medium_airport",
                        "KASX": "small_airport",
                        "KASY": "small_airport",
                        "KATL": "large_airport",
                        "KATS": "small_airport",
                        "KATW": "small_airport",
                        "KATY": "medium_airport",
                        "KAUG": "medium_airport",
                        "KAUM": "small_airport",
                        "KAUN": "small_airport",
                        "KAUO": "small_airport",
                        "KAUS": "large_airport",
                        "KAUW": "medium_airport",
                        "KAVL": "large_airport",
                        "KAVO": "small_airport",
                        "KAVP": "medium_airport",
                        "KAVQ": "small_airport",
                        "KAVX": "small_airport",
                        "KAWM": "small_airport",
                        "KAXA": "small_airport",
                        "KAXN": "medium_airport",
                        "KAXS": "small_airport",
                        "KAXV": "small_airport",
                        "KAXX": "small_airport",
                        "KAYE": "closed",
                        "KAYS": "small_airport",
                        "KAYX": "small_airport",
                        "KAZO": "medium_airport",
                        "KBAB": "large_airport",
                        "KBAD": "large_airport",
                        "KBAF": "medium_airport",
                        "KBAK": "medium_airport",
                        "KBAM": "small_airport",
                        "KBBB": "small_airport",
                        "KBBD": "medium_airport",
                        "KBBP": "small_airport",
                        "KBBW": "small_airport",
                        "KBCB": "small_airport",
                        "KBCE": "medium_airport",
                        "KBCT": "small_airport",
                        "KBDE": "medium_airport",
                        "KBDG": "small_airport",
                        "KBDL": "large_airport",
                        "KBDR": "medium_airport",
                        "KBDU": "small_airport",
                        "KBE": "seaplane_base",
                        "KBEC": "small_airport",
                        "KBED": "medium_airport",
                        "KBEH": "small_airport",
                        "KBFD": "medium_airport",
                        "KBFF": "medium_airport",
                        "KBFI": "large_airport",
                        "KBFL": "medium_airport",
                        "KBFM": "medium_airport",
                        "KBFR": "small_airport",
                        "KBGD": "small_airport",
                        "KBGE": "small_airport",
                        "KBGM": "medium_airport",
                        "KBGR": "large_airport",
                        "KBHB": "medium_airport",
                        "KBHM": "large_airport",
                        "KBID": "small_airport",
                        "KBIE": "small_airport",
                        "KBIF": "medium_airport",
                        "KBIH": "medium_airport",
                        "KBIL": "large_airport",
                        "KBIS": "medium_airport",
                        "KBIX": "medium_airport",
                        "KBJC": "medium_airport",
                        "KBJI": "medium_airport",
                        "KBJJ": "small_airport",
                        "KBKD": "small_airport",
                        "KBKE": "medium_airport",
                        "KBKF": "medium_airport",
                        "KBKL": "medium_airport",
                        "KBKT": "small_airport",
                        "KBKW": "medium_airport",
                        "KBKX": "small_airport",
                        "KBLF": "medium_airport",
                        "KBLH": "medium_airport",
                        "KBLI": "medium_airport",
                        "KBLM": "small_airport",
                        "KBLU": "small_airport",
                        "KBLV": "large_airport",
                        "KBM": "small_airport",
                        "KBMC": "small_airport",
                        "KBMG": "medium_airport",
                        "KBMI": "large_airport",
                        "KBML": "small_airport",
                        "KBMT": "small_airport",
                        "KBNA": "large_airport",
                        "KBNG": "small_airport",
                        "KBNL": "small_airport",
                        "KBNO": "medium_airport",
                        "KBNW": "small_airport",
                        "KBOI": "large_airport",
                        "KBOS": "large_airport",
                        "KBOW": "small_airport",
                        "KBPG": "small_airport",
                        "KBPI": "medium_airport",
                        "KBPK": "medium_airport",
                        "KBPT": "medium_airport",
                        "KBQK": "medium_airport",
                        "KBRD": "medium_airport",
                        "KBRL": "medium_airport",
                        "KBRO": "medium_airport",
                        "KBRY": "small_airport",
                        "KBTF": "small_airport",
                        "KBTL": "medium_airport",
                        "KBTM": "medium_airport",
                        "KBTN": "small_airport",
                        "KBTP": "small_airport",
                        "KBTR": "medium_airport",
                        "KBTV": "medium_airport",
                        "KBTY": "small_airport",
                        "KBUB": "small_airport",
                        "KBUF": "large_airport",
                        "KBUM": "small_airport",
                        "KBUR": "medium_airport",
                        "KBVI": "medium_airport",
                        "KBVO": "small_airport",
                        "KBVS": "small_airport",
                        "KBVX": "small_airport",
                        "KBVY": "medium_airport",
                        "KBWC": "small_airport",
                        "KBWD": "small_airport",
                        "KBWG": "medium_airport",
                        "KBWI": "large_airport",
                        "KBWP": "small_airport",
                        "KBWW": "small_airport",
                        "KBXA": "small_airport",
                        "KBXK": "small_airport",
                        "KBYG": "small_airport",
                        "KBYH": "medium_airport",
                        "KBYI": "medium_airport",
                        "KBYS": "medium_airport",
                        "KBYY": "small_airport",
                        "KBZN": "medium_airport",
                        "KC02": "small_airport",
                        "KC65": "small_airport",
                        "KC80": "small_airport",
                        "KCAD": "small_airport",
                        "KCAE": "large_airport",
                        "KCAG": "small_airport",
                        "KCAK": "medium_airport",
                        "KCAO": "small_airport",
                        "KCAR": "medium_airport",
                        "KCBE": "small_airport",
                        "KCBF": "small_airport",
                        "KCBK": "small_airport",
                        "KCBM": "large_airport",
                        "KCC": "seaplane_base",
                        "KCCB": "small_airport",
                        "KCCR": "small_airport",
                        "KCCY": "medium_airport",
                        "KCDA": "small_airport",
                        "KCDC": "medium_airport",
                        "KCDH": "small_airport",
                        "KCDN": "small_airport",
                        "KCDR": "medium_airport",
                        "KCDS": "medium_airport",
                        "KCDW": "small_airport",
                        "KCEA": "small_airport",
                        "KCEC": "medium_airport",
                        "KCEF": "medium_airport",
                        "KCEU": "small_airport",
                        "KCEV": "small_airport",
                        "KCEW": "medium_airport",
                        "KCEY": "small_airport",
                        "KCEZ": "small_airport",
                        "KCFD": "small_airport",
                        "KCFS": "small_airport",
                        "KCFT": "small_airport",
                        "KCFV": "small_airport",
                        "KCG": "closed",
                        "KCGE": "small_airport",
                        "KCGF": "medium_airport",
                        "KCGI": "medium_airport",
                        "KCGS": "small_airport",
                        "KCGX": "closed",
                        "KCGZ": "small_airport",
                        "KCHA": "large_airport",
                        "KCHK": "small_airport",
                        "KCHO": "medium_airport",
                        "KCHS": "large_airport",
                        "KCIC": "small_airport",
                        "KCID": "large_airport",
                        "KCIN": "small_airport",
                        "KCIR": "small_airport",
                        "KCIU": "medium_airport",
                        "KCKA": "small_airport",
                        "KCKB": "medium_airport",
                        "KCKC": "small_airport",
                        "KCKM": "small_airport",
                        "KCKN": "small_airport",
                        "KCKV": "small_airport",
                        "KCL": "small_airport",
                        "KCLE": "large_airport",
                        "KCLI": "small_airport",
                        "KCLK": "small_airport",
                        "KCLL": "medium_airport",
                        "KCLM": "medium_airport",
                        "KCLR": "small_airport",
                        "KCLS": "small_airport",
                        "KCLT": "large_airport",
                        "KCLW": "small_airport",
                        "KCMH": "large_airport",
                        "KCMI": "medium_airport",
                        "KCMX": "medium_airport",
                        "KCMY": "small_airport",
                        "KCN": "seaplane_base",
                        "KCNH": "small_airport",
                        "KCNK": "small_airport",
                        "KCNM": "medium_airport",
                        "KCNO": "small_airport",
                        "KCNU": "medium_airport",
                        "KCNW": "small_airport",
                        "KCNY": "small_airport",
                        "KCOD": "medium_airport",
                        "KCOE": "medium_airport",
                        "KCOF": "medium_airport",
                        "KCOI": "small_airport",
                        "KCOM": "small_airport",
                        "KCON": "medium_airport",
                        "KCOS": "large_airport",
                        "KCOT": "small_airport",
                        "KCOU": "medium_airport",
                        "KCPM": "small_airport",
                        "KCPR": "medium_airport",
                        "KCPS": "small_airport",
                        "KCQW": "small_airport",
                        "KCR": "small_airport",
                        "KCRE": "medium_airport",
                        "KCRG": "medium_airport",
                        "KCRO": "small_airport",
                        "KCRP": "large_airport",
                        "KCRQ": "medium_airport",
                        "KCRS": "small_airport",
                        "KCRT": "small_airport",
                        "KCRW": "large_airport",
                        "KCRX": "small_airport",
                        "KCSG": "medium_airport",
                        "KCSM": "small_airport",
                        "KCSQ": "small_airport",
                        "KCSV": "medium_airport",
                        "KCTB": "medium_airport",
                        "KCTY": "small_airport",
                        "KCTZ": "small_airport",
                        "KCUB": "medium_airport",
                        "KCUH": "small_airport",
                        "KCVG": "large_airport",
                        "KCVK": "small_airport",
                        "KCVN": "small_airport",
                        "KCVO": "medium_airport",
                        "KCVS": "large_airport",
                        "KCWA": "medium_airport",
                        "KCWC": "small_airport",
                        "KCWF": "small_airport",
                        "KCWI": "small_airport",
                        "KCXL": "small_airport",
                        "KCXO": "medium_airport",
                        "KCXP": "small_airport",
                        "KCXY": "small_airport",
                        "KCYS": "medium_airport",
                        "KCZ": "medium_airport",
                        "KCZT": "small_airport",
                        "KD60": "small_airport",
                        "KDAA": "medium_airport",
                        "KDAB": "large_airport",
                        "KDAG": "medium_airport",
                        "KDAL": "large_airport",
                        "KDAN": "medium_airport",
                        "KDAY": "large_airport",
                        "KDBN": "small_airport",
                        "KDBQ": "large_airport",
                        "KDCA": "large_airport",
                        "KDCU": "small_airport",
                        "KDDC": "medium_airport",
                        "KDEC": "medium_airport",
                        "KDEH": "small_airport",
                        "KDEN": "large_airport",
                        "KDET": "medium_airport",
                        "KDFI": "small_airport",
                        "KDFW": "large_airport",
                        "KDGL": "small_airport",
                        "KDGW": "small_airport",
                        "KDHN": "medium_airport",
                        "KDHT": "medium_airport",
                        "KDIK": "medium_airport",
                        "KDKK": "small_airport",
                        "KDLC": "small_airport",
                        "KDLF": "large_airport",
                        "KDLH": "large_airport",
                        "KDLN": "small_airport",
                        "KDLS": "medium_airport",
                        "KDMA": "medium_airport",
                        "KDMN": "medium_airport",
                        "KDMO": "small_airport",
                        "KDNL": "medium_airport",
                        "KDNN": "small_airport",
                        "KDNS": "small_airport",
                        "KDNV": "small_airport",
                        "KDOV": "large_airport",
                        "KDP": "small_airport",
                        "KDPA": "medium_airport",
                        "KDPG": "small_airport",
                        "KDQ": "small_airport",
                        "KDRA": "medium_airport",
                        "KDRI": "medium_airport",
                        "KDRM": "small_airport",
                        "KDRO": "small_airport",
                        "KDRT": "medium_airport",
                        "KDS": "small_airport",
                        "KDSM": "large_airport",
                        "KDSV": "small_airport",
                        "KDTA": "small_airport",
                        "KDTL": "small_airport",
                        "KDTN": "small_airport",
                        "KDTS": "small_airport",
                        "KDTW": "large_airport",
                        "KDUA": "small_airport",
                        "KDUC": "small_airport",
                        "KDUG": "medium_airport",
                        "KDUJ": "medium_airport",
                        "KDVL": "small_airport",
                        "KDVN": "small_airport",
                        "KDVO": "small_airport",
                        "KDVP": "small_airport",
                        "KDVT": "small_airport",
                        "KDW": "seaplane_base",
                        "KDWF": "closed",
                        "KDWH": "small_airport",
                        "KDXR": "small_airport",
                        "KDYL": "small_airport",
                        "KDYS": "large_airport",
                        "KDZ": "seaplane_base",
                        "KE-0060": "small_airport",
                        "KE-0069": "small_airport",
                        "KE-0133": "small_airport",
                        "KE-0182": "small_airport",
                        "KE-0208": "small_airport",
                        "KE-6087": "small_airport",
                        "KE-6112": "small_airport",
                        "KE-6208": "small_airport",
                        "KE-6978": "small_airport",
                        "KE-KIU": "small_airport",
                        "KE-LBK": "small_airport",
                        "KE-LBN": "small_airport",
                        "KE-LKU": "small_airport",
                        "KE-MRE": "small_airport",
                        "KE-MUM": "small_airport",
                        "KE01": "small_airport",
                        "KE13": "small_airport",
                        "KE14": "small_airport",
                        "KE24": "small_airport",
                        "KE38": "small_airport",
                        "KE51": "small_airport",
                        "KEAN": "small_airport",
                        "KEAR": "small_airport",
                        "KEAT": "medium_airport",
                        "KEAU": "medium_airport",
                        "KEB": "small_airport",
                        "KEBS": "small_airport",
                        "KECG": "medium_airport",
                        "KECP": "medium_airport",
                        "KECS": "small_airport",
                        "KEDE": "small_airport",
                        "KEDN": "small_airport",
                        "KEDW": "large_airport",
                        "KEED": "medium_airport",
                        "KEEN": "medium_airport",
                        "KEFD": "medium_airport",
                        "KEFK": "small_airport",
                        "KEFW": "small_airport",
                        "KEG": "small_airport",
                        "KEGE": "medium_airport",
                        "KEGI": "medium_airport",
                        "KEGV": "small_airport",
                        "KEK": "small_airport",
                        "KEKA": "medium_airport",
                        "KEKM": "small_airport",
                        "KEKN": "medium_airport",
                        "KEKO": "medium_airport",
                        "KEKX": "small_airport",
                        "KELA": "small_airport",
                        "KELD": "medium_airport",
                        "KELK": "small_airport",
                        "KELM": "medium_airport",
                        "KELN": "small_airport",
                        "KELO": "medium_airport",
                        "KELP": "medium_airport",
                        "KELY": "medium_airport",
                        "KELZ": "small_airport",
                        "KEMM": "small_airport",
                        "KEMP": "small_airport",
                        "KEMT": "small_airport",
                        "KEND": "large_airport",
                        "KENL": "small_airport",
                        "KENV": "medium_airport",
                        "KENW": "medium_airport",
                        "KEOK": "small_airport",
                        "KEPH": "small_airport",
                        "KEQA": "small_airport",
                        "KERI": "large_airport",
                        "KERR": "small_airport",
                        "KERV": "small_airport",
                        "KESC": "small_airport",
                        "KESF": "medium_airport",
                        "KESN": "small_airport",
                        "KEST": "small_airport",
                        "KESW": "small_airport",
                        "KETB": "small_airport",
                        "KETN": "small_airport",
                        "KEUF": "small_airport",
                        "KEUG": "medium_airport",
                        "KEVM": "small_airport",
                        "KEVV": "medium_airport",
                        "KEVW": "medium_airport",
                        "KEWB": "medium_airport",
                        "KEWK": "small_airport",
                        "KEWN": "medium_airport",
                        "KEWR": "large_airport",
                        "KEX": "small_airport",
                        "KEYW": "medium_airport",
                        "KEZ": "seaplane_base",
                        "KF05": "small_airport",
                        "KF70": "small_airport",
                        "KFAF": "medium_airport",
                        "KFAM": "small_airport",
                        "KFAR": "medium_airport",
                        "KFAT": "medium_airport",
                        "KFAY": "medium_airport",
                        "KFBG": "medium_airport",
                        "KFBL": "small_airport",
                        "KFBR": "small_airport",
                        "KFBY": "small_airport",
                        "KFCH": "small_airport",
                        "KFCM": "small_airport",
                        "KFCS": "medium_airport",
                        "KFCY": "small_airport",
                        "KFDK": "small_airport",
                        "KFDR": "small_airport",
                        "KFDY": "medium_airport",
                        "KFEP": "small_airport",
                        "KFET": "small_airport",
                        "KFFA": "small_airport",
                        "KFFL": "small_airport",
                        "KFFM": "small_airport",
                        "KFFO": "large_airport",
                        "KFFT": "small_airport",
                        "KFFZ": "small_airport",
                        "KFHR": "small_airport",
                        "KFHU": "medium_airport",
                        "KFKL": "medium_airport",
                        "KFKN": "small_airport",
                        "KFLD": "small_airport",
                        "KFLG": "medium_airport",
                        "KFLL": "large_airport",
                        "KFLO": "medium_airport",
                        "KFLP": "small_airport",
                        "KFLU": "closed",
                        "KFLV": "small_airport",
                        "KFLX": "small_airport",
                        "KFME": "medium_airport",
                        "KFMH": "small_airport",
                        "KFMN": "medium_airport",
                        "KFMY": "medium_airport",
                        "KFNL": "small_airport",
                        "KFNT": "medium_airport",
                        "KFOD": "medium_airport",
                        "KFOE": "medium_airport",
                        "KFOK": "small_airport",
                        "KFOM": "small_airport",
                        "KFPR": "medium_airport",
                        "KFRG": "medium_airport",
                        "KFRH": "small_airport",
                        "KFRI": "medium_airport",
                        "KFRM": "small_airport",
                        "KFRR": "small_airport",
                        "KFSD": "medium_airport",
                        "KFSI": "medium_airport",
                        "KFSK": "small_airport",
                        "KFSM": "large_airport",
                        "KFST": "medium_airport",
                        "KFSU": "small_airport",
                        "KFSW": "small_airport",
                        "KFTK": "medium_airport",
                        "KFTW": "large_airport",
                        "KFTY": "medium_airport",
                        "KFUL": "small_airport",
                        "KFVE": "small_airport",
                        "KFWA": "large_airport",
                        "KFXE": "medium_airport",
                        "KFXY": "small_airport",
                        "KFYM": "small_airport",
                        "KFYV": "medium_airport",
                        "KGAD": "small_airport",
                        "KGAG": "small_airport",
                        "KGAI": "small_airport",
                        "KGBD": "small_airport",
                        "KGBG": "small_airport",
                        "KGBR": "small_airport",
                        "KGCC": "medium_airport",
                        "KGCD": "small_airport",
                        "KGCK": "medium_airport",
                        "KGCN": "medium_airport",
                        "KGCY": "small_airport",
                        "KGDM": "small_airport",
                        "KGDV": "medium_airport",
                        "KGDW": "small_airport",
                        "KGED": "small_airport",
                        "KGEG": "large_airport",
                        "KGEY": "small_airport",
                        "KGFA": "heliport",
                        "KGFK": "medium_airport",
                        "KGFL": "medium_airport",
                        "KGGE": "small_airport",
                        "KGGG": "medium_airport",
                        "KGGW": "medium_airport",
                        "KGHM": "small_airport",
                        "KGIF": "small_airport",
                        "KGJT": "medium_airport",
                        "KGKJ": "small_airport",
                        "KGKT": "small_airport",
                        "KGLD": "medium_airport",
                        "KGLE": "small_airport",
                        "KGLH": "medium_airport",
                        "KGLR": "small_airport",
                        "KGLS": "medium_airport",
                        "KGLW": "small_airport",
                        "KGMU": "medium_airport",
                        "KGNG": "small_airport",
                        "KGNT": "small_airport",
                        "KGNV": "medium_airport",
                        "KGOK": "small_airport",
                        "KGON": "medium_airport",
                        "KGPI": "medium_airport",
                        "KGPT": "large_airport",
                        "KGPZ": "small_airport",
                        "KGQQ": "small_airport",
                        "KGRB": "large_airport",
                        "KGRD": "small_airport",
                        "KGRE": "small_airport",
                        "KGRF": "medium_airport",
                        "KGRI": "medium_airport",
                        "KGRK": "medium_airport",
                        "KGRN": "small_airport",
                        "KGRR": "medium_airport",
                        "KGSB": "large_airport",
                        "KGSH": "small_airport",
                        "KGSO": "large_airport",
                        "KGSP": "large_airport",
                        "KGSW": "closed",
                        "KGTF": "medium_airport",
                        "KGTG": "small_airport",
                        "KGTR": "medium_airport",
                        "KGUC": "small_airport",
                        "KGUP": "medium_airport",
                        "KGUS": "large_airport",
                        "KGUY": "medium_airport",
                        "KGVL": "small_airport",
                        "KGVT": "small_airport",
                        "KGVW": "closed",
                        "KGW": "small_airport",
                        "KGWO": "medium_airport",
                        "KGWS": "small_airport",
                        "KGX": "small_airport",
                        "KGXY": "small_airport",
                        "KGYH": "small_airport",
                        "KGYI": "small_airport",
                        "KGYR": "small_airport",
                        "KGYY": "medium_airport",
                        "KGZ": "small_airport",
                        "KHAB": "small_airport",
                        "KHAF": "small_airport",
                        "KHAI": "small_airport",
                        "KHAO": "small_airport",
                        "KHAX": "closed",
                        "KHBG": "medium_airport",
                        "KHBR": "medium_airport",
                        "KHDE": "small_airport",
                        "KHDN": "small_airport",
                        "KHEE": "small_airport",
                        "KHEF": "small_airport",
                        "KHEZ": "small_airport",
                        "KHFD": "medium_airport",
                        "KHFF": "small_airport",
                        "KHGR": "medium_airport",
                        "KHHR": "medium_airport",
                        "KHHW": "small_airport",
                        "KHIB": "large_airport",
                        "KHIE": "small_airport",
                        "KHIF": "medium_airport",
                        "KHII": "small_airport",
                        "KHIO": "medium_airport",
                        "KHKA": "small_airport",
                        "KHKS": "small_airport",
                        "KHKY": "medium_airport",
                        "KHL": "closed",
                        "KHLB": "small_airport",
                        "KHLC": "small_airport",
                        "KHLG": "medium_airport",
                        "KHLM": "small_airport",
                        "KHLN": "medium_airport",
                        "KHLR": "medium_airport",
                        "KHMN": "large_airport",
                        "KHMT": "small_airport",
                        "KHNB": "small_airport",
                        "KHND": "small_airport",
                        "KHOB": "medium_airport",
                        "KHON": "medium_airport",
                        "KHOP": "medium_airport",
                        "KHOT": "medium_airport",
                        "KHOU": "large_airport",
                        "KHPN": "medium_airport",
                        "KHPT": "small_airport",
                        "KHPY": "small_airport",
                        "KHQM": "medium_airport",
                        "KHRI": "small_airport",
                        "KHRL": "medium_airport",
                        "KHRO": "medium_airport",
                        "KHSB": "small_airport",
                        "KHSE": "small_airport",
                        "KHSG": "small_airport",
                        "KHSI": "small_airport",
                        "KHSP": "small_airport",
                        "KHST": "medium_airport",
                        "KHSV": "large_airport",
                        "KHTH": "small_airport",
                        "KHTL": "small_airport",
                        "KHTO": "small_airport",
                        "KHTS": "large_airport",
                        "KHTW": "small_airport",
                        "KHUA": "medium_airport",
                        "KHUF": "medium_airport",
                        "KHUL": "medium_airport",
                        "KHUM": "small_airport",
                        "KHUT": "medium_airport",
                        "KHVE": "small_airport",
                        "KHVN": "medium_airport",
                        "KHVR": "medium_airport",
                        "KHVS": "small_airport",
                        "KHWD": "small_airport",
                        "KHWO": "small_airport",
                        "KHWV": "small_airport",
                        "KHXD": "small_airport",
                        "KHYA": "medium_airport",
                        "KHYR": "medium_airport",
                        "KHYS": "medium_airport",
                        "KHZL": "small_airport",
                        "KHZY": "small_airport",
                        "KIA": "closed",
                        "KIAB": "medium_airport",
                        "KIAD": "large_airport",
                        "KIAG": "medium_airport",
                        "KIAH": "large_airport",
                        "KIB": "seaplane_base",
                        "KICL": "small_airport",
                        "KICT": "large_airport",
                        "KIDA": "medium_airport",
                        "KIDI": "small_airport",
                        "KIDP": "small_airport",
                        "KIEN": "small_airport",
                        "KIFA": "small_airport",
                        "KIFP": "small_airport",
                        "KIGM": "small_airport",
                        "KIKK": "medium_airport",
                        "KIL": "small_airport",
                        "KILE": "small_airport",
                        "KILG": "medium_airport",
                        "KILM": "medium_airport",
                        "KILN": "medium_airport",
                        "KIML": "small_airport",
                        "KIMM": "small_airport",
                        "KIMS": "small_airport",
                        "KIMT": "small_airport",
                        "KIND": "large_airport",
                        "KINK": "medium_airport",
                        "KINL": "medium_airport",
                        "KINS": "small_airport",
                        "KINT": "medium_airport",
                        "KINW": "medium_airport",
                        "KIOW": "small_airport",
                        "KIPL": "medium_airport",
                        "KIPT": "medium_airport",
                        "KIQ": "small_airport",
                        "KIRK": "medium_airport",
                        "KIRS": "small_airport",
                        "KISM": "medium_airport",
                        "KISN": "medium_airport",
                        "KISO": "medium_airport",
                        "KISP": "medium_airport",
                        "KISQ": "small_airport",
                        "KISW": "small_airport",
                        "KITH": "medium_airport",
                        "KIWA": "medium_airport",
                        "KIWD": "small_airport",
                        "KIWI": "small_airport",
                        "KIWS": "small_airport",
                        "KIXD": "small_airport",
                        "KIYK": "small_airport",
                        "KIZ": "closed",
                        "KIZA": "small_airport",
                        "KIZG": "small_airport",
                        "KJAC": "medium_airport",
                        "KJAN": "large_airport",
                        "KJAS": "small_airport",
                        "KJAX": "large_airport",
                        "KJBR": "medium_airport",
                        "KJCT": "medium_airport",
                        "KJDN": "small_airport",
                        "KJEF": "small_airport",
                        "KJFK": "large_airport",
                        "KJHW": "medium_airport",
                        "KJKA": "small_airport",
                        "KJLN": "large_airport",
                        "KJMS": "medium_airport",
                        "KJOT": "small_airport",
                        "KJQF": "small_airport",
                        "KJSO": "small_airport",
                        "KJST": "medium_airport",
                        "KJU": "closed",
                        "KJVL": "small_airport",
                        "KJXN": "medium_airport",
                        "KKB": "seaplane_base",
                        "KKIC": "small_airport",
                        "KKL": "seaplane_base",
                        "KKLS": "medium_airport",
                        "KKU": "small_airport",
                        "KL06": "small_airport",
                        "KL08": "small_airport",
                        "KL35": "small_airport",
                        "KL72": "small_airport",
                        "KLAA": "small_airport",
                        "KLAF": "medium_airport",
                        "KLAL": "medium_airport",
                        "KLAM": "small_airport",
                        "KLAN": "medium_airport",
                        "KLAR": "medium_airport",
                        "KLAS": "large_airport",
                        "KLAW": "medium_airport",
                        "KLAX": "large_airport",
                        "KLBB": "large_airport",
                        "KLBE": "medium_airport",
                        "KLBF": "medium_airport",
                        "KLBL": "medium_airport",
                        "KLBT": "medium_airport",
                        "KLBX": "medium_airport",
                        "KLCH": "medium_airport",
                        "KLCI": "small_airport",
                        "KLCK": "large_airport",
                        "KLCQ": "small_airport",
                        "KLDJ": "small_airport",
                        "KLDM": "small_airport",
                        "KLEB": "medium_airport",
                        "KLEE": "medium_airport",
                        "KLEM": "small_airport",
                        "KLEW": "small_airport",
                        "KLEX": "large_airport",
                        "KLFI": "large_airport",
                        "KLFK": "medium_airport",
                        "KLFT": "large_airport",
                        "KLGA": "large_airport",
                        "KLGB": "medium_airport",
                        "KLGC": "small_airport",
                        "KLGD": "small_airport",
                        "KLGF": "small_airport",
                        "KLGU": "medium_airport",
                        "KLHC": "closed",
                        "KLHV": "small_airport",
                        "KLHW": "small_airport",
                        "KLHZ": "small_airport",
                        "KLIC": "small_airport",
                        "KLIT": "large_airport",
                        "KLKP": "small_airport",
                        "KLKU": "small_airport",
                        "KLKV": "small_airport",
                        "KLLJ": "small_airport",
                        "KLMS": "small_airport",
                        "KLMT": "medium_airport",
                        "KLNA": "small_airport",
                        "KLND": "medium_airport",
                        "KLNK": "medium_airport",
                        "KLNN": "small_airport",
                        "KLNP": "small_airport",
                        "KLNR": "small_airport",
                        "KLNS": "medium_airport",
                        "KLOL": "medium_airport",
                        "KLOM": "small_airport",
                        "KLOT": "small_airport",
                        "KLOU": "medium_airport",
                        "KLOZ": "medium_airport",
                        "KLPC": "small_airport",
                        "KLQK": "small_airport",
                        "KLRD": "medium_airport",
                        "KLRF": "medium_airport",
                        "KLRJ": "small_airport",
                        "KLRU": "medium_airport",
                        "KLSB": "small_airport",
                        "KLSE": "medium_airport",
                        "KLSF": "medium_airport",
                        "KLSK": "small_airport",
                        "KLSN": "small_airport",
                        "KLSV": "medium_airport",
                        "KLTS": "large_airport",
                        "KLUF": "large_airport",
                        "KLUK": "medium_airport",
                        "KLUL": "small_airport",
                        "KLVK": "small_airport",
                        "KLVL": "small_airport",
                        "KLVM": "medium_airport",
                        "KLVS": "medium_airport",
                        "KLWB": "medium_airport",
                        "KLWC": "small_airport",
                        "KLWL": "small_airport",
                        "KLWM": "medium_airport",
                        "KLWS": "medium_airport",
                        "KLWT": "medium_airport",
                        "KLWV": "small_airport",
                        "KLXN": "small_airport",
                        "KLXV": "small_airport",
                        "KLYH": "medium_airport",
                        "KLYO": "small_airport",
                        "KLZU": "small_airport",
                        "KM13": "small_airport",
                        "KM75": "small_airport",
                        "KMAC": "small_airport",
                        "KMAE": "small_airport",
                        "KMAF": "medium_airport",
                        "KMAW": "small_airport",
                        "KMB": "small_airport",
                        "KMBG": "medium_airport",
                        "KMBL": "small_airport",
                        "KMBO": "small_airport",
                        "KMBS": "large_airport",
                        "KMBY": "small_airport",
                        "KMCB": "medium_airport",
                        "KMCC": "medium_airport",
                        "KMCD": "small_airport",
                        "KMCE": "medium_airport",
                        "KMCF": "large_airport",
                        "KMCI": "large_airport",
                        "KMCK": "medium_airport",
                        "KMCN": "medium_airport",
                        "KMCO": "large_airport",
                        "KMCW": "medium_airport",
                        "KMDA": "heliport",
                        "KMDD": "small_airport",
                        "KMDH": "medium_airport",
                        "KMDS": "small_airport",
                        "KMDT": "medium_airport",
                        "KMDW": "large_airport",
                        "KMDZ": "small_airport",
                        "KMEB": "small_airport",
                        "KMEI": "medium_airport",
                        "KMEM": "large_airport",
                        "KMER": "medium_airport",
                        "KMEV": "small_airport",
                        "KMF": "small_airport",
                        "KMFD": "medium_airport",
                        "KMFE": "medium_airport",
                        "KMFI": "small_airport",
                        "KMFR": "medium_airport",
                        "KMFV": "small_airport",
                        "KMGC": "small_airport",
                        "KMGE": "large_airport",
                        "KMGJ": "small_airport",
                        "KMGM": "large_airport",
                        "KMGR": "small_airport",
                        "KMGW": "medium_airport",
                        "KMGY": "small_airport",
                        "KMHE": "small_airport",
                        "KMHK": "medium_airport",
                        "KMHL": "small_airport",
                        "KMHR": "medium_airport",
                        "KMHT": "large_airport",
                        "KMHV": "small_airport",
                        "KMIA": "large_airport",
                        "KMIB": "medium_airport",
                        "KMIE": "medium_airport",
                        "KMIT": "small_airport",
                        "KMIV": "medium_airport",
                        "KMJX": "small_airport",
                        "KMKC": "medium_airport",
                        "KMKE": "large_airport",
                        "KMKG": "medium_airport",
                        "KMKL": "medium_airport",
                        "KMKY": "small_airport",
                        "KMLB": "medium_airport",
                        "KMLC": "medium_airport",
                        "KMLI": "large_airport",
                        "KMLS": "medium_airport",
                        "KMLU": "large_airport",
                        "KMM": "small_airport",
                        "KMMH": "small_airport",
                        "KMMI": "small_airport",
                        "KMML": "small_airport",
                        "KMMS": "small_airport",
                        "KMMT": "medium_airport",
                        "KMMU": "medium_airport",
                        "KMNM": "small_airport",
                        "KMNN": "small_airport",
                        "KMOB": "large_airport",
                        "KMOD": "medium_airport",
                        "KMOT": "medium_airport",
                        "KMPI": "small_airport",
                        "KMPJ": "small_airport",
                        "KMPO": "small_airport",
                        "KMPV": "medium_airport",
                        "KMPZ": "small_airport",
                        "KMQB": "small_airport",
                        "KMQI": "small_airport",
                        "KMQS": "small_airport",
                        "KMQY": "medium_airport",
                        "KMRB": "medium_airport",
                        "KMRC": "small_airport",
                        "KMRF": "small_airport",
                        "KMRN": "small_airport",
                        "KMRY": "medium_airport",
                        "KMSL": "medium_airport",
                        "KMSN": "large_airport",
                        "KMSO": "medium_airport",
                        "KMSP": "large_airport",
                        "KMSS": "medium_airport",
                        "KMSV": "small_airport",
                        "KMSY": "large_airport",
                        "KMT": "closed",
                        "KMTC": "medium_airport",
                        "KMTH": "medium_airport",
                        "KMTJ": "medium_airport",
                        "KMTN": "medium_airport",
                        "KMTO": "small_airport",
                        "KMTP": "small_airport",
                        "KMTW": "small_airport",
                        "KMUI": "medium_airport",
                        "KMUO": "large_airport",
                        "KMUT": "small_airport",
                        "KMVC": "small_airport",
                        "KMVE": "small_airport",
                        "KMVL": "small_airport",
                        "KMVY": "small_airport",
                        "KMWA": "small_airport",
                        "KMWC": "small_airport",
                        "KMWH": "medium_airport",
                        "KMWL": "small_airport",
                        "KMWO": "small_airport",
                        "KMXA": "small_airport",
                        "KMXF": "medium_airport",
                        "KMYF": "small_airport",
                        "KMYL": "medium_airport",
                        "KMYR": "medium_airport",
                        "KMYV": "medium_airport",
                        "KMZJ": "small_airport",
                        "KMZZ": "small_airport",
                        "KN03": "small_airport",
                        "KN23": "small_airport",
                        "KN53": "small_airport",
                        "KNBG": "medium_airport",
                        "KNBJ": "small_airport",
                        "KNDY": "small_airport",
                        "KNEL": "medium_airport",
                        "KNEN": "small_airport",
                        "KNEW": "medium_airport",
                        "KNFL": "medium_airport",
                        "KNFW": "medium_airport",
                        "KNGU": "medium_airport",
                        "KNGZ": "closed",
                        "KNHK": "medium_airport",
                        "KNHZ": "medium_airport",
                        "KNIP": "medium_airport",
                        "KNJK": "medium_airport",
                        "KNKX": "medium_airport",
                        "KNL": "closed",
                        "KNLC": "medium_airport",
                        "KNPA": "medium_airport",
                        "KNQA": "medium_airport",
                        "KNQI": "medium_airport",
                        "KNQX": "medium_airport",
                        "KNRB": "medium_airport",
                        "KNRS": "small_airport",
                        "KNSE": "medium_airport",
                        "KNTD": "medium_airport",
                        "KNTU": "medium_airport",
                        "KNUQ": "medium_airport",
                        "KNUW": "medium_airport",
                        "KNYL": "medium_airport",
                        "KNZY": "medium_airport",
                        "KO02": "small_airport",
                        "KO22": "small_airport",
                        "KO27": "small_airport",
                        "KO43": "small_airport",
                        "KOAJ": "medium_airport",
                        "KOAK": "large_airport",
                        "KOAR": "small_airport",
                        "KOBE": "small_airport",
                        "KOCF": "small_airport",
                        "KOCH": "small_airport",
                        "KOCW": "small_airport",
                        "KOEA": "small_airport",
                        "KOEO": "small_airport",
                        "KOFF": "medium_airport",
                        "KOFK": "medium_airport",
                        "KOGA": "small_airport",
                        "KOGB": "medium_airport",
                        "KOGD": "medium_airport",
                        "KOGS": "small_airport",
                        "KOIC": "small_airport",
                        "KOIL": "closed",
                        "KOJC": "small_airport",
                        "KOKB": "small_airport",
                        "KOKC": "large_airport",
                        "KOKH": "small_airport",
                        "KOKK": "small_airport",
                        "KOKM": "small_airport",
                        "KOKS": "small_airport",
                        "KOKV": "small_airport",
                        "KOL": "closed",
                        "KOLD": "small_airport",
                        "KOLF": "medium_airport",
                        "KOLM": "medium_airport",
                        "KOLS": "medium_airport",
                        "KOLV": "small_airport",
                        "KOM": "small_airport",
                        "KOMA": "large_airport",
                        "KOMK": "small_airport",
                        "KONL": "small_airport",
                        "KONO": "small_airport",
                        "KONP": "medium_airport",
                        "KONT": "large_airport",
                        "KOPF": "medium_airport",
                        "KOQU": "medium_airport",
                        "KOR": "small_airport",
                        "KORD": "large_airport",
                        "KORF": "large_airport",
                        "KORH": "medium_airport",
                        "KORL": "small_airport",
                        "KORS": "small_airport",
                        "KOSC": "small_airport",
                        "KOSH": "medium_airport",
                        "KOSU": "medium_airport",
                        "KOTH": "medium_airport",
                        "KOTM": "medium_airport",
                        "KOUN": "small_airport",
                        "KOVE": "small_airport",
                        "KOWA": "small_airport",
                        "KOWB": "medium_airport",
                        "KOWD": "medium_airport",
                        "KOWK": "small_airport",
                        "KOXB": "small_airport",
                        "KOXC": "small_airport",
                        "KOXD": "small_airport",
                        "KOXR": "medium_airport",
                        "KOY": "seaplane_base",
                        "KOYM": "small_airport",
                        "KOZA": "small_airport",
                        "KOZR": "medium_airport",
                        "KP-0029": "small_airport",
                        "KP-0032": "medium_airport",
                        "KP04": "small_airport",
                        "KP10": "small_airport",
                        "KP13": "small_airport",
                        "KP14": "small_airport",
                        "KP33": "small_airport",
                        "KPAE": "medium_airport",
                        "KPAH": "medium_airport",
                        "KPAM": "large_airport",
                        "KPAN": "small_airport",
                        "KPAO": "small_airport",
                        "KPB": "seaplane_base",
                        "KPBF": "medium_airport",
                        "KPBG": "medium_airport",
                        "KPBI": "large_airport",
                        "KPBX": "small_airport",
                        "KPDC": "small_airport",
                        "KPDK": "medium_airport",
                        "KPDT": "medium_airport",
                        "KPDX": "large_airport",
                        "KPE": "small_airport",
                        "KPEQ": "small_airport",
                        "KPGA": "medium_airport",
                        "KPGD": "small_airport",
                        "KPGR": "small_airport",
                        "KPGV": "medium_airport",
                        "KPHD": "small_airport",
                        "KPHF": "large_airport",
                        "KPHH": "small_airport",
                        "KPHK": "small_airport",
                        "KPHL": "large_airport",
                        "KPHN": "small_airport",
                        "KPHP": "small_airport",
                        "KPHT": "small_airport",
                        "KPHX": "large_airport",
                        "KPIA": "large_airport",
                        "KPIB": "medium_airport",
                        "KPIE": "medium_airport",
                        "KPIH": "medium_airport",
                        "KPIM": "small_airport",
                        "KPIR": "medium_airport",
                        "KPIT": "large_airport",
                        "KPKB": "medium_airport",
                        "KPKD": "small_airport",
                        "KPKF": "small_airport",
                        "KPL": "small_airport",
                        "KPLB": "closed",
                        "KPLK": "small_airport",
                        "KPLN": "medium_airport",
                        "KPLR": "small_airport",
                        "KPMB": "small_airport",
                        "KPMD": "medium_airport",
                        "KPMH": "small_airport",
                        "KPMP": "small_airport",
                        "KPNA": "medium_airport",
                        "KPNC": "medium_airport",
                        "KPNE": "medium_airport",
                        "KPNN": "small_airport",
                        "KPNS": "medium_airport",
                        "KPOB": "medium_airport",
                        "KPOC": "small_airport",
                        "KPOE": "medium_airport",
                        "KPOF": "small_airport",
                        "KPOU": "medium_airport",
                        "KPOY": "small_airport",
                        "KPPA": "small_airport",
                        "KPPF": "small_airport",
                        "KPPO": "small_airport",
                        "KPQI": "medium_airport",
                        "KPQL": "small_airport",
                        "KPR": "seaplane_base",
                        "KPRB": "medium_airport",
                        "KPRC": "medium_airport",
                        "KPRO": "small_airport",
                        "KPRX": "medium_airport",
                        "KPSC": "medium_airport",
                        "KPSF": "small_airport",
                        "KPSK": "small_airport",
                        "KPSM": "medium_airport",
                        "KPSN": "small_airport",
                        "KPSO": "small_airport",
                        "KPSP": "medium_airport",
                        "KPSX": "small_airport",
                        "KPTB": "small_airport",
                        "KPTK": "medium_airport",
                        "KPTN": "small_airport",
                        "KPTT": "small_airport",
                        "KPTV": "small_airport",
                        "KPTW": "small_airport",
                        "KPUB": "small_airport",
                        "KPUC": "small_airport",
                        "KPUW": "medium_airport",
                        "KPVC": "small_airport",
                        "KPVD": "large_airport",
                        "KPVF": "small_airport",
                        "KPVU": "medium_airport",
                        "KPVW": "small_airport",
                        "KPVZ": "closed",
                        "KPWA": "small_airport",
                        "KPWD": "small_airport",
                        "KPWK": "medium_airport",
                        "KPWM": "large_airport",
                        "KPWT": "medium_airport",
                        "KPY": "seaplane_base",
                        "KPYM": "small_airport",
                        "KQA": "seaplane_base",
                        "KQL": "small_airport",
                        "KR-0018": "heliport",
                        "KRAC": "small_airport",
                        "KRAL": "medium_airport",
                        "KRAP": "medium_airport",
                        "KRBD": "small_airport",
                        "KRBG": "small_airport",
                        "KRBL": "medium_airport",
                        "KRBW": "small_airport",
                        "KRCA": "medium_airport",
                        "KRCK": "small_airport",
                        "KRCR": "small_airport",
                        "KRCT": "small_airport",
                        "KRDD": "medium_airport",
                        "KRDG": "medium_airport",
                        "KRDM": "medium_airport",
                        "KRDR": "medium_airport",
                        "KRDU": "large_airport",
                        "KREO": "small_airport",
                        "KRFD": "large_airport",
                        "KRFG": "small_airport",
                        "KRHI": "medium_airport",
                        "KRHV": "small_airport",
                        "KRIC": "large_airport",
                        "KRIL": "medium_airport",
                        "KRIV": "medium_airport",
                        "KRIW": "medium_airport",
                        "KRJ": "small_airport",
                        "KRKD": "small_airport",
                        "KRKP": "small_airport",
                        "KRKS": "medium_airport",
                        "KRKW": "small_airport",
                        "KRME": "medium_airport",
                        "KRMG": "medium_airport",
                        "KRNC": "small_airport",
                        "KRND": "large_airport",
                        "KRNO": "large_airport",
                        "KRNT": "small_airport",
                        "KROA": "large_airport",
                        "KROC": "large_airport",
                        "KROG": "small_airport",
                        "KROW": "medium_airport",
                        "KROX": "small_airport",
                        "KRPD": "small_airport",
                        "KRPX": "small_airport",
                        "KRQB": "small_airport",
                        "KRQO": "small_airport",
                        "KRRL": "small_airport",
                        "KRRT": "small_airport",
                        "KRSL": "medium_airport",
                        "KRSN": "small_airport",
                        "KRST": "large_airport",
                        "KRSW": "large_airport",
                        "KRTN": "small_airport",
                        "KRU": "small_airport",
                        "KRUQ": "small_airport",
                        "KRUT": "medium_airport",
                        "KRVL": "small_airport",
                        "KRVS": "medium_airport",
                        "KRWF": "medium_airport",
                        "KRWI": "medium_airport",
                        "KRWL": "medium_airport",
                        "KRXE": "small_airport",
                        "KRZL": "small_airport",
                        "KRZZ": "closed",
                        "KS03": "small_airport",
                        "KS05": "small_airport",
                        "KS21": "small_airport",
                        "KS33": "small_airport",
                        "KS39": "small_airport",
                        "KS80": "small_airport",
                        "KSAA": "small_airport",
                        "KSAC": "medium_airport",
                        "KSAD": "small_airport",
                        "KSAF": "medium_airport",
                        "KSAN": "large_airport",
                        "KSAR": "small_airport",
                        "KSAT": "large_airport",
                        "KSAV": "large_airport",
                        "KSAW": "medium_airport",
                        "KSBA": "medium_airport",
                        "KSBD": "medium_airport",
                        "KSBM": "small_airport",
                        "KSBN": "large_airport",
                        "KSBP": "medium_airport",
                        "KSBS": "small_airport",
                        "KSBX": "small_airport",
                        "KSBY": "medium_airport",
                        "KSCB": "small_airport",
                        "KSCH": "medium_airport",
                        "KSCK": "medium_airport",
                        "KSDF": "large_airport",
                        "KSDL": "small_airport",
                        "KSDM": "medium_airport",
                        "KSDY": "medium_airport",
                        "KSEA": "large_airport",
                        "KSEE": "small_airport",
                        "KSEF": "small_airport",
                        "KSEG": "small_airport",
                        "KSEM": "small_airport",
                        "KSEP": "small_airport",
                        "KSER": "small_airport",
                        "KSEZ": "small_airport",
                        "KSFB": "large_airport",
                        "KSFF": "medium_airport",
                        "KSFM": "small_airport",
                        "KSFO": "large_airport",
                        "KSFZ": "small_airport",
                        "KSG": "closed",
                        "KSGF": "large_airport",
                        "KSGH": "medium_airport",
                        "KSGJ": "medium_airport",
                        "KSGR": "medium_airport",
                        "KSGT": "small_airport",
                        "KSGU": "medium_airport",
                        "KSHD": "small_airport",
                        "KSHN": "small_airport",
                        "KSHR": "medium_airport",
                        "KSHV": "medium_airport",
                        "KSIK": "small_airport",
                        "KSIV": "small_airport",
                        "KSJC": "large_airport",
                        "KSJN": "small_airport",
                        "KSJT": "medium_airport",
                        "KSKA": "large_airport",
                        "KSKF": "medium_airport",
                        "KSKX": "small_airport",
                        "KSKY": "closed",
                        "KSLB": "small_airport",
                        "KSLC": "large_airport",
                        "KSLE": "medium_airport",
                        "KSLG": "small_airport",
                        "KSLK": "medium_airport",
                        "KSLN": "medium_airport",
                        "KSLO": "small_airport",
                        "KSLR": "small_airport",
                        "KSMD": "small_airport",
                        "KSME": "medium_airport",
                        "KSMF": "large_airport",
                        "KSMN": "medium_airport",
                        "KSMO": "small_airport",
                        "KSMS": "small_airport",
                        "KSMX": "medium_airport",
                        "KSNA": "large_airport",
                        "KSNK": "small_airport",
                        "KSNL": "small_airport",
                        "KSNS": "medium_airport",
                        "KSNY": "medium_airport",
                        "KSOP": "small_airport",
                        "KSOW": "small_airport",
                        "KSP": "small_airport",
                        "KSPA": "small_airport",
                        "KSPF": "small_airport",
                        "KSPG": "small_airport",
                        "KSPI": "large_airport",
                        "KSPS": "large_airport",
                        "KSPW": "small_airport",
                        "KSQI": "small_airport",
                        "KSQL": "small_airport",
                        "KSRQ": "large_airport",
                        "KSRR": "medium_airport",
                        "KSSC": "large_airport",
                        "KSSF": "medium_airport",
                        "KSSI": "medium_airport",
                        "KSTC": "medium_airport",
                        "KSTE": "small_airport",
                        "KSTJ": "medium_airport",
                        "KSTK": "small_airport",
                        "KSTL": "large_airport",
                        "KSTP": "medium_airport",
                        "KSTS": "medium_airport",
                        "KSUA": "small_airport",
                        "KSUD": "small_airport",
                        "KSUE": "small_airport",
                        "KSUN": "medium_airport",
                        "KSUS": "large_airport",
                        "KSUU": "large_airport",
                        "KSUW": "small_airport",
                        "KSUX": "large_airport",
                        "KSVC": "small_airport",
                        "KSVE": "small_airport",
                        "KSVH": "small_airport",
                        "KSVN": "medium_airport",
                        "KSWF": "medium_airport",
                        "KSWO": "medium_airport",
                        "KSWW": "small_airport",
                        "KSYI": "small_airport",
                        "KSYR": "large_airport",
                        "KSYV": "small_airport",
                        "KSZL": "large_airport",
                        "KT03": "small_airport",
                        "KTAD": "small_airport",
                        "KTB": "seaplane_base",
                        "KTBN": "medium_airport",
                        "KTBR": "small_airport",
                        "KTC": "small_airport",
                        "KTCC": "medium_airport",
                        "KTCL": "medium_airport",
                        "KTCM": "large_airport",
                        "KTCS": "medium_airport",
                        "KTDO": "small_airport",
                        "KTDW": "small_airport",
                        "KTDZ": "small_airport",
                        "KTEB": "medium_airport",
                        "KTEX": "small_airport",
                        "KTHA": "small_airport",
                        "KTHM": "small_airport",
                        "KTHV": "small_airport",
                        "KTIK": "large_airport",
                        "KTIW": "medium_airport",
                        "KTIX": "medium_airport",
                        "KTK": "closed",
                        "KTKX": "small_airport",
                        "KTLH": "large_airport",
                        "KTLR": "small_airport",
                        "KTMA": "small_airport",
                        "KTMB": "medium_airport",
                        "KTMK": "small_airport",
                        "KTNP": "small_airport",
                        "KTNT": "small_airport",
                        "KTNU": "small_airport",
                        "KTNX": "small_airport",
                        "KTOA": "small_airport",
                        "KTOC": "small_airport",
                        "KTOI": "medium_airport",
                        "KTOL": "large_airport",
                        "KTOP": "medium_airport",
                        "KTOR": "small_airport",
                        "KTPA": "large_airport",
                        "KTPF": "small_airport",
                        "KTPH": "medium_airport",
                        "KTPL": "medium_airport",
                        "KTRI": "large_airport",
                        "KTRK": "medium_airport",
                        "KTRL": "small_airport",
                        "KTRM": "medium_airport",
                        "KTSP": "small_airport",
                        "KTTD": "medium_airport",
                        "KTTN": "medium_airport",
                        "KTUL": "large_airport",
                        "KTUP": "medium_airport",
                        "KTUS": "large_airport",
                        "KTVC": "medium_airport",
                        "KTVF": "small_airport",
                        "KTVI": "small_airport",
                        "KTVL": "medium_airport",
                        "KTWF": "medium_airport",
                        "KTXK": "medium_airport",
                        "KTYL": "small_airport",
                        "KTYR": "medium_airport",
                        "KTYS": "large_airport",
                        "KU07": "small_airport",
                        "KU14": "small_airport",
                        "KU34": "small_airport",
                        "KU55": "small_airport",
                        "KU70": "small_airport",
                        "KUBS": "small_airport",
                        "KUCY": "small_airport",
                        "KUDD": "small_airport",
                        "KUES": "small_airport",
                        "KUGN": "small_airport",
                        "KUIL": "small_airport",
                        "KUIN": "medium_airport",
                        "KUIZ": "closed",
                        "KUKF": "small_airport",
                        "KUKI": "medium_airport",
                        "KUKT": "small_airport",
                        "KULM": "small_airport",
                        "KUNI": "small_airport",
                        "KUNU": "small_airport",
                        "KUNV": "medium_airport",
                        "KUOS": "small_airport",
                        "KUOX": "medium_airport",
                        "KUP": "small_airport",
                        "KUTA": "small_airport",
                        "KUTS": "medium_airport",
                        "KUUU": "small_airport",
                        "KUVA": "small_airport",
                        "KUX": "small_airport",
                        "KUZA": "small_airport",
                        "KVAD": "medium_airport",
                        "KVAY": "small_airport",
                        "KVBG": "large_airport",
                        "KVCT": "medium_airport",
                        "KVCV": "small_airport",
                        "KVDI": "small_airport",
                        "KVE": "small_airport",
                        "KVEL": "medium_airport",
                        "KVGT": "medium_airport",
                        "KVHN": "small_airport",
                        "KVIH": "small_airport",
                        "KVIS": "small_airport",
                        "KVJI": "small_airport",
                        "KVKS": "small_airport",
                        "KVLA": "small_airport",
                        "KVLD": "medium_airport",
                        "KVNC": "small_airport",
                        "KVNY": "medium_airport",
                        "KVOK": "medium_airport",
                        "KVPS": "large_airport",
                        "KVPZ": "medium_airport",
                        "KVQQ": "small_airport",
                        "KVRB": "medium_airport",
                        "KVSF": "small_airport",
                        "KVTN": "medium_airport",
                        "KVYS": "small_airport",
                        "KW05": "small_airport",
                        "KW28": "small_airport",
                        "KW99": "small_airport",
                        "KWAL": "small_airport",
                        "KWAY": "small_airport",
                        "KWBW": "small_airport",
                        "KWDG": "small_airport",
                        "KWDR": "small_airport",
                        "KWF": "seaplane_base",
                        "KWHP": "small_airport",
                        "KWJF": "medium_airport",
                        "KWLD": "small_airport",
                        "KWLW": "small_airport",
                        "KWMC": "medium_airport",
                        "KWP": "seaplane_base",
                        "KWRB": "large_airport",
                        "KWRI": "medium_airport",
                        "KWRL": "medium_airport",
                        "KWSD": "small_airport",
                        "KWST": "small_airport",
                        "KWV": "closed",
                        "KWVI": "small_airport",
                        "KWVL": "small_airport",
                        "KWWD": "medium_airport",
                        "KWWR": "medium_airport",
                        "KWX": "closed",
                        "KWY": "medium_airport",
                        "KWYS": "medium_airport",
                        "KX39": "small_airport",
                        "KXA": "seaplane_base",
                        "KXNA": "medium_airport",
                        "KY31": "small_airport",
                        "KYIP": "medium_airport",
                        "KYKM": "medium_airport",
                        "KYKN": "medium_airport",
                        "KYL": "closed",
                        "KYNG": "medium_airport",
                        "KZ-0016": "medium_airport",
                        "KZ-0067": "medium_airport",
                        "KZ-0082": "small_airport",
                        "KZ-ATX": "small_airport",
                        "KZB": "seaplane_base",
                        "KZF": "small_airport",
                        "KZPH": "small_airport",
                        "KZZV": "medium_airport",
                        "LAC": "small_airport",
                        "LATI": "large_airport",
                        "LBBG": "large_airport",
                        "LBGO": "medium_airport",
                        "LBH": "seaplane_base",
                        "LBM": "small_airport",
                        "LBPD": "medium_airport",
                        "LBRS": "closed",
                        "LBSF": "large_airport",
                        "LBSS": "closed",
                        "LBSZ": "small_airport",
                        "LBTG": "small_airport",
                        "LBWN": "large_airport",
                        "LCEN": "medium_airport",
                        "LCLK": "large_airport",
                        "LCP": "small_airport",
                        "LCPH": "large_airport",
                        "LCRA": "large_airport",
                        "LDDU": "medium_airport",
                        "LDLO": "small_airport",
                        "LDOS": "medium_airport",
                        "LDPL": "medium_airport",
                        "LDRI": "medium_airport",
                        "LDSB": "medium_airport",
                        "LDSP": "medium_airport",
                        "LDW": "small_airport",
                        "LDZA": "large_airport",
                        "LDZD": "medium_airport",
                        "LEAB": "medium_airport",
                        "LEAL": "large_airport",
                        "LEAM": "medium_airport",
                        "LEAS": "medium_airport",
                        "LEBA": "medium_airport",
                        "LEBB": "medium_airport",
                        "LEBG": "medium_airport",
                        "LEBL": "large_airport",
                        "LEBZ": "medium_airport",
                        "LECH": "medium_airport",
                        "LECO": "medium_airport",
                        "LECU": "medium_airport",
                        "LEDA": "medium_airport",
                        "LEGE": "medium_airport",
                        "LEGR": "medium_airport",
                        "LEHC": "small_airport",
                        "LEIB": "medium_airport",
                        "LEJR": "medium_airport",
                        "LELC": "medium_airport",
                        "LELL": "small_airport",
                        "LELN": "medium_airport",
                        "LELO": "small_airport",
                        "LEMD": "large_airport",
                        "LEMG": "large_airport",
                        "LEMH": "medium_airport",
                        "LEMO": "medium_airport",
                        "LEO": "small_airport",
                        "LEPA": "large_airport",
                        "LEPP": "medium_airport",
                        "LERL": "closed",
                        "LERS": "medium_airport",
                        "LERT": "medium_airport",
                        "LESA": "medium_airport",
                        "LESO": "medium_airport",
                        "LEST": "large_airport",
                        "LESU": "small_airport",
                        "LETL": "medium_airport",
                        "LETO": "medium_airport",
                        "LEVC": "medium_airport",
                        "LEVD": "medium_airport",
                        "LEVT": "medium_airport",
                        "LEVX": "medium_airport",
                        "LEXJ": "medium_airport",
                        "LEZG": "medium_airport",
                        "LEZL": "medium_airport",
                        "LFAB": "small_airport",
                        "LFAC": "medium_airport",
                        "LFAQ": "small_airport",
                        "LFAT": "medium_airport",
                        "LFAV": "medium_airport",
                        "LFBA": "medium_airport",
                        "LFBD": "large_airport",
                        "LFBE": "medium_airport",
                        "LFBG": "medium_airport",
                        "LFBH": "medium_airport",
                        "LFBI": "medium_airport",
                        "LFBK": "medium_airport",
                        "LFBL": "medium_airport",
                        "LFBN": "medium_airport",
                        "LFBO": "large_airport",
                        "LFBP": "medium_airport",
                        "LFBT": "medium_airport",
                        "LFBU": "medium_airport",
                        "LFBX": "medium_airport",
                        "LFBY": "small_airport",
                        "LFBZ": "medium_airport",
                        "LFCC": "medium_airport",
                        "LFCI": "medium_airport",
                        "LFCK": "medium_airport",
                        "LFCR": "medium_airport",
                        "LFCY": "medium_airport",
                        "LFDN": "medium_airport",
                        "LFEY": "medium_airport",
                        "LFGA": "medium_airport",
                        "LFGJ": "medium_airport",
                        "LFHM": "small_airport",
                        "LFHO": "medium_airport",
                        "LFHP": "medium_airport",
                        "LFHU": "small_airport",
                        "LFJL": "medium_airport",
                        "LFJR": "medium_airport",
                        "LFKB": "medium_airport",
                        "LFKC": "medium_airport",
                        "LFKF": "medium_airport",
                        "LFKJ": "medium_airport",
                        "LFKO": "small_airport",
                        "LFKS": "medium_airport",
                        "LFKX": "small_airport",
                        "LFLA": "medium_airport",
                        "LFLB": "medium_airport",
                        "LFLC": "medium_airport",
                        "LFLD": "medium_airport",
                        "LFLI": "medium_airport",
                        "LFLJ": "small_airport",
                        "LFLL": "large_airport",
                        "LFLN": "medium_airport",
                        "LFLO": "medium_airport",
                        "LFLP": "medium_airport",
                        "LFLS": "medium_airport",
                        "LFLU": "medium_airport",
                        "LFLV": "medium_airport",
                        "LFLW": "medium_airport",
                        "LFLX": "medium_airport",
                        "LFLY": "medium_airport",
                        "LFMD": "medium_airport",
                        "LFMH": "medium_airport",
                        "LFMK": "medium_airport",
                        "LFML": "large_airport",
                        "LFMN": "large_airport",
                        "LFMO": "medium_airport",
                        "LFMP": "medium_airport",
                        "LFMQ": "medium_airport",
                        "LFMR": "small_airport",
                        "LFMT": "medium_airport",
                        "LFMU": "medium_airport",
                        "LFMV": "medium_airport",
                        "LFNA": "small_airport",
                        "LFNB": "medium_airport",
                        "LFNC": "small_airport",
                        "LFOB": "medium_airport",
                        "LFOE": "medium_airport",
                        "LFOH": "medium_airport",
                        "LFOJ": "medium_airport",
                        "LFOK": "medium_airport",
                        "LFOO": "small_airport",
                        "LFOP": "medium_airport",
                        "LFOT": "medium_airport",
                        "LFOU": "medium_airport",
                        "LFOV": "medium_airport",
                        "LFPB": "medium_airport",
                        "LFPC": "medium_airport",
                        "LFPG": "large_airport",
                        "LFPI": "heliport",
                        "LFPN": "medium_airport",
                        "LFPO": "large_airport",
                        "LFPT": "medium_airport",
                        "LFPV": "medium_airport",
                        "LFQG": "medium_airport",
                        "LFQJ": "medium_airport",
                        "LFQQ": "medium_airport",
                        "LFQT": "medium_airport",
                        "LFQV": "medium_airport",
                        "LFRB": "medium_airport",
                        "LFRC": "medium_airport",
                        "LFRD": "medium_airport",
                        "LFRE": "medium_airport",
                        "LFRF": "medium_airport",
                        "LFRG": "medium_airport",
                        "LFRH": "medium_airport",
                        "LFRI": "medium_airport",
                        "LFRJ": "medium_airport",
                        "LFRK": "medium_airport",
                        "LFRM": "medium_airport",
                        "LFRN": "medium_airport",
                        "LFRO": "medium_airport",
                        "LFRQ": "medium_airport",
                        "LFRS": "medium_airport",
                        "LFRT": "medium_airport",
                        "LFRU": "medium_airport",
                        "LFRV": "medium_airport",
                        "LFRZ": "medium_airport",
                        "LFSB": "large_airport",
                        "LFSD": "medium_airport",
                        "LFSF": "closed",
                        "LFSG": "medium_airport",
                        "LFSL": "medium_airport",
                        "LFSN": "medium_airport",
                        "LFSQ": "closed",
                        "LFSR": "closed",
                        "LFST": "medium_airport",
                        "LFTH": "medium_airport",
                        "LFTU": "closed",
                        "LFTW": "medium_airport",
                        "LFTZ": "small_airport",
                        "LFVM": "medium_airport",
                        "LFVP": "medium_airport",
                        "LGAD": "medium_airport",
                        "LGAG": "closed",
                        "LGAL": "medium_airport",
                        "LGAT": "closed",
                        "LGAV": "large_airport",
                        "LGBL": "medium_airport",
                        "LGE": "small_airport",
                        "LGHI": "medium_airport",
                        "LGHL": "closed",
                        "LGIK": "small_airport",
                        "LGIO": "medium_airport",
                        "LGIR": "large_airport",
                        "LGKA": "small_airport",
                        "LGKC": "small_airport",
                        "LGKF": "medium_airport",
                        "LGKJ": "small_airport",
                        "LGKL": "medium_airport",
                        "LGKO": "medium_airport",
                        "LGKP": "medium_airport",
                        "LGKR": "medium_airport",
                        "LGKS": "small_airport",
                        "LGKV": "medium_airport",
                        "LGKY": "small_airport",
                        "LGKZ": "medium_airport",
                        "LGLE": "small_airport",
                        "LGLM": "small_airport",
                        "LGLR": "small_airport",
                        "LGM": "closed",
                        "LGMK": "medium_airport",
                        "LGML": "small_airport",
                        "LGMT": "medium_airport",
                        "LGN": "small_airport",
                        "LGNX": "small_airport",
                        "LGPA": "small_airport",
                        "LGPL": "small_airport",
                        "LGPZ": "medium_airport",
                        "LGRP": "medium_airport",
                        "LGRX": "medium_airport",
                        "LGSA": "medium_airport",
                        "LGSK": "medium_airport",
                        "LGSM": "medium_airport",
                        "LGSO": "small_airport",
                        "LGSP": "small_airport",
                        "LGSR": "medium_airport",
                        "LGST": "medium_airport",
                        "LGSY": "small_airport",
                        "LGTS": "large_airport",
                        "LGZA": "medium_airport",
                        "LHBP": "large_airport",
                        "LHDC": "medium_airport",
                        "LHMC": "small_airport",
                        "LHP": "closed",
                        "LHPP": "medium_airport",
                        "LHSM": "medium_airport",
                        "LHTA": "medium_airport",
                        "LIAP": "small_airport",
                        "LIBC": "medium_airport",
                        "LIBD": "large_airport",
                        "LIBF": "medium_airport",
                        "LIBG": "medium_airport",
                        "LIBN": "medium_airport",
                        "LIBP": "medium_airport",
                        "LIBR": "medium_airport",
                        "LICA": "medium_airport",
                        "LICB": "medium_airport",
                        "LICC": "large_airport",
                        "LICD": "medium_airport",
                        "LICG": "medium_airport",
                        "LICJ": "large_airport",
                        "LICR": "medium_airport",
                        "LICT": "medium_airport",
                        "LICZ": "medium_airport",
                        "LIDB": "small_airport",
                        "LIDI": "closed",
                        "LIDR": "small_airport",
                        "LIEA": "medium_airport",
                        "LIED": "medium_airport",
                        "LIEE": "large_airport",
                        "LIEO": "medium_airport",
                        "LIER": "small_airport",
                        "LIET": "medium_airport",
                        "LIMC": "large_airport",
                        "LIME": "large_airport",
                        "LIMF": "large_airport",
                        "LIMG": "medium_airport",
                        "LIMJ": "large_airport",
                        "LIML": "large_airport",
                        "LIMP": "medium_airport",
                        "LIMW": "medium_airport",
                        "LIMZ": "medium_airport",
                        "LIPA": "medium_airport",
                        "LIPB": "medium_airport",
                        "LIPD": "small_airport",
                        "LIPE": "large_airport",
                        "LIPH": "large_airport",
                        "LIPK": "medium_airport",
                        "LIPO": "medium_airport",
                        "LIPQ": "medium_airport",
                        "LIPR": "medium_airport",
                        "LIPT": "closed",
                        "LIPU": "medium_airport",
                        "LIPX": "large_airport",
                        "LIPY": "medium_airport",
                        "LIPZ": "large_airport",
                        "LIQL": "small_airport",
                        "LIQN": "small_airport",
                        "LIQS": "medium_airport",
                        "LIQW": "medium_airport",
                        "LIRA": "large_airport",
                        "LIRF": "large_airport",
                        "LIRI": "medium_airport",
                        "LIRJ": "medium_airport",
                        "LIRL": "medium_airport",
                        "LIRN": "large_airport",
                        "LIRP": "large_airport",
                        "LIRQ": "medium_airport",
                        "LIRS": "medium_airport",
                        "LIRZ": "medium_airport",
                        "LJLJ": "large_airport",
                        "LJMB": "medium_airport",
                        "LJPZ": "medium_airport",
                        "LKC": "small_airport",
                        "LKHO": "closed",
                        "LKKU": "medium_airport",
                        "LKKV": "medium_airport",
                        "LKMR": "small_airport",
                        "LKMT": "medium_airport",
                        "LKOL": "small_airport",
                        "LKPD": "medium_airport",
                        "LKPO": "medium_airport",
                        "LKPR": "large_airport",
                        "LKTB": "medium_airport",
                        "LKVO": "medium_airport",
                        "LKZA": "small_airport",
                        "LLBG": "large_airport",
                        "LLBS": "small_airport",
                        "LLER": "medium_airport",
                        "LLET": "closed",
                        "LLEY": "small_airport",
                        "LLH": "small_airport",
                        "LLHA": "medium_airport",
                        "LLIB": "medium_airport",
                        "LLKS": "small_airport",
                        "LLL": "small_airport",
                        "LLMZ": "medium_airport",
                        "LLNV": "medium_airport",
                        "LLOV": "large_airport",
                        "LLRM": "medium_airport",
                        "LLSD": "closed",
                        "LLYO": "small_airport",
                        "LMC": "small_airport",
                        "LMMG": "heliport",
                        "LMML": "large_airport",
                        "LMZ": "small_airport",
                        "LNC": "small_airport",
                        "LNF": "small_airport",
                        "LNM": "small_airport",
                        "LNMC": "heliport",
                        "LNQ": "closed",
                        "LOF": "closed",
                        "LOIH": "small_airport",
                        "LOM": "small_airport",
                        "LOR": "heliport",
                        "LOWG": "medium_airport",
                        "LOWI": "medium_airport",
                        "LOWK": "medium_airport",
                        "LOWL": "medium_airport",
                        "LOWS": "medium_airport",
                        "LOWW": "large_airport",
                        "LPAR": "medium_airport",
                        "LPAZ": "medium_airport",
                        "LPBG": "medium_airport",
                        "LPBJ": "medium_airport",
                        "LPBR": "medium_airport",
                        "LPCH": "small_airport",
                        "LPCR": "small_airport",
                        "LPCS": "medium_airport",
                        "LPFL": "medium_airport",
                        "LPFR": "large_airport",
                        "LPGR": "medium_airport",
                        "LPHR": "medium_airport",
                        "LPLA": "large_airport",
                        "LPMA": "medium_airport",
                        "LPMR": "medium_airport",
                        "LPPD": "large_airport",
                        "LPPI": "medium_airport",
                        "LPPM": "medium_airport",
                        "LPPR": "large_airport",
                        "LPPS": "medium_airport",
                        "LPPT": "large_airport",
                        "LPSI": "closed",
                        "LPSJ": "medium_airport",
                        "LPVR": "medium_airport",
                        "LPVZ": "medium_airport",
                        "LQBK": "medium_airport",
                        "LQMO": "medium_airport",
                        "LQSA": "large_airport",
                        "LQTZ": "medium_airport",
                        "LRAR": "medium_airport",
                        "LRBC": "medium_airport",
                        "LRBM": "medium_airport",
                        "LRBS": "medium_airport",
                        "LRCK": "medium_airport",
                        "LRCL": "medium_airport",
                        "LRCS": "medium_airport",
                        "LRCV": "medium_airport",
                        "LRIA": "medium_airport",
                        "LRO": "closed",
                        "LROD": "medium_airport",
                        "LROP": "large_airport",
                        "LRSB": "medium_airport",
                        "LRSM": "medium_airport",
                        "LRSV": "medium_airport",
                        "LRTC": "medium_airport",
                        "LRTM": "medium_airport",
                        "LRTR": "medium_airport",
                        "LSGG": "large_airport",
                        "LSGN": "small_airport",
                        "LSGS": "medium_airport",
                        "LSME": "medium_airport",
                        "LSMI": "closed",
                        "LSZA": "medium_airport",
                        "LSZB": "medium_airport",
                        "LSZC": "small_airport",
                        "LSZD": "closed",
                        "LSZG": "medium_airport",
                        "LSZH": "large_airport",
                        "LSZL": "small_airport",
                        "LSZR": "medium_airport",
                        "LSZS": "medium_airport",
                        "LT86": "small_airport",
                        "LTAC": "large_airport",
                        "LTAD": "medium_airport",
                        "LTAF": "large_airport",
                        "LTAG": "medium_airport",
                        "LTAH": "medium_airport",
                        "LTAI": "large_airport",
                        "LTAJ": "large_airport",
                        "LTAL": "small_airport",
                        "LTAN": "medium_airport",
                        "LTAP": "small_airport",
                        "LTAR": "small_airport",
                        "LTAS": "small_airport",
                        "LTAT": "medium_airport",
                        "LTAU": "medium_airport",
                        "LTAW": "medium_airport",
                        "LTAY": "medium_airport",
                        "LTAZ": "small_airport",
                        "LTB": "closed",
                        "LTBA": "large_airport",
                        "LTBD": "small_airport",
                        "LTBE": "small_airport",
                        "LTBF": "medium_airport",
                        "LTBG": "medium_airport",
                        "LTBH": "medium_airport",
                        "LTBI": "medium_airport",
                        "LTBJ": "large_airport",
                        "LTBL": "medium_airport",
                        "LTBO": "small_airport",
                        "LTBQ": "medium_airport",
                        "LTBR": "medium_airport",
                        "LTBS": "large_airport",
                        "LTBU": "medium_airport",
                        "LTBV": "small_airport",
                        "LTBY": "medium_airport",
                        "LTBZ": "small_airport",
                        "LTCA": "medium_airport",
                        "LTCC": "medium_airport",
                        "LTCD": "medium_airport",
                        "LTCE": "large_airport",
                        "LTCF": "medium_airport",
                        "LTCG": "large_airport",
                        "LTCH": "medium_airport",
                        "LTCI": "medium_airport",
                        "LTCJ": "medium_airport",
                        "LTCK": "medium_airport",
                        "LTCL": "medium_airport",
                        "LTCM": "small_airport",
                        "LTCN": "medium_airport",
                        "LTCO": "medium_airport",
                        "LTCP": "medium_airport",
                        "LTCR": "medium_airport",
                        "LTCS": "medium_airport",
                        "LTCV": "medium_airport",
                        "LTCW": "medium_airport",
                        "LTDA": "medium_airport",
                        "LTF": "small_airport",
                        "LTFC": "large_airport",
                        "LTFD": "medium_airport",
                        "LTFE": "large_airport",
                        "LTFH": "small_airport",
                        "LTFJ": "large_airport",
                        "LTFM": "large_airport",
                        "LTGP": "medium_airport",
                        "LTH": "closed",
                        "LUBL": "medium_airport",
                        "LUKK": "medium_airport",
                        "LUZ": "medium_airport",
                        "LVB": "closed",
                        "LVGZ": "closed",
                        "LWA": "small_airport",
                        "LWOH": "medium_airport",
                        "LWSK": "large_airport",
                        "LXGB": "medium_airport",
                        "LY-BCQ": "medium_airport",
                        "LY-DNF": "small_airport",
                        "LY-MRA": "small_airport",
                        "LY-QUB": "small_airport",
                        "LYBE": "large_airport",
                        "LYBR": "small_airport",
                        "LYBT": "medium_airport",
                        "LYKV": "small_airport",
                        "LYNI": "medium_airport",
                        "LYNS": "small_airport",
                        "LYPG": "large_airport",
                        "LYTV": "medium_airport",
                        "LZIB": "large_airport",
                        "LZKZ": "medium_airport",
                        "LZLU": "small_airport",
                        "LZPP": "medium_airport",
                        "LZPW": "small_airport",
                        "LZSL": "medium_airport",
                        "LZTT": "medium_airport",
                        "LZZI": "medium_airport",
                        "M26": "small_airport",
                        "MA-GLN": "small_airport",
                        "MA53": "small_airport",
                        "MAP": "small_airport",
                        "MBGT": "medium_airport",
                        "MBMC": "small_airport",
                        "MBNC": "medium_airport",
                        "MBPI": "small_airport",
                        "MBPV": "medium_airport",
                        "MBSC": "medium_airport",
                        "MBSY": "small_airport",
                        "MDBH": "medium_airport",
                        "MDCR": "medium_airport",
                        "MDCY": "medium_airport",
                        "MDCZ": "small_airport",
                        "MDHE": "closed",
                        "MDJB": "medium_airport",
                        "MDLR": "medium_airport",
                        "MDPC": "large_airport",
                        "MDPO": "small_airport",
                        "MDPP": "medium_airport",
                        "MDR": "small_airport",
                        "MDSD": "large_airport",
                        "MDSJ": "closed",
                        "MDST": "medium_airport",
                        "MDV": "small_airport",
                        "ME16": "small_airport",
                        "MEF": "small_airport",
                        "MG-0001": "small_airport",
                        "MG-DOA": "small_airport",
                        "MGCB": "medium_airport",
                        "MGCR": "small_airport",
                        "MGCT": "small_airport",
                        "MGG": "closed",
                        "MGGT": "large_airport",
                        "MGHT": "small_airport",
                        "MGI": "closed",
                        "MGMM": "small_airport",
                        "MGO": "closed",
                        "MGP": "small_airport",
                        "MGPB": "medium_airport",
                        "MGPP": "small_airport",
                        "MGQC": "small_airport",
                        "MGQZ": "small_airport",
                        "MGRB": "medium_airport",
                        "MGRD": "small_airport",
                        "MGRT": "medium_airport",
                        "MGSJ": "medium_airport",
                        "MGTK": "medium_airport",
                        "MH-AIM": "small_airport",
                        "MH-AUL": "small_airport",
                        "MH-BII": "small_airport",
                        "MH-EBN": "small_airport",
                        "MH-JAT": "small_airport",
                        "MH-JEJ": "small_airport",
                        "MH-KBT": "small_airport",
                        "MH-LIK": "small_airport",
                        "MH-LML": "small_airport",
                        "MH-MAV": "small_airport",
                        "MH-MJB": "small_airport",
                        "MH-MJE": "small_airport",
                        "MH-NDK": "small_airport",
                        "MH-RNP": "small_airport",
                        "MH-TIC": "small_airport",
                        "MH-UIT": "small_airport",
                        "MH-WJA": "small_airport",
                        "MH-WTE": "small_airport",
                        "MH-WTO": "small_airport",
                        "MHAH": "small_airport",
                        "MHBL": "small_airport",
                        "MHCA": "small_airport",
                        "MHCL": "closed",
                        "MHCR": "small_airport",
                        "MHCS": "small_airport",
                        "MHCU": "small_airport",
                        "MHEA": "small_airport",
                        "MHGS": "small_airport",
                        "MHIR": "small_airport",
                        "MHJI": "small_airport",
                        "MHJU": "small_airport",
                        "MHLC": "medium_airport",
                        "MHLE": "small_airport",
                        "MHLM": "medium_airport",
                        "MHN": "small_airport",
                        "MHNJ": "medium_airport",
                        "MHPC": "small_airport",
                        "MHPL": "small_airport",
                        "MHRO": "medium_airport",
                        "MHRS": "closed",
                        "MHRU": "small_airport",
                        "MHSC": "medium_airport",
                        "MHTE": "medium_airport",
                        "MHTG": "medium_airport",
                        "MHTJ": "medium_airport",
                        "MHUL": "small_airport",
                        "MHUT": "small_airport",
                        "MHY": "small_airport",
                        "MHYR": "small_airport",
                        "MIZ": "small_airport",
                        "MJJ": "small_airport",
                        "MJS": "small_airport",
                        "MKBS": "medium_airport",
                        "MKJP": "large_airport",
                        "MKJS": "medium_airport",
                        "MKKJ": "medium_airport",
                        "MKN": "small_airport",
                        "MKNG": "small_airport",
                        "MKTP": "medium_airport",
                        "MLIP": "small_airport",
                        "MLQ": "small_airport",
                        "MM-0002": "small_airport",
                        "MM72": "small_airport",
                        "MMAA": "large_airport",
                        "MMAN": "medium_airport",
                        "MMAS": "medium_airport",
                        "MMBT": "medium_airport",
                        "MMCA": "small_airport",
                        "MMCB": "medium_airport",
                        "MMCC": "small_airport",
                        "MMCE": "medium_airport",
                        "MMCG": "small_airport",
                        "MMCL": "medium_airport",
                        "MMCM": "medium_airport",
                        "MMCN": "medium_airport",
                        "MMCO": "small_airport",
                        "MMCP": "medium_airport",
                        "MMCS": "medium_airport",
                        "MMCT": "medium_airport",
                        "MMCU": "medium_airport",
                        "MMCV": "medium_airport",
                        "MMCY": "medium_airport",
                        "MMCZ": "medium_airport",
                        "MMDA": "medium_airport",
                        "MMDM": "small_airport",
                        "MMDO": "medium_airport",
                        "MMEP": "medium_airport",
                        "MMES": "small_airport",
                        "MMGL": "large_airport",
                        "MMGM": "medium_airport",
                        "MMGR": "small_airport",
                        "MMHC": "small_airport",
                        "MMHO": "large_airport",
                        "MMIA": "medium_airport",
                        "MMIM": "small_airport",
                        "MMIO": "medium_airport",
                        "MMIT": "medium_airport",
                        "MMJA": "medium_airport",
                        "MMJC": "small_airport",
                        "MMLC": "medium_airport",
                        "MMLM": "medium_airport",
                        "MMLO": "medium_airport",
                        "MMLP": "medium_airport",
                        "MMLT": "medium_airport",
                        "MMMA": "medium_airport",
                        "MMMD": "medium_airport",
                        "MMMG": "small_airport",
                        "MMML": "medium_airport",
                        "MMMM": "medium_airport",
                        "MMMT": "medium_airport",
                        "MMMV": "medium_airport",
                        "MMMX": "large_airport",
                        "MMMY": "large_airport",
                        "MMMZ": "medium_airport",
                        "MMNG": "small_airport",
                        "MMNL": "medium_airport",
                        "MMOX": "medium_airport",
                        "MMPA": "medium_airport",
                        "MMPB": "medium_airport",
                        "MMPG": "medium_airport",
                        "MMPL": "small_airport",
                        "MMPN": "medium_airport",
                        "MMPQ": "small_airport",
                        "MMPR": "large_airport",
                        "MMPS": "medium_airport",
                        "MMQT": "medium_airport",
                        "MMRX": "medium_airport",
                        "MMSC": "closed",
                        "MMSD": "large_airport",
                        "MMSF": "small_airport",
                        "MMSM": "small_airport",
                        "MMSP": "medium_airport",
                        "MMTC": "medium_airport",
                        "MMTG": "medium_airport",
                        "MMTJ": "large_airport",
                        "MMTM": "medium_airport",
                        "MMTN": "small_airport",
                        "MMTO": "medium_airport",
                        "MMTP": "medium_airport",
                        "MMUN": "large_airport",
                        "MMV": "small_airport",
                        "MMVA": "medium_airport",
                        "MMVR": "medium_airport",
                        "MMZC": "medium_airport",
                        "MMZH": "medium_airport",
                        "MMZM": "small_airport",
                        "MMZO": "medium_airport",
                        "MN-MXW": "small_airport",
                        "MNBL": "medium_airport",
                        "MNBZ": "small_airport",
                        "MNCE": "small_airport",
                        "MNCI": "small_airport",
                        "MNMG": "medium_airport",
                        "MNNG": "small_airport",
                        "MNP": "closed",
                        "MNPC": "medium_airport",
                        "MNRT": "small_airport",
                        "MNSC": "small_airport",
                        "MNSI": "small_airport",
                        "MNWP": "small_airport",
                        "MOH": "small_airport",
                        "MP00": "small_airport",
                        "MPBO": "medium_airport",
                        "MPCE": "medium_airport",
                        "MPCH": "medium_airport",
                        "MPDA": "medium_airport",
                        "MPEJ": "medium_airport",
                        "MPG": "small_airport",
                        "MPHO": "small_airport",
                        "MPI": "small_airport",
                        "MPJE": "small_airport",
                        "MPLP": "closed",
                        "MPMG": "medium_airport",
                        "MPOA": "small_airport",
                        "MPRH": "small_airport",
                        "MPSA": "medium_airport",
                        "MPTO": "large_airport",
                        "MPU": "small_airport",
                        "MPVR": "small_airport",
                        "MPWN": "small_airport",
                        "MPX": "small_airport",
                        "MQO": "small_airport",
                        "MRAN": "small_airport",
                        "MRAO": "medium_airport",
                        "MRBA": "medium_airport",
                        "MRBC": "medium_airport",
                        "MRCC": "medium_airport",
                        "MRCH": "small_airport",
                        "MRCR": "small_airport",
                        "MRDK": "small_airport",
                        "MRFL": "small_airport",
                        "MRGF": "medium_airport",
                        "MRGP": "medium_airport",
                        "MRIA": "small_airport",
                        "MRLB": "large_airport",
                        "MRLC": "medium_airport",
                        "MRLM": "medium_airport",
                        "MRMJ": "small_airport",
                        "MRNC": "small_airport",
                        "MRNS": "medium_airport",
                        "MROC": "medium_airport",
                        "MRPJ": "medium_airport",
                        "MRPM": "medium_airport",
                        "MRPV": "medium_airport",
                        "MRQP": "medium_airport",
                        "MRRF": "small_airport",
                        "MRSI": "small_airport",
                        "MRSV": "medium_airport",
                        "MRT": "small_airport",
                        "MRTM": "small_airport",
                        "MRTR": "small_airport",
                        "MRUP": "medium_airport",
                        "MSB": "seaplane_base",
                        "MSLP": "large_airport",
                        "MTCA": "medium_airport",
                        "MTCH": "medium_airport",
                        "MTF": "small_airport",
                        "MTJA": "medium_airport",
                        "MTJE": "medium_airport",
                        "MTPP": "medium_airport",
                        "MTPX": "medium_airport",
                        "MTU": "small_airport",
                        "MUBA": "medium_airport",
                        "MUBR": "small_airport",
                        "MUBY": "medium_airport",
                        "MUCA": "medium_airport",
                        "MUCC": "medium_airport",
                        "MUCF": "medium_airport",
                        "MUCL": "medium_airport",
                        "MUCM": "medium_airport",
                        "MUCO": "small_airport",
                        "MUCU": "medium_airport",
                        "MUGM": "medium_airport",
                        "MUGT": "medium_airport",
                        "MUHA": "large_airport",
                        "MUHG": "medium_airport",
                        "MUKW": "medium_airport",
                        "MULM": "medium_airport",
                        "MUMA": "small_airport",
                        "MUMJ": "small_airport",
                        "MUMO": "medium_airport",
                        "MUMZ": "medium_airport",
                        "MUNB": "medium_airport",
                        "MUNC": "small_airport",
                        "MUNG": "medium_airport",
                        "MUPB": "medium_airport",
                        "MUPR": "medium_airport",
                        "MUSC": "medium_airport",
                        "MUSJ": "medium_airport",
                        "MUSN": "medium_airport",
                        "MUSS": "small_airport",
                        "MUTD": "medium_airport",
                        "MUVR": "large_airport",
                        "MUVT": "medium_airport",
                        "MVI": "closed",
                        "MWCB": "medium_airport",
                        "MWCL": "medium_airport",
                        "MWCR": "large_airport",
                        "MWR": "small_airport",
                        "MX-AJS": "small_airport",
                        "MX-AZG": "small_airport",
                        "MX-PCM": "small_airport",
                        "MX-PCV": "small_airport",
                        "MX-SCX": "small_airport",
                        "MX-SGM": "small_airport",
                        "MX-TUY": "small_airport",
                        "MX-UAC": "small_airport",
                        "MX-XAL": "small_airport",
                        "MXC": "small_airport",
                        "MXK": "small_airport",
                        "MXR": "small_airport",
                        "MY-GTB": "closed",
                        "MY-GTK": "small_airport",
                        "MY-LBP": "small_airport",
                        "MY-LLM": "small_airport",
                        "MY-MZS": "small_airport",
                        "MY-SPT": "small_airport",
                        "MYAB": "medium_airport",
                        "MYAF": "medium_airport",
                        "MYAK": "medium_airport",
                        "MYAM": "medium_airport",
                        "MYAN": "medium_airport",
                        "MYAP": "medium_airport",
                        "MYAT": "medium_airport",
                        "MYAW": "small_airport",
                        "MYBC": "medium_airport",
                        "MYBG": "medium_airport",
                        "MYBS": "medium_airport",
                        "MYCA": "medium_airport",
                        "MYCB": "medium_airport",
                        "MYCC": "small_airport",
                        "MYCI": "medium_airport",
                        "MYCP": "small_airport",
                        "MYEF": "medium_airport",
                        "MYEH": "medium_airport",
                        "MYEM": "medium_airport",
                        "MYEN": "medium_airport",
                        "MYER": "medium_airport",
                        "MYES": "medium_airport",
                        "MYGF": "medium_airport",
                        "MYGM": "small_airport",
                        "MYGW": "small_airport",
                        "MYIG": "medium_airport",
                        "MYK": "small_airport",
                        "MYLD": "medium_airport",
                        "MYLS": "medium_airport",
                        "MYMM": "medium_airport",
                        "MYNN": "large_airport",
                        "MYPI": "closed",
                        "MYRD": "medium_airport",
                        "MYRP": "small_airport",
                        "MYS": "small_airport",
                        "MYSM": "medium_airport",
                        "MYX": "small_airport",
                        "MYZ2": "closed",
                        "MZ-0004": "closed",
                        "MZ-0005": "small_airport",
                        "MZ-0012": "small_airport",
                        "MZ-TGS": "small_airport",
                        "MZBZ": "large_airport",
                        "MZE": "small_airport",
                        "MZMF": "small_airport",
                        "N04": "closed",
                        "N20": "small_airport",
                        "NA-0012": "small_airport",
                        "NBS": "medium_airport",
                        "NCAI": "small_airport",
                        "NCAT": "small_airport",
                        "NCMG": "small_airport",
                        "NCMH": "small_airport",
                        "NCMK": "small_airport",
                        "NCMR": "small_airport",
                        "NCPK": "small_airport",
                        "NCPY": "small_airport",
                        "NCRG": "large_airport",
                        "NDI": "small_airport",
                        "NDN": "small_airport",
                        "NE69": "small_airport",
                        "NFBG": "closed",
                        "NFCI": "small_airport",
                        "NFCS": "seaplane_base",
                        "NFFN": "medium_airport",
                        "NFFO": "small_airport",
                        "NFFR": "small_airport",
                        "NFKD": "small_airport",
                        "NFMA": "small_airport",
                        "NFMO": "small_airport",
                        "NFNA": "medium_airport",
                        "NFNB": "small_airport",
                        "NFNG": "small_airport",
                        "NFNH": "small_airport",
                        "NFNK": "small_airport",
                        "NFNL": "medium_airport",
                        "NFNM": "small_airport",
                        "NFNO": "small_airport",
                        "NFNR": "small_airport",
                        "NFNS": "small_airport",
                        "NFNU": "closed",
                        "NFNV": "small_airport",
                        "NFNW": "small_airport",
                        "NFOL": "small_airport",
                        "NFSW": "small_airport",
                        "NFTE": "small_airport",
                        "NFTF": "medium_airport",
                        "NFTL": "medium_airport",
                        "NFTO": "small_airport",
                        "NFTP": "small_airport",
                        "NFTV": "medium_airport",
                        "NFUL": "seaplane_base",
                        "NFVB": "small_airport",
                        "NFVL": "small_airport",
                        "NG-0003": "medium_airport",
                        "NG-0004": "small_airport",
                        "NG-QRW": "medium_airport",
                        "NGAB": "small_airport",
                        "NGBR": "small_airport",
                        "NGFU": "medium_airport",
                        "NGKT": "small_airport",
                        "NGMA": "small_airport",
                        "NGMK": "small_airport",
                        "NGMN": "small_airport",
                        "NGNU": "small_airport",
                        "NGON": "small_airport",
                        "NGTA": "medium_airport",
                        "NGTB": "small_airport",
                        "NGTE": "medium_airport",
                        "NGTM": "small_airport",
                        "NGTO": "small_airport",
                        "NGTR": "small_airport",
                        "NGTS": "small_airport",
                        "NGTU": "small_airport",
                        "NGUK": "small_airport",
                        "NIK": "small_airport",
                        "NIUE": "medium_airport",
                        "NKD": "small_airport",
                        "NLH": "medium_airport",
                        "NLWF": "medium_airport",
                        "NLWW": "medium_airport",
                        "NM83": "small_airport",
                        "NND": "small_airport",
                        "NOM": "small_airport",
                        "NOO": "small_airport",
                        "NP-MWP": "small_airport",
                        "NPG": "small_airport",
                        "NRC": "closed",
                        "NRY": "small_airport",
                        "NSAS": "small_airport",
                        "NSAU": "small_airport",
                        "NSFA": "medium_airport",
                        "NSFI": "closed",
                        "NSFQ": "medium_airport",
                        "NSMA": "small_airport",
                        "NSTU": "medium_airport",
                        "NTA": "closed",
                        "NTAA": "large_airport",
                        "NTAM": "small_airport",
                        "NTAR": "medium_airport",
                        "NTAT": "medium_airport",
                        "NTAV": "small_airport",
                        "NTGA": "medium_airport",
                        "NTGB": "medium_airport",
                        "NTGC": "medium_airport",
                        "NTGD": "small_airport",
                        "NTGE": "medium_airport",
                        "NTGF": "medium_airport",
                        "NTGH": "small_airport",
                        "NTGI": "medium_airport",
                        "NTGJ": "medium_airport",
                        "NTGK": "medium_airport",
                        "NTGM": "medium_airport",
                        "NTGN": "small_airport",
                        "NTGO": "small_airport",
                        "NTGP": "medium_airport",
                        "NTGQ": "small_airport",
                        "NTGT": "medium_airport",
                        "NTGU": "medium_airport",
                        "NTGV": "medium_airport",
                        "NTGW": "small_airport",
                        "NTGY": "small_airport",
                        "NTHE": "medium_airport",
                        "NTKA": "small_airport",
                        "NTKF": "small_airport",
                        "NTKH": "small_airport",
                        "NTKK": "small_airport",
                        "NTKM": "small_airport",
                        "NTKN": "small_airport",
                        "NTKO": "small_airport",
                        "NTKR": "medium_airport",
                        "NTKT": "small_airport",
                        "NTKU": "small_airport",
                        "NTMD": "medium_airport",
                        "NTMN": "medium_airport",
                        "NTMP": "small_airport",
                        "NTMU": "small_airport",
                        "NTTB": "medium_airport",
                        "NTTE": "small_airport",
                        "NTTG": "medium_airport",
                        "NTTH": "medium_airport",
                        "NTTM": "medium_airport",
                        "NTTO": "medium_airport",
                        "NTTP": "medium_airport",
                        "NTTR": "medium_airport",
                        "NTTU": "small_airport",
                        "NTTX": "small_airport",
                        "NTUV": "small_airport",
                        "NUF": "seaplane_base",
                        "NUG": "small_airport",
                        "NV11": "small_airport",
                        "NVSA": "small_airport",
                        "NVSC": "small_airport",
                        "NVSD": "small_airport",
                        "NVSE": "small_airport",
                        "NVSF": "small_airport",
                        "NVSG": "small_airport",
                        "NVSH": "small_airport",
                        "NVSI": "small_airport",
                        "NVSL": "small_airport",
                        "NVSM": "small_airport",
                        "NVSN": "small_airport",
                        "NVSO": "small_airport",
                        "NVSP": "small_airport",
                        "NVSQ": "medium_airport",
                        "NVSR": "small_airport",
                        "NVSS": "medium_airport",
                        "NVST": "small_airport",
                        "NVSU": "small_airport",
                        "NVSV": "small_airport",
                        "NVSW": "small_airport",
                        "NVSX": "small_airport",
                        "NVSZ": "small_airport",
                        "NVVA": "small_airport",
                        "NVVB": "small_airport",
                        "NVVD": "small_airport",
                        "NVVF": "small_airport",
                        "NVVI": "small_airport",
                        "NVVQ": "small_airport",
                        "NVVV": "medium_airport",
                        "NVVW": "medium_airport",
                        "NWT": "small_airport",
                        "NWWA": "medium_airport",
                        "NWWC": "medium_airport",
                        "NWWD": "medium_airport",
                        "NWWE": "medium_airport",
                        "NWWH": "small_airport",
                        "NWWK": "medium_airport",
                        "NWWL": "medium_airport",
                        "NWWM": "medium_airport",
                        "NWWO": "small_airport",
                        "NWWP": "small_airport",
                        "NWWQ": "small_airport",
                        "NWWR": "medium_airport",
                        "NWWU": "medium_airport",
                        "NWWV": "medium_airport",
                        "NWWW": "medium_airport",
                        "NZAA": "large_airport",
                        "NZAP": "medium_airport",
                        "NZAR": "medium_airport",
                        "NZAS": "small_airport",
                        "NZCH": "large_airport",
                        "NZCI": "medium_airport",
                        "NZCX": "small_airport",
                        "NZDA": "small_airport",
                        "NZDN": "medium_airport",
                        "NZFJ": "small_airport",
                        "NZGB": "small_airport",
                        "NZGM": "small_airport",
                        "NZGS": "medium_airport",
                        "NZGT": "medium_airport",
                        "NZHK": "medium_airport",
                        "NZHN": "medium_airport",
                        "NZKE": "small_airport",
                        "NZKI": "small_airport",
                        "NZKK": "medium_airport",
                        "NZKO": "small_airport",
                        "NZKT": "medium_airport",
                        "NZLX": "medium_airport",
                        "NZMA": "small_airport",
                        "NZMB": "heliport",
                        "NZMC": "medium_airport",
                        "NZMF": "small_airport",
                        "NZMK": "small_airport",
                        "NZMO": "medium_airport",
                        "NZMS": "medium_airport",
                        "NZNP": "medium_airport",
                        "NZNR": "medium_airport",
                        "NZNS": "medium_airport",
                        "NZNV": "medium_airport",
                        "NZOH": "medium_airport",
                        "NZOU": "medium_airport",
                        "NZPM": "medium_airport",
                        "NZPN": "small_airport",
                        "NZPP": "medium_airport",
                        "NZQN": "medium_airport",
                        "NZRA": "small_airport",
                        "NZRC": "small_airport",
                        "NZRO": "medium_airport",
                        "NZTG": "medium_airport",
                        "NZTH": "small_airport",
                        "NZTK": "small_airport",
                        "NZTO": "small_airport",
                        "NZTS": "small_airport",
                        "NZTU": "medium_airport",
                        "NZUK": "medium_airport",
                        "NZWB": "medium_airport",
                        "NZWF": "medium_airport",
                        "NZWK": "medium_airport",
                        "NZWN": "large_airport",
                        "NZWO": "medium_airport",
                        "NZWR": "medium_airport",
                        "NZWS": "medium_airport",
                        "NZWT": "small_airport",
                        "NZWU": "medium_airport",
                        "O19": "small_airport",
                        "O85": "small_airport",
                        "OAA": "small_airport",
                        "OABN": "small_airport",
                        "OABT": "small_airport",
                        "OACC": "small_airport",
                        "OADS": "small_airport",
                        "OADZ": "small_airport",
                        "OAFR": "small_airport",
                        "OAFZ": "small_airport",
                        "OAGN": "small_airport",
                        "OAHN": "small_airport",
                        "OAHR": "medium_airport",
                        "OAIX": "medium_airport",
                        "OAJL": "medium_airport",
                        "OAKA": "small_airport",
                        "OAKB": "medium_airport",
                        "OAKN": "medium_airport",
                        "OAKS": "small_airport",
                        "OAMN": "medium_airport",
                        "OAMS": "medium_airport",
                        "OAOG": "small_airport",
                        "OAQN": "small_airport",
                        "OASA": "small_airport",
                        "OASD": "small_airport",
                        "OASN": "small_airport",
                        "OATN": "small_airport",
                        "OATQ": "small_airport",
                        "OAUZ": "medium_airport",
                        "OAZI": "small_airport",
                        "OAZJ": "small_airport",
                        "OBBI": "large_airport",
                        "OBK": "closed",
                        "OCS": "small_airport",
                        "OEAB": "medium_airport",
                        "OEAH": "medium_airport",
                        "OEBA": "medium_airport",
                        "OEBH": "medium_airport",
                        "OEDF": "large_airport",
                        "OEDM": "medium_airport",
                        "OEDR": "large_airport",
                        "OEGN": "medium_airport",
                        "OEGS": "medium_airport",
                        "OEGT": "medium_airport",
                        "OEHL": "medium_airport",
                        "OEJB": "medium_airport",
                        "OEJN": "large_airport",
                        "OEKK": "medium_airport",
                        "OEKM": "medium_airport",
                        "OEMA": "large_airport",
                        "OENG": "medium_airport",
                        "OENN": "medium_airport",
                        "OEPA": "medium_airport",
                        "OEPS": "medium_airport",
                        "OERF": "medium_airport",
                        "OERK": "large_airport",
                        "OERR": "medium_airport",
                        "OERY": "medium_airport",
                        "OESH": "medium_airport",
                        "OESK": "medium_airport",
                        "OESL": "small_airport",
                        "OETB": "medium_airport",
                        "OETF": "medium_airport",
                        "OETR": "medium_airport",
                        "OEWD": "medium_airport",
                        "OEWJ": "medium_airport",
                        "OEYN": "medium_airport",
                        "OEZL": "small_airport",
                        "OGE": "small_airport",
                        "OGM": "small_airport",
                        "OI20": "small_airport",
                        "OIAA": "medium_airport",
                        "OIAD": "small_airport",
                        "OIAG": "medium_airport",
                        "OIAH": "medium_airport",
                        "OIAJ": "small_airport",
                        "OIAM": "medium_airport",
                        "OIAW": "medium_airport",
                        "OIBA": "medium_airport",
                        "OIBB": "medium_airport",
                        "OIBI": "closed",
                        "OIBJ": "medium_airport",
                        "OIBK": "medium_airport",
                        "OIBL": "medium_airport",
                        "OIBP": "medium_airport",
                        "OIBQ": "medium_airport",
                        "OIBS": "medium_airport",
                        "OIBV": "medium_airport",
                        "OICC": "medium_airport",
                        "OICI": "medium_airport",
                        "OICK": "medium_airport",
                        "OICS": "medium_airport",
                        "OIFE": "small_airport",
                        "OIFK": "medium_airport",
                        "OIFM": "medium_airport",
                        "OIFS": "medium_airport",
                        "OIGG": "medium_airport",
                        "OIHH": "medium_airport",
                        "OIHR": "small_airport",
                        "OIIE": "large_airport",
                        "OIII": "large_airport",
                        "OIIK": "medium_airport",
                        "OIIP": "medium_airport",
                        "OIIS": "small_airport",
                        "OIKB": "medium_airport",
                        "OIKJ": "medium_airport",
                        "OIKK": "medium_airport",
                        "OIKM": "medium_airport",
                        "OIKP": "small_airport",
                        "OIKR": "medium_airport",
                        "OIKY": "medium_airport",
                        "OIMB": "medium_airport",
                        "OIMC": "medium_airport",
                        "OIMJ": "small_airport",
                        "OIMM": "large_airport",
                        "OIMN": "medium_airport",
                        "OIMS": "medium_airport",
                        "OIMT": "medium_airport",
                        "OINE": "small_airport",
                        "OING": "medium_airport",
                        "OINJ": "small_airport",
                        "OINN": "medium_airport",
                        "OINR": "medium_airport",
                        "OINZ": "medium_airport",
                        "OISF": "medium_airport",
                        "OISJ": "small_airport",
                        "OISL": "medium_airport",
                        "OISR": "small_airport",
                        "OISS": "large_airport",
                        "OISY": "medium_airport",
                        "OITK": "small_airport",
                        "OITL": "medium_airport",
                        "OITM": "small_airport",
                        "OITP": "small_airport",
                        "OITR": "medium_airport",
                        "OITT": "large_airport",
                        "OITU": "small_airport",
                        "OITZ": "medium_airport",
                        "OIYY": "medium_airport",
                        "OIZB": "medium_airport",
                        "OIZC": "medium_airport",
                        "OIZH": "medium_airport",
                        "OIZI": "medium_airport",
                        "OIZJ": "small_airport",
                        "OJAI": "large_airport",
                        "OJAM": "medium_airport",
                        "OJAQ": "medium_airport",
                        "OJJR": "closed",
                        "OJMF": "small_airport",
                        "OK03": "closed",
                        "OKAJ": "medium_airport",
                        "OKBK": "large_airport",
                        "OKV": "small_airport",
                        "OLBA": "large_airport",
                        "OLKA": "medium_airport",
                        "OLQ": "small_airport",
                        "OM-0001": "small_airport",
                        "OM-0002": "small_airport",
                        "OM-0003": "large_airport",
                        "OM-0004": "large_airport",
                        "OMAA": "large_airport",
                        "OMAD": "medium_airport",
                        "OMAL": "medium_airport",
                        "OMAM": "medium_airport",
                        "OMBY": "medium_airport",
                        "OMDB": "large_airport",
                        "OMDM": "medium_airport",
                        "OMDW": "large_airport",
                        "OMFJ": "medium_airport",
                        "OML": "closed",
                        "OMN": "small_airport",
                        "OMRK": "medium_airport",
                        "OMSJ": "large_airport",
                        "OMY": "small_airport",
                        "ONB": "small_airport",
                        "OOBR": "small_airport",
                        "OOFD": "small_airport",
                        "OOGB": "small_airport",
                        "OOJA": "small_airport",
                        "OOKB": "medium_airport",
                        "OOLK": "small_airport",
                        "OOMA": "medium_airport",
                        "OOMS": "large_airport",
                        "OOMX": "small_airport",
                        "OOSA": "medium_airport",
                        "OOSH": "small_airport",
                        "OOSR": "small_airport",
                        "OOTH": "medium_airport",
                        "OP17": "small_airport",
                        "OPBG": "small_airport",
                        "OPBN": "small_airport",
                        "OPBR": "small_airport",
                        "OPBW": "medium_airport",
                        "OPCH": "medium_airport",
                        "OPCL": "small_airport",
                        "OPDB": "small_airport",
                        "OPDD": "small_airport",
                        "OPDG": "medium_airport",
                        "OPDI": "medium_airport",
                        "OPFA": "medium_airport",
                        "OPGD": "medium_airport",
                        "OPGT": "medium_airport",
                        "OPIS": "large_airport",
                        "OPJA": "medium_airport",
                        "OPJI": "small_airport",
                        "OPKC": "medium_airport",
                        "OPKD": "small_airport",
                        "OPKH": "small_airport",
                        "OPKL": "small_airport",
                        "OPKT": "small_airport",
                        "OPLA": "medium_airport",
                        "OPLL": "small_airport",
                        "OPMA": "medium_airport",
                        "OPMF": "medium_airport",
                        "OPMI": "medium_airport",
                        "OPMJ": "medium_airport",
                        "OPMP": "small_airport",
                        "OPMS": "medium_airport",
                        "OPMT": "medium_airport",
                        "OPNH": "medium_airport",
                        "OPNK": "small_airport",
                        "OPOR": "small_airport",
                        "OPPC": "small_airport",
                        "OPPG": "medium_airport",
                        "OPPI": "medium_airport",
                        "OPPS": "medium_airport",
                        "OPQT": "medium_airport",
                        "OPRK": "medium_airport",
                        "OPRT": "medium_airport",
                        "OPSB": "small_airport",
                        "OPSD": "medium_airport",
                        "OPSK": "medium_airport",
                        "OPSN": "medium_airport",
                        "OPSR": "medium_airport",
                        "OPSS": "medium_airport",
                        "OPST": "large_airport",
                        "OPSU": "medium_airport",
                        "OPSW": "small_airport",
                        "OPTA": "small_airport",
                        "OPTH": "small_airport",
                        "OPTT": "small_airport",
                        "OPTU": "medium_airport",
                        "OPWN": "small_airport",
                        "OPZB": "medium_airport",
                        "ORAA": "medium_airport",
                        "ORAT": "medium_airport",
                        "ORBB": "small_airport",
                        "ORBI": "large_airport",
                        "ORBM": "medium_airport",
                        "ORER": "medium_airport",
                        "ORKK": "medium_airport",
                        "ORMM": "large_airport",
                        "ORNI": "small_airport",
                        "ORQW": "medium_airport",
                        "ORSU": "medium_airport",
                        "ORTL": "medium_airport",
                        "OSAP": "large_airport",
                        "OSDI": "large_airport",
                        "OSDZ": "medium_airport",
                        "OSE": "small_airport",
                        "OSG": "small_airport",
                        "OSKL": "medium_airport",
                        "OSLK": "large_airport",
                        "OSPR": "medium_airport",
                        "OTBD": "medium_airport",
                        "OTBH": "medium_airport",
                        "OTHH": "large_airport",
                        "OTT": "small_airport",
                        "OTY": "small_airport",
                        "OUM": "small_airport",
                        "OXO": "small_airport",
                        "OYAA": "medium_airport",
                        "OYAB": "small_airport",
                        "OYAT": "medium_airport",
                        "OYBI": "small_airport",
                        "OYBN": "medium_airport",
                        "OYBQ": "small_airport",
                        "OYGD": "medium_airport",
                        "OYHD": "medium_airport",
                        "OYKM": "small_airport",
                        "OYMB": "small_airport",
                        "OYMS": "small_airport",
                        "OYQN": "small_airport",
                        "OYRN": "medium_airport",
                        "OYSH": "small_airport",
                        "OYSN": "medium_airport",
                        "OYSQ": "medium_airport",
                        "OYSY": "medium_airport",
                        "OYTZ": "medium_airport",
                        "PA-0001": "small_airport",
                        "PA-0003": "small_airport",
                        "PA-0004": "small_airport",
                        "PA-0011": "small_airport",
                        "PA-0013": "small_airport",
                        "PA-0017": "small_airport",
                        "PA-0022": "small_airport",
                        "PA-AML": "closed",
                        "PA-BFQ": "small_airport",
                        "PA-ELE": "small_airport",
                        "PA-OTD": "small_airport",
                        "PA-SAX": "small_airport",
                        "PAAK": "small_airport",
                        "PAAL": "small_airport",
                        "PAAP": "seaplane_base",
                        "PAAQ": "medium_airport",
                        "PAAT": "closed",
                        "PABA": "medium_airport",
                        "PABE": "medium_airport",
                        "PABG": "small_airport",
                        "PABI": "medium_airport",
                        "PABL": "medium_airport",
                        "PABM": "small_airport",
                        "PABR": "medium_airport",
                        "PABT": "medium_airport",
                        "PACD": "medium_airport",
                        "PACE": "small_airport",
                        "PACI": "small_airport",
                        "PACK": "small_airport",
                        "PACM": "medium_airport",
                        "PACR": "small_airport",
                        "PACV": "medium_airport",
                        "PACX": "small_airport",
                        "PACY": "small_airport",
                        "PACZ": "medium_airport",
                        "PADE": "medium_airport",
                        "PADG": "small_airport",
                        "PADK": "medium_airport",
                        "PADL": "medium_airport",
                        "PADM": "medium_airport",
                        "PADQ": "medium_airport",
                        "PADU": "medium_airport",
                        "PADY": "small_airport",
                        "PAED": "medium_airport",
                        "PAEE": "small_airport",
                        "PAEG": "small_airport",
                        "PAEH": "medium_airport",
                        "PAEI": "medium_airport",
                        "PAEL": "seaplane_base",
                        "PAEM": "medium_airport",
                        "PAEN": "medium_airport",
                        "PAEW": "small_airport",
                        "PAFA": "large_airport",
                        "PAFB": "medium_airport",
                        "PAFM": "medium_airport",
                        "PAFR": "heliport",
                        "PAFS": "small_airport",
                        "PAFV": "small_airport",
                        "PAFW": "small_airport",
                        "PAGA": "medium_airport",
                        "PAGB": "small_airport",
                        "PAGG": "small_airport",
                        "PAGH": "small_airport",
                        "PAGK": "medium_airport",
                        "PAGL": "small_airport",
                        "PAGM": "medium_airport",
                        "PAGN": "seaplane_base",
                        "PAGQ": "small_airport",
                        "PAGS": "medium_airport",
                        "PAGT": "small_airport",
                        "PAGY": "small_airport",
                        "PAHC": "medium_airport",
                        "PAHL": "medium_airport",
                        "PAHN": "medium_airport",
                        "PAHO": "medium_airport",
                        "PAHP": "medium_airport",
                        "PAHU": "small_airport",
                        "PAHX": "small_airport",
                        "PAHY": "seaplane_base",
                        "PAIG": "small_airport",
                        "PAII": "medium_airport",
                        "PAIK": "medium_airport",
                        "PAIL": "medium_airport",
                        "PAIM": "medium_airport",
                        "PAIN": "small_airport",
                        "PAIW": "small_airport",
                        "PAJN": "medium_airport",
                        "PAJZ": "small_airport",
                        "PAKD": "small_airport",
                        "PAKF": "small_airport",
                        "PAKH": "small_airport",
                        "PAKI": "small_airport",
                        "PAKK": "small_airport",
                        "PAKL": "small_airport",
                        "PAKN": "medium_airport",
                        "PAKO": "small_airport",
                        "PAKP": "medium_airport",
                        "PAKT": "medium_airport",
                        "PAKU": "small_airport",
                        "PAKV": "small_airport",
                        "PAKW": "medium_airport",
                        "PAKY": "small_airport",
                        "PALB": "small_airport",
                        "PALG": "small_airport",
                        "PALP": "small_airport",
                        "PALR": "small_airport",
                        "PALU": "medium_airport",
                        "PAMB": "small_airport",
                        "PAMC": "medium_airport",
                        "PAMD": "small_airport",
                        "PAMH": "small_airport",
                        "PAMK": "small_airport",
                        "PAML": "small_airport",
                        "PAMM": "seaplane_base",
                        "PAMO": "small_airport",
                        "PAMR": "medium_airport",
                        "PAMX": "small_airport",
                        "PAMY": "medium_airport",
                        "PANA": "small_airport",
                        "PANC": "large_airport",
                        "PANI": "medium_airport",
                        "PANN": "medium_airport",
                        "PANO": "small_airport",
                        "PANR": "seaplane_base",
                        "PANT": "medium_airport",
                        "PANU": "small_airport",
                        "PANV": "medium_airport",
                        "PANW": "small_airport",
                        "PAOB": "small_airport",
                        "PAOC": "small_airport",
                        "PAOH": "small_airport",
                        "PAOM": "medium_airport",
                        "PAOO": "small_airport",
                        "PAOR": "medium_airport",
                        "PAOT": "medium_airport",
                        "PAOU": "small_airport",
                        "PAPB": "medium_airport",
                        "PAPC": "medium_airport",
                        "PAPE": "small_airport",
                        "PAPG": "medium_airport",
                        "PAPH": "medium_airport",
                        "PAPK": "small_airport",
                        "PAPM": "medium_airport",
                        "PAPN": "small_airport",
                        "PAPO": "small_airport",
                        "PAPR": "small_airport",
                        "PAQH": "small_airport",
                        "PAQT": "medium_airport",
                        "PARC": "medium_airport",
                        "PARS": "small_airport",
                        "PARY": "medium_airport",
                        "PASA": "medium_airport",
                        "PASC": "medium_airport",
                        "PASD": "medium_airport",
                        "PASH": "medium_airport",
                        "PASI": "medium_airport",
                        "PASK": "medium_airport",
                        "PASL": "small_airport",
                        "PASM": "medium_airport",
                        "PASN": "medium_airport",
                        "PASO": "small_airport",
                        "PASP": "small_airport",
                        "PAST": "small_airport",
                        "PASV": "medium_airport",
                        "PASW": "small_airport",
                        "PASX": "medium_airport",
                        "PASY": "medium_airport",
                        "PATA": "small_airport",
                        "PATC": "small_airport",
                        "PATE": "small_airport",
                        "PATG": "medium_airport",
                        "PATK": "medium_airport",
                        "PATL": "medium_airport",
                        "PATQ": "medium_airport",
                        "PAUK": "small_airport",
                        "PAUM": "small_airport",
                        "PAUN": "small_airport",
                        "PAUO": "small_airport",
                        "PAVA": "small_airport",
                        "PAVC": "small_airport",
                        "PAVD": "medium_airport",
                        "PAVE": "small_airport",
                        "PAVL": "medium_airport",
                        "PAWB": "small_airport",
                        "PAWD": "medium_airport",
                        "PAWG": "medium_airport",
                        "PAWI": "medium_airport",
                        "PAWM": "small_airport",
                        "PAWN": "medium_airport",
                        "PAWS": "medium_airport",
                        "PAYA": "medium_airport",
                        "PCIS": "medium_airport",
                        "PCO": "small_airport",
                        "PCQ": "small_airport",
                        "PDI": "small_airport",
                        "PDR": "small_airport",
                        "PE-MFT": "small_airport",
                        "PEC": "seaplane_base",
                        "PFAK": "small_airport",
                        "PFAL": "small_airport",
                        "PFC": "small_airport",
                        "PFCB": "small_airport",
                        "PFCL": "small_airport",
                        "PFEL": "small_airport",
                        "PFKA": "small_airport",
                        "PFKK": "small_airport",
                        "PFKO": "small_airport",
                        "PFKT": "small_airport",
                        "PFKU": "small_airport",
                        "PFKW": "small_airport",
                        "PFNO": "small_airport",
                        "PFSH": "small_airport",
                        "PFTO": "small_airport",
                        "PFWS": "small_airport",
                        "PFYU": "medium_airport",
                        "PG-0005": "small_airport",
                        "PG-0006": "small_airport",
                        "PG-0007": "small_airport",
                        "PG-0008": "small_airport",
                        "PG-0009": "small_airport",
                        "PG-0011": "small_airport",
                        "PG-0020": "small_airport",
                        "PG-0022": "small_airport",
                        "PG-0035": "small_airport",
                        "PG-0038": "small_airport",
                        "PG-0048": "small_airport",
                        "PG-AKG": "small_airport",
                        "PG-ATP": "small_airport",
                        "PG-AWB": "small_airport",
                        "PG-BAA": "small_airport",
                        "PG-CVB": "small_airport",
                        "PG-GMI": "small_airport",
                        "PG-GVI": "small_airport",
                        "PG-HYF": "small_airport",
                        "PG-IHU": "small_airport",
                        "PG-IIS": "small_airport",
                        "PG-JAQ": "small_airport",
                        "PG-KDR": "small_airport",
                        "PG-KKD": "small_airport",
                        "PG-KUY": "small_airport",
                        "PG-KWO": "small_airport",
                        "PG-KXR": "closed",
                        "PG-LMI": "small_airport",
                        "PG-LMY": "small_airport",
                        "PG-OBX": "small_airport",
                        "PG-OPU": "medium_airport",
                        "PG-SKC": "small_airport",
                        "PG-TFI": "small_airport",
                        "PG-TFM": "small_airport",
                        "PG-TLO": "small_airport",
                        "PG-UKU": "small_airport",
                        "PG-ULE": "small_airport",
                        "PG-VMU": "medium_airport",
                        "PG-WPM": "small_airport",
                        "PGE": "small_airport",
                        "PGM": "small_airport",
                        "PGP": "closed",
                        "PGRO": "medium_airport",
                        "PGSN": "medium_airport",
                        "PGUA": "medium_airport",
                        "PGUM": "large_airport",
                        "PGWT": "medium_airport",
                        "PH-0001": "large_airport",
                        "PH-ENI": "small_airport",
                        "PHBK": "medium_airport",
                        "PHDH": "medium_airport",
                        "PHHI": "medium_airport",
                        "PHHN": "medium_airport",
                        "PHJH": "medium_airport",
                        "PHJR": "medium_airport",
                        "PHKO": "medium_airport",
                        "PHKP": "closed",
                        "PHLI": "medium_airport",
                        "PHLU": "small_airport",
                        "PHMK": "medium_airport",
                        "PHMU": "medium_airport",
                        "PHNG": "medium_airport",
                        "PHNL": "large_airport",
                        "PHNY": "medium_airport",
                        "PHOG": "medium_airport",
                        "PHPA": "small_airport",
                        "PHSF": "medium_airport",
                        "PHTO": "medium_airport",
                        "PHUP": "medium_airport",
                        "PJON": "closed",
                        "PK-BHC": "small_airport",
                        "PK-CWP": "small_airport",
                        "PK-GRT": "small_airport",
                        "PK-HRA": "small_airport",
                        "PK-KCF": "small_airport",
                        "PK-REQ": "small_airport",
                        "PK-RZS": "small_airport",
                        "PK-SWV": "closed",
                        "PKMA": "small_airport",
                        "PKMJ": "medium_airport",
                        "PKSA": "medium_airport",
                        "PKWA": "medium_airport",
                        "PLCH": "medium_airport",
                        "PLE": "small_airport",
                        "PLFA": "small_airport",
                        "PLWN": "small_airport",
                        "PMDY": "medium_airport",
                        "PMM": "small_airport",
                        "PMP": "small_airport",
                        "PPIZ": "medium_airport",
                        "PPX": "small_airport",
                        "PQD": "heliport",
                        "PR-DDP": "closed",
                        "PR-HUC": "small_airport",
                        "PR-PPD": "closed",
                        "PRNO": "small_airport",
                        "PTKK": "medium_airport",
                        "PTPN": "medium_airport",
                        "PTRO": "medium_airport",
                        "PTSA": "medium_airport",
                        "PTYA": "medium_airport",
                        "PUA": "small_airport",
                        "PUI": "closed",
                        "PWAK": "medium_airport",
                        "PWR": "seaplane_base",
                        "PY-BFA": "small_airport",
                        "PY-OLK": "small_airport",
                        "PY-PBT": "small_airport",
                        "PY-PCJ": "small_airport",
                        "PYL": "seaplane_base",
                        "Q51": "small_airport",
                        "RAA": "small_airport",
                        "RAW": "small_airport",
                        "RAX": "small_airport",
                        "RBP": "small_airport",
                        "RCBS": "large_airport",
                        "RCFG": "medium_airport",
                        "RCFN": "medium_airport",
                        "RCGI": "small_airport",
                        "RCKH": "large_airport",
                        "RCKU": "medium_airport",
                        "RCKW": "medium_airport",
                        "RCLG": "closed",
                        "RCLY": "medium_airport",
                        "RCMQ": "medium_airport",
                        "RCMT": "medium_airport",
                        "RCN": "closed",
                        "RCNN": "medium_airport",
                        "RCPO": "medium_airport",
                        "RCQC": "medium_airport",
                        "RCSQ": "medium_airport",
                        "RCSS": "medium_airport",
                        "RCTP": "large_airport",
                        "RCWA": "small_airport",
                        "RCYU": "medium_airport",
                        "RDV": "small_airport",
                        "REH": "closed",
                        "RGE": "closed",
                        "RHR": "seaplane_base",
                        "RHT": "small_airport",
                        "RJAA": "large_airport",
                        "RJAF": "medium_airport",
                        "RJAH": "medium_airport",
                        "RJAM": "small_airport",
                        "RJAW": "medium_airport",
                        "RJBB": "large_airport",
                        "RJBD": "medium_airport",
                        "RJBE": "medium_airport",
                        "RJBH": "heliport",
                        "RJBT": "medium_airport",
                        "RJCB": "medium_airport",
                        "RJCC": "large_airport",
                        "RJCH": "medium_airport",
                        "RJCK": "medium_airport",
                        "RJCM": "medium_airport",
                        "RJCN": "medium_airport",
                        "RJCO": "medium_airport",
                        "RJCR": "small_airport",
                        "RJCW": "medium_airport",
                        "RJDA": "small_airport",
                        "RJDB": "medium_airport",
                        "RJDC": "medium_airport",
                        "RJDT": "medium_airport",
                        "RJEB": "medium_airport",
                        "RJEC": "medium_airport",
                        "RJEO": "medium_airport",
                        "RJER": "medium_airport",
                        "RJFC": "medium_airport",
                        "RJFE": "medium_airport",
                        "RJFF": "large_airport",
                        "RJFG": "medium_airport",
                        "RJFK": "large_airport",
                        "RJFM": "medium_airport",
                        "RJFO": "medium_airport",
                        "RJFR": "medium_airport",
                        "RJFS": "medium_airport",
                        "RJFT": "medium_airport",
                        "RJFU": "medium_airport",
                        "RJGG": "large_airport",
                        "RJKA": "medium_airport",
                        "RJKB": "medium_airport",
                        "RJKI": "medium_airport",
                        "RJKN": "medium_airport",
                        "RJNA": "medium_airport",
                        "RJNF": "medium_airport",
                        "RJNG": "medium_airport",
                        "RJNK": "medium_airport",
                        "RJNO": "medium_airport",
                        "RJNT": "medium_airport",
                        "RJNW": "medium_airport",
                        "RJOA": "medium_airport",
                        "RJOB": "medium_airport",
                        "RJOC": "medium_airport",
                        "RJOH": "medium_airport",
                        "RJOI": "medium_airport",
                        "RJOK": "medium_airport",
                        "RJOM": "medium_airport",
                        "RJOO": "large_airport",
                        "RJOR": "medium_airport",
                        "RJOS": "medium_airport",
                        "RJOT": "medium_airport",
                        "RJOW": "medium_airport",
                        "RJSA": "medium_airport",
                        "RJSC": "medium_airport",
                        "RJSD": "medium_airport",
                        "RJSF": "medium_airport",
                        "RJSH": "medium_airport",
                        "RJSI": "medium_airport",
                        "RJSK": "medium_airport",
                        "RJSM": "medium_airport",
                        "RJSN": "medium_airport",
                        "RJSR": "medium_airport",
                        "RJSS": "medium_airport",
                        "RJSY": "medium_airport",
                        "RJTA": "medium_airport",
                        "RJTH": "medium_airport",
                        "RJTO": "medium_airport",
                        "RJTQ": "medium_airport",
                        "RJTT": "large_airport",
                        "RJTY": "large_airport",
                        "RKJB": "large_airport",
                        "RKJJ": "medium_airport",
                        "RKJK": "large_airport",
                        "RKJM": "closed",
                        "RKJU": "small_airport",
                        "RKJY": "medium_airport",
                        "RKNC": "closed",
                        "RKND": "closed",
                        "RKNN": "medium_airport",
                        "RKNW": "medium_airport",
                        "RKNY": "medium_airport",
                        "RKPC": "large_airport",
                        "RKPD": "medium_airport",
                        "RKPE": "small_airport",
                        "RKPK": "large_airport",
                        "RKPS": "medium_airport",
                        "RKPU": "medium_airport",
                        "RKRO": "small_airport",
                        "RKSI": "large_airport",
                        "RKSM": "medium_airport",
                        "RKSO": "large_airport",
                        "RKSS": "large_airport",
                        "RKSW": "medium_airport",
                        "RKTA": "small_airport",
                        "RKTH": "medium_airport",
                        "RKTN": "medium_airport",
                        "RKTP": "small_airport",
                        "RKTU": "large_airport",
                        "RKTY": "medium_airport",
                        "RKU": "small_airport",
                        "RKY": "small_airport",
                        "RLP": "small_airport",
                        "RMP": "small_airport",
                        "RMU": "medium_airport",
                        "RNR": "closed",
                        "ROAH": "large_airport",
                        "RODN": "large_airport",
                        "ROKJ": "medium_airport",
                        "ROKR": "small_airport",
                        "ROMD": "medium_airport",
                        "ROMY": "medium_airport",
                        "RORA": "small_airport",
                        "RORE": "medium_airport",
                        "RORH": "small_airport",
                        "RORK": "medium_airport",
                        "RORS": "medium_airport",
                        "RORT": "medium_airport",
                        "RORY": "medium_airport",
                        "ROYN": "medium_airport",
                        "RPBL": "small_airport",
                        "RPLB": "medium_airport",
                        "RPLC": "large_airport",
                        "RPLI": "medium_airport",
                        "RPLL": "large_airport",
                        "RPLO": "small_airport",
                        "RPLP": "medium_airport",
                        "RPLS": "small_airport",
                        "RPLU": "medium_airport",
                        "RPMA": "medium_airport",
                        "RPMC": "medium_airport",
                        "RPMD": "large_airport",
                        "RPME": "medium_airport",
                        "RPMF": "medium_airport",
                        "RPMG": "medium_airport",
                        "RPMH": "medium_airport",
                        "RPMI": "small_airport",
                        "RPMJ": "medium_airport",
                        "RPMM": "small_airport",
                        "RPMN": "medium_airport",
                        "RPMO": "medium_airport",
                        "RPMP": "medium_airport",
                        "RPMQ": "medium_airport",
                        "RPMR": "medium_airport",
                        "RPMS": "medium_airport",
                        "RPMU": "small_airport",
                        "RPMV": "small_airport",
                        "RPMW": "medium_airport",
                        "RPMZ": "medium_airport",
                        "RPNS": "small_airport",
                        "RPSD": "small_airport",
                        "RPSV": "small_airport",
                        "RPUB": "medium_airport",
                        "RPUD": "medium_airport",
                        "RPUH": "medium_airport",
                        "RPUM": "medium_airport",
                        "RPUN": "medium_airport",
                        "RPUO": "medium_airport",
                        "RPUR": "medium_airport",
                        "RPUS": "medium_airport",
                        "RPUT": "medium_airport",
                        "RPUV": "medium_airport",
                        "RPUW": "medium_airport",
                        "RPUY": "medium_airport",
                        "RPV": "small_airport",
                        "RPVA": "medium_airport",
                        "RPVB": "medium_airport",
                        "RPVC": "medium_airport",
                        "RPVD": "medium_airport",
                        "RPVE": "medium_airport",
                        "RPVF": "medium_airport",
                        "RPVI": "medium_airport",
                        "RPVJ": "medium_airport",
                        "RPVK": "medium_airport",
                        "RPVM": "large_airport",
                        "RPVO": "medium_airport",
                        "RPVP": "medium_airport",
                        "RPVR": "medium_airport",
                        "RPVS": "medium_airport",
                        "RPVT": "medium_airport",
                        "RPVU": "medium_airport",
                        "RPVV": "medium_airport",
                        "RRM": "small_airport",
                        "RSE": "seaplane_base",
                        "RU-0025": "small_airport",
                        "RU-0035": "large_airport",
                        "RU-0200": "small_airport",
                        "RU-0493": "small_airport",
                        "RU-VUS": "small_airport",
                        "RUU": "small_airport",
                        "RVC": "small_airport",
                        "RZH": "closed",
                        "S31": "small_airport",
                        "S60": "seaplane_base",
                        "SA14": "small_airport",
                        "SA30": "closed",
                        "SAAC": "medium_airport",
                        "SAAG": "medium_airport",
                        "SAAJ": "small_airport",
                        "SAAP": "medium_airport",
                        "SAAR": "medium_airport",
                        "SAAV": "medium_airport",
                        "SABE": "medium_airport",
                        "SACC": "small_airport",
                        "SACO": "medium_airport",
                        "SADF": "medium_airport",
                        "SADL": "medium_airport",
                        "SADP": "medium_airport",
                        "SAEZ": "large_airport",
                        "SAFR": "small_airport",
                        "SAHC": "small_airport",
                        "SAHE": "small_airport",
                        "SAHR": "small_airport",
                        "SAHS": "medium_airport",
                        "SAHZ": "medium_airport",
                        "SAM": "small_airport",
                        "SAME": "medium_airport",
                        "SAMM": "medium_airport",
                        "SAMR": "medium_airport",
                        "SANC": "medium_airport",
                        "SANE": "medium_airport",
                        "SANL": "medium_airport",
                        "SANR": "medium_airport",
                        "SANT": "medium_airport",
                        "SANU": "medium_airport",
                        "SANW": "small_airport",
                        "SAOC": "medium_airport",
                        "SAOD": "medium_airport",
                        "SAOR": "medium_airport",
                        "SAOS": "small_airport",
                        "SAOU": "medium_airport",
                        "SARC": "medium_airport",
                        "SARE": "medium_airport",
                        "SARF": "medium_airport",
                        "SARI": "medium_airport",
                        "SARL": "medium_airport",
                        "SARM": "medium_airport",
                        "SARP": "medium_airport",
                        "SAS": "small_airport",
                        "SASA": "medium_airport",
                        "SASJ": "medium_airport",
                        "SASO": "medium_airport",
                        "SAST": "medium_airport",
                        "SATC": "small_airport",
                        "SATD": "small_airport",
                        "SATG": "medium_airport",
                        "SATK": "small_airport",
                        "SATM": "small_airport",
                        "SATR": "medium_airport",
                        "SATU": "medium_airport",
                        "SAVB": "medium_airport",
                        "SAVC": "medium_airport",
                        "SAVD": "small_airport",
                        "SAVE": "medium_airport",
                        "SAVH": "medium_airport",
                        "SAVJ": "small_airport",
                        "SAVN": "small_airport",
                        "SAVQ": "small_airport",
                        "SAVR": "small_airport",
                        "SAVS": "small_airport",
                        "SAVT": "medium_airport",
                        "SAVV": "medium_airport",
                        "SAVY": "medium_airport",
                        "SAWA": "closed",
                        "SAWC": "medium_airport",
                        "SAWD": "medium_airport",
                        "SAWE": "medium_airport",
                        "SAWG": "medium_airport",
                        "SAWH": "medium_airport",
                        "SAWJ": "medium_airport",
                        "SAWM": "small_airport",
                        "SAWP": "medium_airport",
                        "SAWR": "small_airport",
                        "SAWS": "small_airport",
                        "SAWT": "small_airport",
                        "SAWU": "medium_airport",
                        "SAZB": "medium_airport",
                        "SAZC": "small_airport",
                        "SAZF": "small_airport",
                        "SAZG": "medium_airport",
                        "SAZH": "medium_airport",
                        "SAZL": "medium_airport",
                        "SAZM": "medium_airport",
                        "SAZN": "medium_airport",
                        "SAZO": "medium_airport",
                        "SAZP": "medium_airport",
                        "SAZR": "medium_airport",
                        "SAZS": "medium_airport",
                        "SAZT": "medium_airport",
                        "SAZV": "medium_airport",
                        "SAZW": "medium_airport",
                        "SAZY": "medium_airport",
                        "SB-VIU": "small_airport",
                        "SB29": "closed",
                        "SBAA": "medium_airport",
                        "SBAC": "medium_airport",
                        "SBAQ": "medium_airport",
                        "SBAR": "medium_airport",
                        "SBAT": "medium_airport",
                        "SBAU": "medium_airport",
                        "SBAX": "small_airport",
                        "SBBE": "large_airport",
                        "SBBG": "medium_airport",
                        "SBBH": "medium_airport",
                        "SBBI": "medium_airport",
                        "SBBP": "small_airport",
                        "SBBR": "large_airport",
                        "SBBT": "medium_airport",
                        "SBBU": "medium_airport",
                        "SBBV": "medium_airport",
                        "SBBW": "medium_airport",
                        "SBBZ": "medium_airport",
                        "SBCA": "medium_airport",
                        "SBCB": "small_airport",
                        "SBCD": "small_airport",
                        "SBCF": "large_airport",
                        "SBCG": "medium_airport",
                        "SBCH": "medium_airport",
                        "SBCI": "medium_airport",
                        "SBCJ": "medium_airport",
                        "SBCM": "medium_airport",
                        "SBCN": "small_airport",
                        "SBCP": "medium_airport",
                        "SBCR": "medium_airport",
                        "SBCT": "large_airport",
                        "SBCV": "medium_airport",
                        "SBCX": "medium_airport",
                        "SBCY": "medium_airport",
                        "SBCZ": "medium_airport",
                        "SBDB": "small_airport",
                        "SBDN": "medium_airport",
                        "SBEG": "large_airport",
                        "SBEK": "medium_airport",
                        "SBEP": "small_airport",
                        "SBFI": "medium_airport",
                        "SBFL": "large_airport",
                        "SBFN": "medium_airport",
                        "SBFZ": "medium_airport",
                        "SBGL": "large_airport",
                        "SBGM": "medium_airport",
                        "SBGO": "medium_airport",
                        "SBGR": "large_airport",
                        "SBGU": "small_airport",
                        "SBGV": "small_airport",
                        "SBGW": "medium_airport",
                        "SBHT": "medium_airport",
                        "SBIC": "medium_airport",
                        "SBIH": "medium_airport",
                        "SBIL": "medium_airport",
                        "SBIP": "medium_airport",
                        "SBIT": "small_airport",
                        "SBIZ": "medium_airport",
                        "SBJA": "small_airport",
                        "SBJF": "medium_airport",
                        "SBJP": "medium_airport",
                        "SBJU": "small_airport",
                        "SBJV": "medium_airport",
                        "SBKG": "medium_airport",
                        "SBKP": "medium_airport",
                        "SBLE": "small_airport",
                        "SBLJ": "medium_airport",
                        "SBLN": "medium_airport",
                        "SBLO": "medium_airport",
                        "SBLP": "medium_airport",
                        "SBMA": "medium_airport",
                        "SBMC": "medium_airport",
                        "SBMD": "medium_airport",
                        "SBME": "medium_airport",
                        "SBMG": "medium_airport",
                        "SBMK": "medium_airport",
                        "SBML": "medium_airport",
                        "SBMN": "medium_airport",
                        "SBMO": "medium_airport",
                        "SBMQ": "medium_airport",
                        "SBMS": "medium_airport",
                        "SBMY": "medium_airport",
                        "SBNF": "medium_airport",
                        "SBNM": "medium_airport",
                        "SBOI": "medium_airport",
                        "SBPA": "medium_airport",
                        "SBPB": "small_airport",
                        "SBPC": "medium_airport",
                        "SBPF": "medium_airport",
                        "SBPJ": "medium_airport",
                        "SBPK": "medium_airport",
                        "SBPL": "medium_airport",
                        "SBPN": "medium_airport",
                        "SBPP": "medium_airport",
                        "SBPS": "medium_airport",
                        "SBPV": "medium_airport",
                        "SBRB": "medium_airport",
                        "SBRF": "medium_airport",
                        "SBRJ": "medium_airport",
                        "SBRP": "medium_airport",
                        "SBRR": "small_airport",
                        "SBSC": "medium_airport",
                        "SBSG": "large_airport",
                        "SBSJ": "medium_airport",
                        "SBSL": "medium_airport",
                        "SBSM": "medium_airport",
                        "SBSN": "medium_airport",
                        "SBSO": "small_airport",
                        "SBSP": "large_airport",
                        "SBSR": "medium_airport",
                        "SBST": "medium_airport",
                        "SBSV": "large_airport",
                        "SBTA": "medium_airport",
                        "SBTB": "medium_airport",
                        "SBTC": "small_airport",
                        "SBTD": "small_airport",
                        "SBTE": "medium_airport",
                        "SBTF": "medium_airport",
                        "SBTK": "medium_airport",
                        "SBTL": "medium_airport",
                        "SBTR": "small_airport",
                        "SBTT": "medium_airport",
                        "SBTU": "medium_airport",
                        "SBUA": "medium_airport",
                        "SBUF": "medium_airport",
                        "SBUG": "medium_airport",
                        "SBUL": "medium_airport",
                        "SBUR": "medium_airport",
                        "SBVG": "medium_airport",
                        "SBVH": "medium_airport",
                        "SBVT": "medium_airport",
                        "SBYS": "medium_airport",
                        "SBZM": "medium_airport",
                        "SCAC": "small_airport",
                        "SCAN": "small_airport",
                        "SCAP": "small_airport",
                        "SCAR": "medium_airport",
                        "SCAS": "small_airport",
                        "SCAT": "medium_airport",
                        "SCBA": "medium_airport",
                        "SCBE": "medium_airport",
                        "SCBI": "small_airport",
                        "SCCC": "medium_airport",
                        "SCCF": "medium_airport",
                        "SCCH": "medium_airport",
                        "SCCI": "medium_airport",
                        "SCCQ": "small_airport",
                        "SCCY": "medium_airport",
                        "SCDA": "medium_airport",
                        "SCEL": "large_airport",
                        "SCES": "medium_airport",
                        "SCEV": "small_airport",
                        "SCFA": "medium_airport",
                        "SCFM": "medium_airport",
                        "SCFT": "small_airport",
                        "SCGE": "small_airport",
                        "SCGZ": "medium_airport",
                        "SCHR": "medium_airport",
                        "SCIE": "medium_airport",
                        "SCIP": "medium_airport",
                        "SCJO": "medium_airport",
                        "SCKP": "small_airport",
                        "SCLL": "medium_airport",
                        "SCLN": "small_airport",
                        "SCNT": "medium_airport",
                        "SCOV": "small_airport",
                        "SCPC": "small_airport",
                        "SCPQ": "small_airport",
                        "SCPV": "small_airport",
                        "SCQP": "medium_airport",
                        "SCR": "closed",
                        "SCRA": "medium_airport",
                        "SCRD": "small_airport",
                        "SCRG": "medium_airport",
                        "SCRM": "small_airport",
                        "SCSB": "small_airport",
                        "SCSE": "medium_airport",
                        "SCSF": "small_airport",
                        "SCST": "small_airport",
                        "SCTC": "medium_airport",
                        "SCTE": "medium_airport",
                        "SCTI": "closed",
                        "SCTL": "medium_airport",
                        "SCTN": "medium_airport",
                        "SCTO": "medium_airport",
                        "SCTT": "medium_airport",
                        "SCVD": "medium_airport",
                        "SCVM": "medium_airport",
                        "SDAM": "small_airport",
                        "SDBK": "small_airport",
                        "SDCG": "small_airport",
                        "SDCO": "small_airport",
                        "SDDR": "small_airport",
                        "SDI": "small_airport",
                        "SDJL": "small_airport",
                        "SDKK": "small_airport",
                        "SDLP": "small_airport",
                        "SDNY": "small_airport",
                        "SDOU": "small_airport",
                        "SDOW": "small_airport",
                        "SDPW": "small_airport",
                        "SDRK": "small_airport",
                        "SDRS": "small_airport",
                        "SDSC": "small_airport",
                        "SDUB": "small_airport",
                        "SDUN": "small_airport",
                        "SDVG": "small_airport",
                        "SDYM": "small_airport",
                        "SDZY": "medium_airport",
                        "SE-0016": "medium_airport",
                        "SEAM": "medium_airport",
                        "SECO": "medium_airport",
                        "SECU": "medium_airport",
                        "SEGS": "small_airport",
                        "SEGU": "medium_airport",
                        "SEII": "small_airport",
                        "SEJI": "small_airport",
                        "SELT": "large_airport",
                        "SEMA": "medium_airport",
                        "SEMC": "medium_airport",
                        "SEMH": "medium_airport",
                        "SEMT": "medium_airport",
                        "SENL": "small_airport",
                        "SEPT": "small_airport",
                        "SEPV": "medium_airport",
                        "SEQM": "large_airport",
                        "SERO": "medium_airport",
                        "SESA": "medium_airport",
                        "SESC": "small_airport",
                        "SESM": "small_airport",
                        "SEST": "small_airport",
                        "SESV": "small_airport",
                        "SETH": "small_airport",
                        "SETI": "small_airport",
                        "SETM": "small_airport",
                        "SETN": "small_airport",
                        "SETR": "medium_airport",
                        "SETU": "medium_airport",
                        "SFAL": "medium_airport",
                        "SFR": "closed",
                        "SFU": "small_airport",
                        "SGAS": "medium_airport",
                        "SGAY": "medium_airport",
                        "SGCO": "small_airport",
                        "SGEN": "small_airport",
                        "SGES": "medium_airport",
                        "SGFI": "small_airport",
                        "SGL": "heliport",
                        "SGME": "medium_airport",
                        "SGPI": "medium_airport",
                        "SGPJ": "small_airport",
                        "SIC": "small_airport",
                        "SILC": "small_airport",
                        "SIMK": "small_airport",
                        "SIZ": "small_airport",
                        "SIZX": "small_airport",
                        "SJF": "seaplane_base",
                        "SJHG": "small_airport",
                        "SJNP": "small_airport",
                        "SJRG": "small_airport",
                        "SJTC": "medium_airport",
                        "SJVO": "small_airport",
                        "SK-001": "small_airport",
                        "SK-002": "small_airport",
                        "SK-005": "small_airport",
                        "SK-008": "small_airport",
                        "SK-020": "small_airport",
                        "SK-021": "small_airport",
                        "SK-045": "small_airport",
                        "SK-046": "small_airport",
                        "SK-047": "small_airport",
                        "SK-048": "small_airport",
                        "SK-059": "small_airport",
                        "SK-109": "small_airport",
                        "SK-115": "small_airport",
                        "SK-116": "small_airport",
                        "SK-120": "small_airport",
                        "SK-131": "small_airport",
                        "SK-144": "small_airport",
                        "SK-154": "small_airport",
                        "SK-174": "small_airport",
                        "SK-183": "small_airport",
                        "SK-189": "small_airport",
                        "SK-193": "small_airport",
                        "SK-197": "small_airport",
                        "SK-198": "small_airport",
                        "SK-209": "small_airport",
                        "SK-217": "small_airport",
                        "SK-223": "small_airport",
                        "SK-236": "small_airport",
                        "SK-245": "small_airport",
                        "SK-272": "small_airport",
                        "SK-277": "small_airport",
                        "SK-284": "small_airport",
                        "SK-307": "small_airport",
                        "SK-348": "small_airport",
                        "SK-354": "small_airport",
                        "SK-360": "small_airport",
                        "SK-365": "small_airport",
                        "SK-367": "small_airport",
                        "SK-392": "small_airport",
                        "SK-422": "small_airport",
                        "SK-442": "small_airport",
                        "SK-462": "small_airport",
                        "SK-471": "small_airport",
                        "SK-481": "small_airport",
                        "SK-482": "small_airport",
                        "SK-511": "small_airport",
                        "SK-521": "small_airport",
                        "SKAC": "small_airport",
                        "SKAD": "small_airport",
                        "SKAM": "small_airport",
                        "SKAN": "closed",
                        "SKAP": "medium_airport",
                        "SKAR": "medium_airport",
                        "SKAS": "medium_airport",
                        "SKBC": "medium_airport",
                        "SKBG": "medium_airport",
                        "SKBO": "large_airport",
                        "SKBQ": "medium_airport",
                        "SKBS": "medium_airport",
                        "SKBU": "medium_airport",
                        "SKCA": "small_airport",
                        "SKCC": "medium_airport",
                        "SKCD": "small_airport",
                        "SKCG": "medium_airport",
                        "SKCI": "small_airport",
                        "SKCL": "medium_airport",
                        "SKCM": "small_airport",
                        "SKCN": "small_airport",
                        "SKCO": "medium_airport",
                        "SKCR": "small_airport",
                        "SKCU": "medium_airport",
                        "SKCV": "medium_airport",
                        "SKCZ": "medium_airport",
                        "SKEB": "medium_airport",
                        "SKEJ": "medium_airport",
                        "SKFL": "medium_airport",
                        "SKFU": "small_airport",
                        "SKGA": "small_airport",
                        "SKGI": "medium_airport",
                        "SKGO": "medium_airport",
                        "SKGP": "medium_airport",
                        "SKHA": "small_airport",
                        "SKHC": "small_airport",
                        "SKIB": "medium_airport",
                        "SKIG": "small_airport",
                        "SKIP": "medium_airport",
                        "SKJ": "closed",
                        "SKLC": "medium_airport",
                        "SKLG": "small_airport",
                        "SKLM": "medium_airport",
                        "SKLP": "small_airport",
                        "SKLT": "medium_airport",
                        "SKMD": "medium_airport",
                        "SKMF": "small_airport",
                        "SKMG": "medium_airport",
                        "SKML": "small_airport",
                        "SKMR": "medium_airport",
                        "SKMU": "medium_airport",
                        "SKMZ": "medium_airport",
                        "SKNC": "small_airport",
                        "SKNQ": "small_airport",
                        "SKNV": "medium_airport",
                        "SKOC": "medium_airport",
                        "SKOE": "small_airport",
                        "SKOT": "closed",
                        "SKPC": "medium_airport",
                        "SKPD": "medium_airport",
                        "SKPE": "medium_airport",
                        "SKPI": "medium_airport",
                        "SKPL": "small_airport",
                        "SKPN": "small_airport",
                        "SKPP": "medium_airport",
                        "SKPQ": "medium_airport",
                        "SKPR": "small_airport",
                        "SKPS": "medium_airport",
                        "SKPV": "medium_airport",
                        "SKPZ": "medium_airport",
                        "SKQU": "medium_airport",
                        "SKRG": "medium_airport",
                        "SKRH": "medium_airport",
                        "SKSA": "small_airport",
                        "SKSJ": "medium_airport",
                        "SKSL": "small_airport",
                        "SKSM": "medium_airport",
                        "SKSO": "small_airport",
                        "SKSP": "medium_airport",
                        "SKSR": "small_airport",
                        "SKSV": "medium_airport",
                        "SKTB": "small_airport",
                        "SKTD": "small_airport",
                        "SKTL": "small_airport",
                        "SKTM": "medium_airport",
                        "SKTQ": "medium_airport",
                        "SKTU": "small_airport",
                        "SKUC": "medium_airport",
                        "SKUI": "medium_airport",
                        "SKUL": "medium_airport",
                        "SKUR": "small_airport",
                        "SKVG": "small_airport",
                        "SKVL": "small_airport",
                        "SKVP": "medium_airport",
                        "SKVV": "medium_airport",
                        "SKYA": "small_airport",
                        "SKYP": "medium_airport",
                        "SLAG": "small_airport",
                        "SLAL": "medium_airport",
                        "SLAP": "small_airport",
                        "SLAS": "small_airport",
                        "SLBJ": "medium_airport",
                        "SLCA": "small_airport",
                        "SLCB": "medium_airport",
                        "SLCH": "small_airport",
                        "SLCO": "medium_airport",
                        "SLCP": "small_airport",
                        "SLET": "medium_airport",
                        "SLGY": "small_airport",
                        "SLHJ": "small_airport",
                        "SLJ": "small_airport",
                        "SLJE": "small_airport",
                        "SLJO": "small_airport",
                        "SLJV": "small_airport",
                        "SLLP": "medium_airport",
                        "SLMG": "small_airport",
                        "SLOR": "medium_airport",
                        "SLPO": "medium_airport",
                        "SLPR": "small_airport",
                        "SLPS": "medium_airport",
                        "SLRA": "small_airport",
                        "SLRB": "small_airport",
                        "SLRI": "small_airport",
                        "SLRY": "small_airport",
                        "SLSA": "medium_airport",
                        "SLSB": "small_airport",
                        "SLSI": "small_airport",
                        "SLSM": "small_airport",
                        "SLSR": "small_airport",
                        "SLTI": "small_airport",
                        "SLTJ": "medium_airport",
                        "SLTR": "medium_airport",
                        "SLUY": "small_airport",
                        "SLVG": "small_airport",
                        "SLVM": "medium_airport",
                        "SLVR": "large_airport",
                        "SLYA": "medium_airport",
                        "SMBN": "small_airport",
                        "SMCO": "small_airport",
                        "SMDA": "small_airport",
                        "SMH": "small_airport",
                        "SMJP": "medium_airport",
                        "SMMO": "small_airport",
                        "SMNI": "small_airport",
                        "SMP": "small_airport",
                        "SMPA": "small_airport",
                        "SMST": "small_airport",
                        "SMWA": "small_airport",
                        "SMZO": "small_airport",
                        "SNAI": "small_airport",
                        "SNAL": "small_airport",
                        "SNAR": "small_airport",
                        "SNAX": "small_airport",
                        "SNBC": "small_airport",
                        "SNBL": "small_airport",
                        "SNBR": "small_airport",
                        "SNBS": "small_airport",
                        "SNBU": "small_airport",
                        "SNBX": "small_airport",
                        "SNCP": "small_airport",
                        "SNCU": "small_airport",
                        "SNCX": "small_airport",
                        "SNDC": "small_airport",
                        "SNDN": "small_airport",
                        "SNDT": "small_airport",
                        "SNDV": "small_airport",
                        "SNED": "small_airport",
                        "SNFX": "small_airport",
                        "SNGA": "small_airport",
                        "SNGD": "small_airport",
                        "SNGI": "small_airport",
                        "SNGN": "small_airport",
                        "SNHA": "small_airport",
                        "SNIC": "small_airport",
                        "SNIG": "small_airport",
                        "SNIP": "small_airport",
                        "SNIU": "small_airport",
                        "SNJB": "small_airport",
                        "SNJD": "small_airport",
                        "SNJK": "small_airport",
                        "SNJN": "small_airport",
                        "SNJR": "small_airport",
                        "SNKE": "small_airport",
                        "SNKI": "small_airport",
                        "SNKN": "small_airport",
                        "SNLO": "small_airport",
                        "SNMA": "small_airport",
                        "SNMU": "small_airport",
                        "SNMX": "small_airport",
                        "SNMZ": "small_airport",
                        "SNNU": "small_airport",
                        "SNOS": "small_airport",
                        "SNOU": "small_airport",
                        "SNOX": "small_airport",
                        "SNPC": "small_airport",
                        "SNPD": "small_airport",
                        "SNPX": "small_airport",
                        "SNQ": "small_airport",
                        "SNQG": "small_airport",
                        "SNRD": "small_airport",
                        "SNRU": "small_airport",
                        "SNSW": "small_airport",
                        "SNTF": "small_airport",
                        "SNTI": "small_airport",
                        "SNTO": "small_airport",
                        "SNVB": "small_airport",
                        "SNVI": "small_airport",
                        "SNVS": "small_airport",
                        "SNWC": "small_airport",
                        "SNYE": "small_airport",
                        "SNYH": "small_airport",
                        "SNYW": "small_airport",
                        "SNZA": "small_airport",
                        "SNZW": "small_airport",
                        "SO-BXX": "small_airport",
                        "SO-GTA": "small_airport",
                        "SOA": "small_airport",
                        "SOCA": "medium_airport",
                        "SOGS": "small_airport",
                        "SOI": "heliport",
                        "SOOA": "small_airport",
                        "SOOG": "medium_airport",
                        "SOOM": "small_airport",
                        "SOOR": "small_airport",
                        "SOOS": "small_airport",
                        "SOR": "small_airport",
                        "SPAO": "small_airport",
                        "SPAR": "small_airport",
                        "SPAS": "small_airport",
                        "SPAY": "medium_airport",
                        "SPBB": "small_airport",
                        "SPBL": "small_airport",
                        "SPBR": "medium_airport",
                        "SPCL": "medium_airport",
                        "SPDR": "small_airport",
                        "SPEO": "medium_airport",
                        "SPGM": "small_airport",
                        "SPHI": "medium_airport",
                        "SPHO": "medium_airport",
                        "SPHY": "small_airport",
                        "SPHZ": "medium_airport",
                        "SPIL": "small_airport",
                        "SPIM": "large_airport",
                        "SPIZ": "small_airport",
                        "SPJA": "medium_airport",
                        "SPJE": "medium_airport",
                        "SPJI": "medium_airport",
                        "SPJJ": "medium_airport",
                        "SPJL": "medium_airport",
                        "SPJN": "small_airport",
                        "SPJR": "medium_airport",
                        "SPLN": "small_airport",
                        "SPLO": "medium_airport",
                        "SPME": "medium_airport",
                        "SPMF": "small_airport",
                        "SPMR": "small_airport",
                        "SPMS": "medium_airport",
                        "SPNC": "medium_airport",
                        "SPOA": "small_airport",
                        "SPPY": "medium_airport",
                        "SPQT": "medium_airport",
                        "SPQU": "medium_airport",
                        "SPRU": "medium_airport",
                        "SPSO": "medium_airport",
                        "SPST": "medium_airport",
                        "SPSY": "small_airport",
                        "SPTN": "medium_airport",
                        "SPTU": "medium_airport",
                        "SPUR": "medium_airport",
                        "SPYL": "medium_airport",
                        "SPZA": "medium_airport",
                        "SPZO": "large_airport",
                        "SQD": "medium_airport",
                        "SQJ": "medium_airport",
                        "SQT": "small_airport",
                        "SR-AAJ": "small_airport",
                        "SR-KCB": "small_airport",
                        "SRF": "closed",
                        "SRL": "small_airport",
                        "SRM": "small_airport",
                        "SRU": "closed",
                        "SRV": "small_airport",
                        "SSAK": "small_airport",
                        "SSAP": "small_airport",
                        "SSBG": "small_airport",
                        "SSBL": "small_airport",
                        "SSCK": "small_airport",
                        "SSCL": "small_airport",
                        "SSCN": "small_airport",
                        "SSCP": "small_airport",
                        "SSDO": "small_airport",
                        "SSER": "small_airport",
                        "SSFB": "small_airport",
                        "SSGY": "small_airport",
                        "SSHZ": "small_airport",
                        "SSIJ": "small_airport",
                        "SSIQ": "small_airport",
                        "SSJA": "small_airport",
                        "SSKM": "small_airport",
                        "SSKS": "small_airport",
                        "SSKU": "small_airport",
                        "SSKW": "small_airport",
                        "SSLN": "small_airport",
                        "SSLT": "small_airport",
                        "SSMF": "small_airport",
                        "SSNG": "small_airport",
                        "SSNH": "small_airport",
                        "SSOE": "small_airport",
                        "SSOG": "small_airport",
                        "SSPB": "small_airport",
                        "SSPG": "small_airport",
                        "SSPI": "small_airport",
                        "SSPN": "small_airport",
                        "SSQT": "small_airport",
                        "SSRU": "small_airport",
                        "SSS": "small_airport",
                        "SSSB": "small_airport",
                        "SSSC": "small_airport",
                        "SSTL": "small_airport",
                        "SSU": "closed",
                        "SSUM": "small_airport",
                        "SSUO": "heliport",
                        "SSV": "small_airport",
                        "SSVI": "small_airport",
                        "SSVP": "small_airport",
                        "SSVR": "closed",
                        "SSXX": "small_airport",
                        "SSYA": "small_airport",
                        "SSZR": "small_airport",
                        "SSZW": "medium_airport",
                        "SUAG": "small_airport",
                        "SUBU": "small_airport",
                        "SUCA": "small_airport",
                        "SUDU": "medium_airport",
                        "SULS": "medium_airport",
                        "SUMO": "small_airport",
                        "SUMU": "large_airport",
                        "SUPU": "small_airport",
                        "SURV": "small_airport",
                        "SUSO": "medium_airport",
                        "SUTB": "small_airport",
                        "SUTR": "small_airport",
                        "SUVO": "small_airport",
                        "SUZ": "closed",
                        "SVAC": "medium_airport",
                        "SVAN": "medium_airport",
                        "SVAS": "small_airport",
                        "SVBC": "large_airport",
                        "SVBI": "medium_airport",
                        "SVBM": "medium_airport",
                        "SVBS": "medium_airport",
                        "SVCB": "medium_airport",
                        "SVCD": "medium_airport",
                        "SVCE": "small_airport",
                        "SVCG": "small_airport",
                        "SVCL": "medium_airport",
                        "SVCN": "medium_airport",
                        "SVCO": "medium_airport",
                        "SVCP": "medium_airport",
                        "SVCR": "medium_airport",
                        "SVCU": "medium_airport",
                        "SVDZ": "small_airport",
                        "SVED": "medium_airport",
                        "SVEZ": "medium_airport",
                        "SVGD": "medium_airport",
                        "SVGI": "medium_airport",
                        "SVGU": "medium_airport",
                        "SVHG": "small_airport",
                        "SVIC": "small_airport",
                        "SVIE": "small_airport",
                        "SVJC": "medium_airport",
                        "SVKA": "small_airport",
                        "SVLF": "medium_airport",
                        "SVMC": "medium_airport",
                        "SVMD": "medium_airport",
                        "SVMG": "medium_airport",
                        "SVMI": "large_airport",
                        "SVMT": "medium_airport",
                        "SVON": "small_airport",
                        "SVPA": "medium_airport",
                        "SVPC": "medium_airport",
                        "SVPE": "small_airport",
                        "SVPH": "small_airport",
                        "SVPM": "medium_airport",
                        "SVPR": "medium_airport",
                        "SVPT": "medium_airport",
                        "SVRS": "small_airport",
                        "SVS": "small_airport",
                        "SVSA": "medium_airport",
                        "SVSB": "small_airport",
                        "SVSE": "medium_airport",
                        "SVSO": "medium_airport",
                        "SVSP": "medium_airport",
                        "SVSR": "medium_airport",
                        "SVST": "medium_airport",
                        "SVSZ": "medium_airport",
                        "SVTC": "medium_airport",
                        "SVTM": "medium_airport",
                        "SVUM": "small_airport",
                        "SVVA": "medium_airport",
                        "SVVG": "medium_airport",
                        "SVVL": "medium_airport",
                        "SVVP": "medium_airport",
                        "SWBC": "small_airport",
                        "SWBG": "small_airport",
                        "SWBR": "small_airport",
                        "SWCA": "small_airport",
                        "SWCQ": "small_airport",
                        "SWDM": "small_airport",
                        "SWDN": "small_airport",
                        "SWE": "small_airport",
                        "SWEI": "small_airport",
                        "SWEK": "small_airport",
                        "SWFX": "small_airport",
                        "SWG": "small_airport",
                        "SWGI": "small_airport",
                        "SWGN": "medium_airport",
                        "SWHT": "small_airport",
                        "SWII": "small_airport",
                        "SWIY": "small_airport",
                        "SWJI": "small_airport",
                        "SWJN": "small_airport",
                        "SWJU": "small_airport",
                        "SWJW": "small_airport",
                        "SWKC": "small_airport",
                        "SWKO": "small_airport",
                        "SWKT": "small_airport",
                        "SWLB": "small_airport",
                        "SWLC": "small_airport",
                        "SWMW": "small_airport",
                        "SWNA": "small_airport",
                        "SWNI": "small_airport",
                        "SWNK": "small_airport",
                        "SWNQ": "small_airport",
                        "SWNS": "small_airport",
                        "SWOB": "small_airport",
                        "SWPG": "small_airport",
                        "SWPI": "small_airport",
                        "SWPM": "small_airport",
                        "SWPQ": "small_airport",
                        "SWR": "small_airport",
                        "SWRA": "small_airport",
                        "SWRD": "small_airport",
                        "SWRP": "small_airport",
                        "SWSI": "small_airport",
                        "SWST": "small_airport",
                        "SWTP": "small_airport",
                        "SWTS": "small_airport",
                        "SWTU": "small_airport",
                        "SWTY": "small_airport",
                        "SWUA": "small_airport",
                        "SWVB": "small_airport",
                        "SWVC": "small_airport",
                        "SWXM": "small_airport",
                        "SWXV": "small_airport",
                        "SXH": "small_airport",
                        "SXP": "small_airport",
                        "SYAH": "small_airport",
                        "SYAN": "small_airport",
                        "SYB": "small_airport",
                        "SYBR": "small_airport",
                        "SYBT": "small_airport",
                        "SYCJ": "medium_airport",
                        "SYF": "seaplane_base",
                        "SYGO": "medium_airport",
                        "SYIB": "small_airport",
                        "SYKM": "small_airport",
                        "SYKR": "small_airport",
                        "SYKS": "small_airport",
                        "SYKT": "small_airport",
                        "SYKZ": "small_airport",
                        "SYL": "heliport",
                        "SYLP": "small_airport",
                        "SYLT": "medium_airport",
                        "SYMB": "small_airport",
                        "SYMD": "small_airport",
                        "SYMK": "small_airport",
                        "SYMM": "small_airport",
                        "SYMR": "small_airport",
                        "SYN": "small_airport",
                        "SYNA": "small_airport",
                        "SYOR": "small_airport",
                        "SYPM": "small_airport",
                        "SYPR": "small_airport",
                        "SYSC": "small_airport",
                        "SYSK": "small_airport",
                        "SZN": "small_airport",
                        "SZP": "small_airport",
                        "TAA": "closed",
                        "TAPA": "medium_airport",
                        "TAPH": "small_airport",
                        "TBA": "closed",
                        "TBE": "small_airport",
                        "TBPB": "medium_airport",
                        "TBQ": "small_airport",
                        "TBV": "small_airport",
                        "TCK": "small_airport",
                        "TCT": "small_airport",
                        "TDB": "small_airport",
                        "TDCF": "medium_airport",
                        "TDPD": "medium_airport",
                        "TDS": "small_airport",
                        "TEO": "small_airport",
                        "TFB": "small_airport",
                        "TFFA": "small_airport",
                        "TFFB": "small_airport",
                        "TFFC": "small_airport",
                        "TFFF": "medium_airport",
                        "TFFG": "medium_airport",
                        "TFFJ": "medium_airport",
                        "TFFM": "medium_airport",
                        "TFFR": "large_airport",
                        "TFFS": "small_airport",
                        "TFY": "small_airport",
                        "TGL": "small_airport",
                        "TGPY": "medium_airport",
                        "TGPZ": "small_airport",
                        "THW": "seaplane_base",
                        "TIG": "closed",
                        "TIST": "medium_airport",
                        "TISX": "medium_airport",
                        "TJAB": "medium_airport",
                        "TJBQ": "medium_airport",
                        "TJC": "small_airport",
                        "TJCG": "closed",
                        "TJCP": "small_airport",
                        "TJFA": "medium_airport",
                        "TJIG": "medium_airport",
                        "TJMZ": "medium_airport",
                        "TJPS": "medium_airport",
                        "TJRV": "small_airport",
                        "TJSJ": "large_airport",
                        "TJVQ": "small_airport",
                        "TKE": "seaplane_base",
                        "TKL": "seaplane_base",
                        "TKPK": "medium_airport",
                        "TKPN": "medium_airport",
                        "TLP": "small_airport",
                        "TLPC": "medium_airport",
                        "TLPL": "medium_airport",
                        "TLT": "small_airport",
                        "TN-0002": "large_airport",
                        "TNCA": "medium_airport",
                        "TNCB": "medium_airport",
                        "TNCC": "medium_airport",
                        "TNCE": "medium_airport",
                        "TNCM": "large_airport",
                        "TNCS": "small_airport",
                        "TNW": "medium_airport",
                        "TOK": "small_airport",
                        "TOV": "seaplane_base",
                        "TPO": "small_airport",
                        "TPT": "small_airport",
                        "TQPF": "medium_airport",
                        "TR-0017": "small_airport",
                        "TR-0025": "small_airport",
                        "TR-0026": "medium_airport",
                        "TRPG": "medium_airport",
                        "TSG": "small_airport",
                        "TSI": "small_airport",
                        "TSK": "closed",
                        "TTCP": "medium_airport",
                        "TTPP": "medium_airport",
                        "TTW": "seaplane_base",
                        "TUE": "small_airport",
                        "TUJ": "small_airport",
                        "TUPA": "small_airport",
                        "TUPJ": "medium_airport",
                        "TUPW": "small_airport",
                        "TUT": "closed",
                        "TUZ": "closed",
                        "TVSA": "medium_airport",
                        "TVSB": "medium_airport",
                        "TVSC": "medium_airport",
                        "TVSM": "medium_airport",
                        "TVSU": "medium_airport",
                        "TVSV": "closed",
                        "TW-0002": "small_airport",
                        "TW-CMJ": "small_airport",
                        "TWH": "seaplane_base",
                        "TXKF": "medium_airport",
                        "TYE": "small_airport",
                        "TZ-GIT": "small_airport",
                        "TZ-LUY": "small_airport",
                        "TZO": "small_airport",
                        "U41": "small_airport",
                        "UA-0049": "small_airport",
                        "UA-0050": "small_airport",
                        "UAAA": "large_airport",
                        "UAAH": "medium_airport",
                        "UAAR": "small_airport",
                        "UACC": "large_airport",
                        "UACK": "medium_airport",
                        "UACP": "medium_airport",
                        "UADD": "medium_airport",
                        "UAE": "small_airport",
                        "UAFL": "medium_airport",
                        "UAFM": "large_airport",
                        "UAFO": "medium_airport",
                        "UAII": "medium_airport",
                        "UAKD": "medium_airport",
                        "UAKK": "large_airport",
                        "UAOO": "small_airport",
                        "UARR": "medium_airport",
                        "UASB": "medium_airport",
                        "UASE": "small_airport",
                        "UASK": "medium_airport",
                        "UASP": "medium_airport",
                        "UASS": "medium_airport",
                        "UATE": "medium_airport",
                        "UATG": "medium_airport",
                        "UATT": "medium_airport",
                        "UAUR": "small_airport",
                        "UAUU": "medium_airport",
                        "UBBB": "large_airport",
                        "UBBG": "medium_airport",
                        "UBBL": "small_airport",
                        "UBBN": "medium_airport",
                        "UBBQ": "medium_airport",
                        "UBBY": "small_airport",
                        "UBEE": "small_airport",
                        "UBI": "small_airport",
                        "UDSG": "medium_airport",
                        "UDYZ": "large_airport",
                        "UEBB": "small_airport",
                        "UEBS": "small_airport",
                        "UEBT": "small_airport",
                        "UECT": "small_airport",
                        "UEEA": "small_airport",
                        "UEEE": "medium_airport",
                        "UELL": "medium_airport",
                        "UEMA": "medium_airport",
                        "UEMM": "medium_airport",
                        "UEMO": "medium_airport",
                        "UEMT": "medium_airport",
                        "UEMU": "small_airport",
                        "UENI": "small_airport",
                        "UENN": "small_airport",
                        "UENS": "small_airport",
                        "UENW": "medium_airport",
                        "UERL": "medium_airport",
                        "UERO": "small_airport",
                        "UERP": "medium_airport",
                        "UERR": "medium_airport",
                        "UERS": "medium_airport",
                        "UESG": "medium_airport",
                        "UESK": "medium_airport",
                        "UESO": "medium_airport",
                        "UESS": "medium_airport",
                        "UEST": "medium_airport",
                        "UESU": "medium_airport",
                        "UG-0002": "small_airport",
                        "UG-OYG": "small_airport",
                        "UGB": "small_airport",
                        "UGKO": "medium_airport",
                        "UGSB": "medium_airport",
                        "UGSS": "medium_airport",
                        "UGTB": "large_airport",
                        "UHBB": "medium_airport",
                        "UHBI": "small_airport",
                        "UHBW": "small_airport",
                        "UHHH": "large_airport",
                        "UHKK": "medium_airport",
                        "UHKM": "medium_airport",
                        "UHMA": "medium_airport",
                        "UHMD": "medium_airport",
                        "UHMK": "medium_airport",
                        "UHMM": "medium_airport",
                        "UHMO": "small_airport",
                        "UHMP": "medium_airport",
                        "UHNB": "small_airport",
                        "UHNN": "medium_airport",
                        "UHOO": "medium_airport",
                        "UHPP": "medium_airport",
                        "UHSB": "small_airport",
                        "UHSH": "small_airport",
                        "UHSI": "small_airport",
                        "UHSK": "small_airport",
                        "UHSM": "small_airport",
                        "UHSO": "small_airport",
                        "UHSS": "medium_airport",
                        "UHTG": "small_airport",
                        "UHTQ": "small_airport",
                        "UHWE": "small_airport",
                        "UHWK": "small_airport",
                        "UHWP": "small_airport",
                        "UHWT": "small_airport",
                        "UHWW": "medium_airport",
                        "UIAA": "medium_airport",
                        "UIBB": "medium_airport",
                        "UIBS": "small_airport",
                        "UIII": "medium_airport",
                        "UIKB": "small_airport",
                        "UIKE": "small_airport",
                        "UIKK": "small_airport",
                        "UITT": "medium_airport",
                        "UIUU": "medium_airport",
                        "UJAP": "small_airport",
                        "UJN": "small_airport",
                        "UKBB": "large_airport",
                        "UKCC": "closed",
                        "UKCK": "small_airport",
                        "UKCM": "medium_airport",
                        "UKCS": "small_airport",
                        "UKCW": "medium_airport",
                        "UKDB": "closed",
                        "UKDD": "medium_airport",
                        "UKDE": "medium_airport",
                        "UKDR": "medium_airport",
                        "UKFB": "medium_airport",
                        "UKFF": "large_airport",
                        "UKFK": "small_airport",
                        "UKH": "small_airport",
                        "UKHH": "large_airport",
                        "UKHP": "small_airport",
                        "UKHS": "small_airport",
                        "UKKE": "medium_airport",
                        "UKKG": "small_airport",
                        "UKKK": "medium_airport",
                        "UKKM": "medium_airport",
                        "UKKV": "small_airport",
                        "UKLC": "small_airport",
                        "UKLH": "medium_airport",
                        "UKLI": "medium_airport",
                        "UKLL": "medium_airport",
                        "UKLN": "medium_airport",
                        "UKLR": "medium_airport",
                        "UKLU": "medium_airport",
                        "UKOH": "small_airport",
                        "UKON": "medium_airport",
                        "UKOO": "large_airport",
                        "UKRR": "closed",
                        "UKWW": "small_airport",
                        "ULAA": "medium_airport",
                        "ULAL": "small_airport",
                        "ULAM": "medium_airport",
                        "ULAS": "small_airport",
                        "ULBC": "medium_airport",
                        "ULDD": "medium_airport",
                        "ULDW": "small_airport",
                        "ULH": "medium_airport",
                        "ULKK": "medium_airport",
                        "ULLI": "large_airport",
                        "ULMK": "small_airport",
                        "ULMM": "medium_airport",
                        "ULNN": "closed",
                        "ULOL": "medium_airport",
                        "ULOO": "medium_airport",
                        "ULPB": "medium_airport",
                        "ULSS": "closed",
                        "ULWW": "medium_airport",
                        "UMBB": "medium_airport",
                        "UMGG": "medium_airport",
                        "UMII": "medium_airport",
                        "UMKK": "medium_airport",
                        "UMMG": "medium_airport",
                        "UMMM": "closed",
                        "UMMS": "large_airport",
                        "UMOO": "medium_airport",
                        "UNAA": "medium_airport",
                        "UNBB": "medium_airport",
                        "UNBG": "medium_airport",
                        "UNEE": "medium_airport",
                        "UNII": "medium_airport",
                        "UNIP": "medium_airport",
                        "UNKL": "large_airport",
                        "UNKS": "medium_airport",
                        "UNKY": "medium_airport",
                        "UNNT": "large_airport",
                        "UNOO": "medium_airport",
                        "UNSS": "medium_airport",
                        "UNTT": "medium_airport",
                        "UNWW": "medium_airport",
                        "UODD": "medium_airport",
                        "UOHH": "medium_airport",
                        "UOII": "medium_airport",
                        "UOOO": "medium_airport",
                        "UOTT": "small_airport",
                        "URB": "closed",
                        "URKA": "medium_airport",
                        "URKE": "medium_airport",
                        "URKG": "medium_airport",
                        "URKK": "medium_airport",
                        "URML": "medium_airport",
                        "URMM": "medium_airport",
                        "URMN": "medium_airport",
                        "URMO": "medium_airport",
                        "URMS": "medium_airport",
                        "URMT": "medium_airport",
                        "URRP": "large_airport",
                        "URRR": "closed",
                        "URRT": "medium_airport",
                        "URRY": "closed",
                        "URSS": "large_airport",
                        "URWA": "medium_airport",
                        "URWI": "medium_airport",
                        "URWW": "medium_airport",
                        "US-0099": "closed",
                        "US-0133": "closed",
                        "US-0370": "seaplane_base",
                        "US-0571": "medium_airport",
                        "US-0F3": "small_airport",
                        "US-BPA": "closed",
                        "USCC": "medium_airport",
                        "USCM": "medium_airport",
                        "USDA": "medium_airport",
                        "USDD": "medium_airport",
                        "USDK": "small_airport",
                        "USDP": "small_airport",
                        "USDS": "small_airport",
                        "USDU": "small_airport",
                        "USHB": "medium_airport",
                        "USHH": "medium_airport",
                        "USHI": "small_airport",
                        "USHN": "medium_airport",
                        "USHS": "medium_airport",
                        "USHU": "medium_airport",
                        "USHY": "medium_airport",
                        "USII": "medium_airport",
                        "USKK": "medium_airport",
                        "USMM": "medium_airport",
                        "USMU": "medium_airport",
                        "USNN": "medium_airport",
                        "USNR": "closed",
                        "USO": "closed",
                        "USPP": "medium_airport",
                        "USRK": "medium_airport",
                        "USRN": "medium_airport",
                        "USRO": "medium_airport",
                        "USRR": "medium_airport",
                        "USSS": "large_airport",
                        "USTO": "small_airport",
                        "USTR": "medium_airport",
                        "USUU": "medium_airport",
                        "UT1H": "small_airport",
                        "UT25": "small_airport",
                        "UTAA": "large_airport",
                        "UTAK": "medium_airport",
                        "UTAM": "medium_airport",
                        "UTAT": "medium_airport",
                        "UTAV": "medium_airport",
                        "UTDD": "medium_airport",
                        "UTDK": "medium_airport",
                        "UTDL": "medium_airport",
                        "UTDT": "medium_airport",
                        "UTKA": "small_airport",
                        "UTKF": "small_airport",
                        "UTKN": "small_airport",
                        "UTNN": "small_airport",
                        "UTNU": "medium_airport",
                        "UTSA": "small_airport",
                        "UTSB": "medium_airport",
                        "UTSL": "small_airport",
                        "UTSN": "small_airport",
                        "UTSS": "medium_airport",
                        "UTST": "medium_airport",
                        "UTTT": "large_airport",
                        "UTU": "small_airport",
                        "UUBA": "medium_airport",
                        "UUBB": "closed",
                        "UUBC": "medium_airport",
                        "UUBI": "medium_airport",
                        "UUBK": "medium_airport",
                        "UUBP": "medium_airport",
                        "UUBW": "large_airport",
                        "UUDD": "large_airport",
                        "UUDL": "small_airport",
                        "UUEE": "large_airport",
                        "UUEM": "medium_airport",
                        "UUMO": "medium_airport",
                        "UUMU": "medium_airport",
                        "UUOB": "medium_airport",
                        "UUOK": "medium_airport",
                        "UUOL": "medium_airport",
                        "UUOO": "medium_airport",
                        "UUOR": "medium_airport",
                        "UUOT": "medium_airport",
                        "UUU": "small_airport",
                        "UUWR": "small_airport",
                        "UUWV": "small_airport",
                        "UUWW": "large_airport",
                        "UUYH": "medium_airport",
                        "UUYI": "medium_airport",
                        "UUYP": "medium_airport",
                        "UUYS": "medium_airport",
                        "UUYW": "medium_airport",
                        "UUYX": "medium_airport",
                        "UUYY": "medium_airport",
                        "UWGG": "medium_airport",
                        "UWKB": "medium_airport",
                        "UWKD": "large_airport",
                        "UWKE": "medium_airport",
                        "UWKJ": "medium_airport",
                        "UWKS": "medium_airport",
                        "UWKV": "medium_airport",
                        "UWLL": "medium_airport",
                        "UWLW": "medium_airport",
                        "UWOO": "medium_airport",
                        "UWOR": "medium_airport",
                        "UWPP": "medium_airport",
                        "UWPS": "medium_airport",
                        "UWSB": "medium_airport",
                        "UWSS": "medium_airport",
                        "UWUB": "small_airport",
                        "UWUF": "small_airport",
                        "UWUK": "small_airport",
                        "UWUU": "large_airport",
                        "UWWW": "large_airport",
                        "UZM": "small_airport",
                        "UZR": "small_airport",
                        "VA1G": "medium_airport",
                        "VA1P": "small_airport",
                        "VAAH": "medium_airport",
                        "VAAK": "medium_airport",
                        "VAAU": "medium_airport",
                        "VABB": "large_airport",
                        "VABI": "medium_airport",
                        "VABJ": "medium_airport",
                        "VABM": "medium_airport",
                        "VABO": "medium_airport",
                        "VABP": "medium_airport",
                        "VABV": "medium_airport",
                        "VADN": "medium_airport",
                        "VAGN": "small_airport",
                        "VAGO": "large_airport",
                        "VAHB": "medium_airport",
                        "VAID": "medium_airport",
                        "VAJB": "medium_airport",
                        "VAJM": "medium_airport",
                        "VAKE": "medium_airport",
                        "VAKJ": "medium_airport",
                        "VAKP": "medium_airport",
                        "VAKS": "medium_airport",
                        "VAND": "medium_airport",
                        "VANP": "medium_airport",
                        "VAOZ": "medium_airport",
                        "VAPO": "medium_airport",
                        "VAPR": "medium_airport",
                        "VARG": "medium_airport",
                        "VARK": "medium_airport",
                        "VARP": "small_airport",
                        "VASD": "medium_airport",
                        "VASL": "medium_airport",
                        "VASU": "medium_airport",
                        "VAUD": "medium_airport",
                        "VCBI": "large_airport",
                        "VCCA": "medium_airport",
                        "VCCB": "medium_airport",
                        "VCCC": "medium_airport",
                        "VCCG": "medium_airport",
                        "VCCH": "medium_airport",
                        "VCCJ": "medium_airport",
                        "VCCK": "medium_airport",
                        "VCCN": "small_airport",
                        "VCCS": "small_airport",
                        "VCCT": "medium_airport",
                        "VCCW": "small_airport",
                        "VCRI": "large_airport",
                        "VDBG": "medium_airport",
                        "VDKH": "medium_airport",
                        "VDKK": "small_airport",
                        "VDKT": "small_airport",
                        "VDMK": "small_airport",
                        "VDPP": "large_airport",
                        "VDRK": "medium_airport",
                        "VDSR": "large_airport",
                        "VDST": "medium_airport",
                        "VDSV": "small_airport",
                        "VDSY": "small_airport",
                        "VE-0095": "heliport",
                        "VE-0114": "small_airport",
                        "VE-0204": "small_airport",
                        "VE-0205": "small_airport",
                        "VE-0206": "small_airport",
                        "VE-0207": "small_airport",
                        "VE-0208": "small_airport",
                        "VE-0209": "small_airport",
                        "VEAN": "medium_airport",
                        "VEAT": "medium_airport",
                        "VEBD": "medium_airport",
                        "VEBG": "small_airport",
                        "VEBI": "medium_airport",
                        "VEBS": "medium_airport",
                        "VECC": "large_airport",
                        "VECO": "small_airport",
                        "VEDB": "medium_airport",
                        "VEDG": "medium_airport",
                        "VEDZ": "small_airport",
                        "VEGK": "medium_airport",
                        "VEGT": "medium_airport",
                        "VEGY": "medium_airport",
                        "VEIM": "medium_airport",
                        "VEJH": "small_airport",
                        "VEJP": "small_airport",
                        "VEJS": "medium_airport",
                        "VEJT": "medium_airport",
                        "VEKM": "small_airport",
                        "VEKR": "medium_airport",
                        "VEKU": "medium_airport",
                        "VEKW": "small_airport",
                        "VELP": "medium_airport",
                        "VELR": "medium_airport",
                        "VEMH": "small_airport",
                        "VEMN": "medium_airport",
                        "VEMR": "medium_airport",
                        "VEMZ": "medium_airport",
                        "VEPG": "small_airport",
                        "VEPT": "medium_airport",
                        "VERC": "medium_airport",
                        "VERK": "medium_airport",
                        "VERU": "small_airport",
                        "VETZ": "medium_airport",
                        "VEVZ": "small_airport",
                        "VEZO": "medium_airport",
                        "VGBR": "medium_airport",
                        "VGCB": "medium_airport",
                        "VGCM": "small_airport",
                        "VGEG": "medium_airport",
                        "VGIS": "medium_airport",
                        "VGJR": "medium_airport",
                        "VGLM": "small_airport",
                        "VGRJ": "medium_airport",
                        "VGSD": "medium_airport",
                        "VGSG": "small_airport",
                        "VGSH": "small_airport",
                        "VGSY": "medium_airport",
                        "VGZR": "large_airport",
                        "VHHH": "large_airport",
                        "VHHX": "closed",
                        "VHST": "heliport",
                        "VI22": "seaplane_base",
                        "VI32": "seaplane_base",
                        "VIAG": "medium_airport",
                        "VIAL": "medium_airport",
                        "VIAR": "large_airport",
                        "VIAX": "small_airport",
                        "VIB": "closed",
                        "VIBK": "medium_airport",
                        "VIBN": "medium_airport",
                        "VIBR": "medium_airport",
                        "VIBT": "medium_airport",
                        "VIBY": "medium_airport",
                        "VICG": "medium_airport",
                        "VICX": "medium_airport",
                        "VIDN": "medium_airport",
                        "VIDP": "large_airport",
                        "VIF": "heliport",
                        "VIGG": "medium_airport",
                        "VIGR": "medium_airport",
                        "VIHR": "medium_airport",
                        "VIJO": "medium_airport",
                        "VIJP": "medium_airport",
                        "VIJR": "medium_airport",
                        "VIJU": "medium_airport",
                        "VIKG": "medium_airport",
                        "VIKO": "medium_airport",
                        "VILD": "medium_airport",
                        "VILH": "medium_airport",
                        "VILK": "medium_airport",
                        "VIPK": "medium_airport",
                        "VIPT": "medium_airport",
                        "VISM": "small_airport",
                        "VISR": "medium_airport",
                        "VIST": "small_airport",
                        "VIV": "small_airport",
                        "VJQ": "small_airport",
                        "VLAP": "small_airport",
                        "VLHS": "small_airport",
                        "VLLB": "medium_airport",
                        "VLLN": "small_airport",
                        "VLOS": "small_airport",
                        "VLPS": "medium_airport",
                        "VLSB": "small_airport",
                        "VLSK": "medium_airport",
                        "VLSN": "medium_airport",
                        "VLSV": "small_airport",
                        "VLTK": "small_airport",
                        "VLVT": "medium_airport",
                        "VLXK": "small_airport",
                        "VLXL": "small_airport",
                        "VMI": "small_airport",
                        "VMMC": "large_airport",
                        "VN-0001": "medium_airport",
                        "VN-KON": "medium_airport",
                        "VNBG": "small_airport",
                        "VNBJ": "small_airport",
                        "VNBL": "small_airport",
                        "VNBP": "small_airport",
                        "VNBR": "small_airport",
                        "VNBT": "small_airport",
                        "VNBW": "medium_airport",
                        "VNCG": "small_airport",
                        "VNDG": "small_airport",
                        "VNDH": "small_airport",
                        "VNDL": "small_airport",
                        "VNDP": "small_airport",
                        "VNDT": "small_airport",
                        "VNGK": "small_airport",
                        "VNJI": "small_airport",
                        "VNJL": "small_airport",
                        "VNJP": "medium_airport",
                        "VNJS": "small_airport",
                        "VNKT": "large_airport",
                        "VNLD": "small_airport",
                        "VNLK": "small_airport",
                        "VNLT": "small_airport",
                        "VNMA": "small_airport",
                        "VNMG": "small_airport",
                        "VNMN": "small_airport",
                        "VNNG": "medium_airport",
                        "VNPK": "medium_airport",
                        "VNPL": "small_airport",
                        "VNRB": "small_airport",
                        "VNRC": "small_airport",
                        "VNRK": "small_airport",
                        "VNRP": "small_airport",
                        "VNRT": "small_airport",
                        "VNSB": "small_airport",
                        "VNSI": "small_airport",
                        "VNSK": "small_airport",
                        "VNSR": "small_airport",
                        "VNST": "small_airport",
                        "VNTJ": "medium_airport",
                        "VNTP": "small_airport",
                        "VNTR": "small_airport",
                        "VNVT": "medium_airport",
                        "VO55": "medium_airport",
                        "VO80": "small_airport",
                        "VOAT": "medium_airport",
                        "VOBI": "medium_airport",
                        "VOBL": "large_airport",
                        "VOBR": "medium_airport",
                        "VOBZ": "medium_airport",
                        "VOCB": "medium_airport",
                        "VOCI": "large_airport",
                        "VOCL": "large_airport",
                        "VOCP": "medium_airport",
                        "VOCX": "medium_airport",
                        "VOHS": "large_airport",
                        "VOHY": "medium_airport",
                        "VOKN": "medium_airport",
                        "VOMD": "medium_airport",
                        "VOML": "medium_airport",
                        "VOMM": "large_airport",
                        "VOMY": "small_airport",
                        "VOPB": "medium_airport",
                        "VOPC": "medium_airport",
                        "VOPN": "medium_airport",
                        "VORG": "small_airport",
                        "VORY": "medium_airport",
                        "VOSM": "medium_airport",
                        "VOTJ": "small_airport",
                        "VOTP": "small_airport",
                        "VOTR": "medium_airport",
                        "VOTV": "large_airport",
                        "VOWA": "small_airport",
                        "VQ10": "small_airport",
                        "VQBT": "small_airport",
                        "VQGP": "medium_airport",
                        "VQPR": "medium_airport",
                        "VREI": "small_airport",
                        "VRMD": "small_airport",
                        "VRMF": "small_airport",
                        "VRMG": "medium_airport",
                        "VRMH": "medium_airport",
                        "VRMK": "medium_airport",
                        "VRMM": "large_airport",
                        "VRMO": "small_airport",
                        "VRMT": "medium_airport",
                        "VRMU": "small_airport",
                        "VRMV": "medium_airport",
                        "VRNT": "small_airport",
                        "VTBD": "large_airport",
                        "VTBH": "small_airport",
                        "VTBK": "medium_airport",
                        "VTBO": "medium_airport",
                        "VTBS": "large_airport",
                        "VTBU": "medium_airport",
                        "VTCC": "large_airport",
                        "VTCH": "medium_airport",
                        "VTCI": "small_airport",
                        "VTCL": "medium_airport",
                        "VTCN": "medium_airport",
                        "VTCP": "medium_airport",
                        "VTCT": "medium_airport",
                        "VTED": "small_airport",
                        "VTPB": "medium_airport",
                        "VTPH": "medium_airport",
                        "VTPI": "medium_airport",
                        "VTPM": "medium_airport",
                        "VTPO": "medium_airport",
                        "VTPP": "medium_airport",
                        "VTPT": "medium_airport",
                        "VTPU": "small_airport",
                        "VTSB": "medium_airport",
                        "VTSC": "medium_airport",
                        "VTSE": "medium_airport",
                        "VTSF": "medium_airport",
                        "VTSG": "medium_airport",
                        "VTSH": "medium_airport",
                        "VTSK": "medium_airport",
                        "VTSM": "medium_airport",
                        "VTSP": "large_airport",
                        "VTSR": "medium_airport",
                        "VTSS": "medium_airport",
                        "VTST": "medium_airport",
                        "VTUD": "medium_airport",
                        "VTUI": "medium_airport",
                        "VTUJ": "medium_airport",
                        "VTUK": "medium_airport",
                        "VTUL": "medium_airport",
                        "VTUO": "medium_airport",
                        "VTUQ": "medium_airport",
                        "VTUU": "medium_airport",
                        "VTUV": "medium_airport",
                        "VTUW": "medium_airport",
                        "VUU": "small_airport",
                        "VVBM": "medium_airport",
                        "VVCA": "small_airport",
                        "VVCI": "medium_airport",
                        "VVCM": "medium_airport",
                        "VVCR": "medium_airport",
                        "VVCS": "medium_airport",
                        "VVCT": "medium_airport",
                        "VVDB": "medium_airport",
                        "VVDL": "medium_airport",
                        "VVDN": "large_airport",
                        "VVN": "small_airport",
                        "VVNB": "large_airport",
                        "VVNS": "small_airport",
                        "VVNT": "closed",
                        "VVPB": "medium_airport",
                        "VVPC": "medium_airport",
                        "VVPK": "medium_airport",
                        "VVPQ": "medium_airport",
                        "VVPR": "small_airport",
                        "VVPT": "small_airport",
                        "VVRG": "medium_airport",
                        "VVTH": "medium_airport",
                        "VVTS": "large_airport",
                        "VVTX": "small_airport",
                        "VVVD": "medium_airport",
                        "VVVH": "medium_airport",
                        "VVVT": "small_airport",
                        "VYAN": "small_airport",
                        "VYBG": "small_airport",
                        "VYBM": "small_airport",
                        "VYBP": "small_airport",
                        "VYDW": "medium_airport",
                        "VYEL": "medium_airport",
                        "VYGG": "small_airport",
                        "VYGW": "small_airport",
                        "VYHH": "medium_airport",
                        "VYHL": "small_airport",
                        "VYHN": "small_airport",
                        "VYKG": "medium_airport",
                        "VYKI": "small_airport",
                        "VYKL": "small_airport",
                        "VYKP": "medium_airport",
                        "VYKT": "medium_airport",
                        "VYKU": "small_airport",
                        "VYLK": "medium_airport",
                        "VYLS": "medium_airport",
                        "VYMD": "large_airport",
                        "VYME": "medium_airport",
                        "VYMK": "medium_airport",
                        "VYMM": "small_airport",
                        "VYMN": "small_airport",
                        "VYMO": "medium_airport",
                        "VYMS": "medium_airport",
                        "VYMT": "small_airport",
                        "VYMW": "small_airport",
                        "VYMY": "small_airport",
                        "VYNS": "medium_airport",
                        "VYNT": "small_airport",
                        "VYPA": "small_airport",
                        "VYPK": "small_airport",
                        "VYPN": "small_airport",
                        "VYPP": "small_airport",
                        "VYPT": "medium_airport",
                        "VYPU": "small_airport",
                        "VYPY": "small_airport",
                        "VYSW": "medium_airport",
                        "VYTD": "medium_airport",
                        "VYTL": "medium_airport",
                        "VYYE": "small_airport",
                        "VYYY": "large_airport",
                        "W33": "seaplane_base",
                        "W49": "seaplane_base",
                        "W55": "seaplane_base",
                        "WA09": "small_airport",
                        "WA44": "small_airport",
                        "WA83": "seaplane_base",
                        "WAAA": "large_airport",
                        "WAAJ": "small_airport",
                        "WAB": "closed",
                        "WABB": "medium_airport",
                        "WABD": "small_airport",
                        "WABG": "small_airport",
                        "WABI": "medium_airport",
                        "WABL": "small_airport",
                        "WABN": "small_airport",
                        "WABO": "small_airport",
                        "WABP": "medium_airport",
                        "WABT": "small_airport",
                        "WAD": "small_airport",
                        "WADA": "small_airport",
                        "WADB": "small_airport",
                        "WADD": "large_airport",
                        "WADL": "medium_airport",
                        "WADS": "small_airport",
                        "WADT": "small_airport",
                        "WADW": "small_airport",
                        "WAFD": "small_airport",
                        "WAHI": "medium_airport",
                        "WAJA": "small_airport",
                        "WAJB": "small_airport",
                        "WAJI": "small_airport",
                        "WAJJ": "large_airport",
                        "WAJL": "small_airport",
                        "WAJM": "small_airport",
                        "WAJO": "small_airport",
                        "WAJR": "small_airport",
                        "WAJS": "small_airport",
                        "WAJU": "small_airport",
                        "WAJW": "medium_airport",
                        "WAKD": "small_airport",
                        "WAKE": "small_airport",
                        "WAKK": "medium_airport",
                        "WAKO": "small_airport",
                        "WAKP": "small_airport",
                        "WAKT": "small_airport",
                        "WALG": "small_airport",
                        "WALJ": "small_airport",
                        "WALK": "medium_airport",
                        "WALL": "small_airport",
                        "WALR": "medium_airport",
                        "WALT": "small_airport",
                        "WALV": "small_airport",
                        "WAMA": "small_airport",
                        "WAME": "small_airport",
                        "WAMG": "small_airport",
                        "WAMH": "medium_airport",
                        "WAMI": "small_airport",
                        "WAMJ": "small_airport",
                        "WAMK": "small_airport",
                        "WAML": "medium_airport",
                        "WAMM": "medium_airport",
                        "WAMN": "small_airport",
                        "WAMP": "medium_airport",
                        "WAMR": "medium_airport",
                        "WAMT": "medium_airport",
                        "WAMW": "medium_airport",
                        "WAMY": "small_airport",
                        "WAN": "small_airport",
                        "WAOC": "small_airport",
                        "WAOI": "small_airport",
                        "WAOK": "small_airport",
                        "WAON": "medium_airport",
                        "WAOO": "small_airport",
                        "WAOP": "small_airport",
                        "WAOS": "small_airport",
                        "WAPA": "small_airport",
                        "WAPC": "small_airport",
                        "WAPD": "small_airport",
                        "WAPE": "small_airport",
                        "WAPF": "medium_airport",
                        "WAPG": "small_airport",
                        "WAPH": "small_airport",
                        "WAPI": "small_airport",
                        "WAPK": "small_airport",
                        "WAPN": "small_airport",
                        "WAPP": "medium_airport",
                        "WAPR": "small_airport",
                        "WAPT": "small_airport",
                        "WAPV": "small_airport",
                        "WARA": "small_airport",
                        "WARC": "small_airport",
                        "WARJ": "small_airport",
                        "WARQ": "small_airport",
                        "WARR": "large_airport",
                        "WARS": "small_airport",
                        "WART": "small_airport",
                        "WARU": "small_airport",
                        "WASB": "small_airport",
                        "WASC": "small_airport",
                        "WASE": "small_airport",
                        "WASF": "medium_airport",
                        "WASI": "small_airport",
                        "WASK": "medium_airport",
                        "WASM": "small_airport",
                        "WASN": "small_airport",
                        "WASO": "medium_airport",
                        "WASR": "medium_airport",
                        "WAST": "small_airport",
                        "WASW": "small_airport",
                        "WATB": "small_airport",
                        "WATC": "small_airport",
                        "WATE": "small_airport",
                        "WATG": "small_airport",
                        "WATM": "small_airport",
                        "WATO": "small_airport",
                        "WATS": "small_airport",
                        "WATT": "small_airport",
                        "WAWB": "small_airport",
                        "WAWD": "medium_airport",
                        "WAWH": "small_airport",
                        "WAWM": "small_airport",
                        "WAWS": "small_airport",
                        "WAWT": "small_airport",
                        "WAWW": "small_airport",
                        "WAXX": "large_airport",
                        "WBB": "small_airport",
                        "WBC": "small_airport",
                        "WBGB": "medium_airport",
                        "WBGC": "small_airport",
                        "WBGD": "small_airport",
                        "WBGF": "small_airport",
                        "WBGG": "medium_airport",
                        "WBGI": "small_airport",
                        "WBGJ": "medium_airport",
                        "WBGK": "small_airport",
                        "WBGL": "small_airport",
                        "WBGM": "medium_airport",
                        "WBGN": "small_airport",
                        "WBGP": "small_airport",
                        "WBGQ": "small_airport",
                        "WBGR": "medium_airport",
                        "WBGS": "medium_airport",
                        "WBGT": "small_airport",
                        "WBGU": "small_airport",
                        "WBGW": "small_airport",
                        "WBGY": "closed",
                        "WBGZ": "medium_airport",
                        "WBKA": "small_airport",
                        "WBKD": "medium_airport",
                        "WBKE": "small_airport",
                        "WBKG": "small_airport",
                        "WBKH": "small_airport",
                        "WBKK": "medium_airport",
                        "WBKL": "medium_airport",
                        "WBKM": "small_airport",
                        "WBKN": "small_airport",
                        "WBKO": "small_airport",
                        "WBKP": "small_airport",
                        "WBKR": "small_airport",
                        "WBKS": "medium_airport",
                        "WBKT": "small_airport",
                        "WBKW": "medium_airport",
                        "WBMU": "medium_airport",
                        "WBSB": "large_airport",
                        "WEA": "small_airport",
                        "WED": "small_airport",
                        "WGU": "closed",
                        "WGY": "closed",
                        "WHL": "small_airport",
                        "WI1A": "small_airport",
                        "WIBB": "medium_airport",
                        "WIBD": "medium_airport",
                        "WIBR": "small_airport",
                        "WIBS": "small_airport",
                        "WIBT": "small_airport",
                        "WICA": "medium_airport",
                        "WICC": "medium_airport",
                        "WICD": "small_airport",
                        "WICM": "small_airport",
                        "WID": "closed",
                        "WIDD": "medium_airport",
                        "WIDE": "small_airport",
                        "WIDL": "small_airport",
                        "WIDN": "small_airport",
                        "WIDS": "small_airport",
                        "WIHH": "medium_airport",
                        "WIHL": "medium_airport",
                        "WIHP": "small_airport",
                        "WIII": "large_airport",
                        "WIKL": "small_airport",
                        "WILL": "medium_airport",
                        "WILP": "small_airport",
                        "WIMB": "medium_airport",
                        "WIME": "medium_airport",
                        "WIMK": "medium_airport",
                        "WIMM": "large_airport",
                        "WIMN": "small_airport",
                        "WIMP": "small_airport",
                        "WIMS": "small_airport",
                        "WIOD": "small_airport",
                        "WIOG": "medium_airport",
                        "WIOK": "medium_airport",
                        "WIOM": "small_airport",
                        "WION": "medium_airport",
                        "WIOO": "medium_airport",
                        "WIOP": "medium_airport",
                        "WIOS": "medium_airport",
                        "WIPA": "small_airport",
                        "WIPI": "small_airport",
                        "WIPK": "small_airport",
                        "WIPL": "medium_airport",
                        "WIPP": "medium_airport",
                        "WIPQ": "medium_airport",
                        "WIPR": "medium_airport",
                        "WIPT": "medium_airport",
                        "WIPU": "small_airport",
                        "WIPV": "small_airport",
                        "WIPY": "small_airport",
                        "WITA": "small_airport",
                        "WITB": "small_airport",
                        "WITC": "small_airport",
                        "WITK": "medium_airport",
                        "WITL": "medium_airport",
                        "WITM": "small_airport",
                        "WITT": "medium_airport",
                        "WJBK": "closed",
                        "WMAN": "small_airport",
                        "WMAU": "small_airport",
                        "WMBA": "small_airport",
                        "WMBI": "small_airport",
                        "WMBT": "medium_airport",
                        "WMKA": "medium_airport",
                        "WMKB": "medium_airport",
                        "WMKC": "medium_airport",
                        "WMKD": "medium_airport",
                        "WMKE": "medium_airport",
                        "WMKI": "medium_airport",
                        "WMKJ": "medium_airport",
                        "WMKK": "large_airport",
                        "WMKL": "medium_airport",
                        "WMKM": "medium_airport",
                        "WMKN": "medium_airport",
                        "WMKP": "medium_airport",
                        "WMPA": "small_airport",
                        "WMPR": "small_airport",
                        "WMSA": "medium_airport",
                        "WN07": "small_airport",
                        "WNU": "small_airport",
                        "WPAT": "small_airport",
                        "WPDB": "medium_airport",
                        "WPDL": "medium_airport",
                        "WPEC": "medium_airport",
                        "WPMN": "small_airport",
                        "WPOC": "medium_airport",
                        "WPVQ": "small_airport",
                        "WRKA": "small_airport",
                        "WRKL": "small_airport",
                        "WRKR": "small_airport",
                        "WRLA": "small_airport",
                        "WRLB": "small_airport",
                        "WRLC": "small_airport",
                        "WRLF": "small_airport",
                        "WRLH": "small_airport",
                        "WRLP": "small_airport",
                        "WRLY": "heliport",
                        "WSA": "small_airport",
                        "WSAP": "medium_airport",
                        "WSAT": "medium_airport",
                        "WSB": "seaplane_base",
                        "WSJ": "seaplane_base",
                        "WSM": "small_airport",
                        "WSSL": "medium_airport",
                        "WSSS": "large_airport",
                        "WTT": "small_airport",
                        "WUV": "small_airport",
                        "WV66": "small_airport",
                        "WZQ": "small_airport",
                        "X-TRPM": "closed",
                        "X44": "seaplane_base",
                        "XBB": "seaplane_base",
                        "XBN": "small_airport",
                        "XIG": "small_airport",
                        "XLO": "closed",
                        "XMA": "small_airport",
                        "XMUC": "closed",
                        "XNG": "closed",
                        "XRQ": "small_airport",
                        "XUBS": "medium_airport",
                        "XVL": "small_airport",
                        "XXZ": "medium_airport",
                        "XZM": "heliport",
                        "Y01": "small_airport",
                        "YABA": "medium_airport",
                        "YABI": "small_airport",
                        "YADS": "small_airport",
                        "YAGD": "small_airport",
                        "YAJ": "seaplane_base",
                        "YALA": "small_airport",
                        "YALX": "small_airport",
                        "YAMC": "small_airport",
                        "YAMK": "small_airport",
                        "YAMM": "small_airport",
                        "YAMT": "small_airport",
                        "YANG": "small_airport",
                        "YANL": "small_airport",
                        "YAPH": "medium_airport",
                        "YAQ": "seaplane_base",
                        "YARA": "medium_airport",
                        "YARG": "small_airport",
                        "YARM": "medium_airport",
                        "YARY": "small_airport",
                        "YAUR": "small_airport",
                        "YAUS": "small_airport",
                        "YAUV": "small_airport",
                        "YAYE": "medium_airport",
                        "YAYR": "small_airport",
                        "YBAF": "small_airport",
                        "YBAM": "small_airport",
                        "YBAR": "medium_airport",
                        "YBAS": "medium_airport",
                        "YBAU": "small_airport",
                        "YBAW": "small_airport",
                        "YBBE": "small_airport",
                        "YBBN": "large_airport",
                        "YBCG": "medium_airport",
                        "YBCK": "medium_airport",
                        "YBCS": "medium_airport",
                        "YBCV": "medium_airport",
                        "YBDF": "small_airport",
                        "YBDG": "small_airport",
                        "YBDV": "medium_airport",
                        "YBEB": "small_airport",
                        "YBEO": "small_airport",
                        "YBGB": "small_airport",
                        "YBGD": "small_airport",
                        "YBGO": "small_airport",
                        "YBH": "seaplane_base",
                        "YBHI": "medium_airport",
                        "YBHM": "medium_airport",
                        "YBIE": "medium_airport",
                        "YBIL": "small_airport",
                        "YBIZ": "small_airport",
                        "YBJ": "seaplane_base",
                        "YBKE": "medium_airport",
                        "YBKT": "small_airport",
                        "YBLA": "medium_airport",
                        "YBLC": "small_airport",
                        "YBLL": "small_airport",
                        "YBLN": "small_airport",
                        "YBMA": "medium_airport",
                        "YBMC": "medium_airport",
                        "YBMD": "small_airport",
                        "YBMK": "medium_airport",
                        "YBNA": "medium_airport",
                        "YBNS": "medium_airport",
                        "YBOI": "small_airport",
                        "YBOK": "medium_airport",
                        "YBOU": "medium_airport",
                        "YBPI": "small_airport",
                        "YBPN": "medium_airport",
                        "YBQ": "seaplane_base",
                        "YBRK": "medium_airport",
                        "YBRL": "small_airport",
                        "YBRM": "medium_airport",
                        "YBRN": "medium_airport",
                        "YBRU": "small_airport",
                        "YBRW": "medium_airport",
                        "YBRY": "small_airport",
                        "YBTH": "medium_airport",
                        "YBTI": "medium_airport",
                        "YBTL": "medium_airport",
                        "YBTR": "medium_airport",
                        "YBTV": "small_airport",
                        "YBUD": "medium_airport",
                        "YBUN": "small_airport",
                        "YBWM": "small_airport",
                        "YBWN": "small_airport",
                        "YBWP": "medium_airport",
                        "YBWR": "small_airport",
                        "YBWW": "medium_airport",
                        "YBWX": "small_airport",
                        "YBYS": "small_airport",
                        "YCAC": "small_airport",
                        "YCAG": "small_airport",
                        "YCAH": "small_airport",
                        "YCAR": "medium_airport",
                        "YCAS": "small_airport",
                        "YCBA": "medium_airport",
                        "YCBB": "medium_airport",
                        "YCBE": "small_airport",
                        "YCBN": "small_airport",
                        "YCBP": "medium_airport",
                        "YCBR": "small_airport",
                        "YCCA": "medium_airport",
                        "YCCT": "small_airport",
                        "YCCY": "medium_airport",
                        "YCDO": "small_airport",
                        "YCDR": "small_airport",
                        "YCDU": "medium_airport",
                        "YCEE": "medium_airport",
                        "YCF": "seaplane_base",
                        "YCFD": "small_airport",
                        "YCFH": "small_airport",
                        "YCFL": "small_airport",
                        "YCGO": "small_airport",
                        "YCHB": "small_airport",
                        "YCHK": "small_airport",
                        "YCHT": "small_airport",
                        "YCIN": "medium_airport",
                        "YCKI": "small_airport",
                        "YCKN": "medium_airport",
                        "YCMT": "medium_airport",
                        "YCMU": "medium_airport",
                        "YCMW": "small_airport",
                        "YCNF": "small_airport",
                        "YCNK": "small_airport",
                        "YCNM": "medium_airport",
                        "YCOD": "small_airport",
                        "YCOE": "medium_airport",
                        "YCOI": "small_airport",
                        "YCOM": "medium_airport",
                        "YCOO": "small_airport",
                        "YCOR": "medium_airport",
                        "YCPN": "small_airport",
                        "YCRG": "medium_airport",
                        "YCRK": "small_airport",
                        "YCRY": "small_airport",
                        "YCSV": "small_airport",
                        "YCTM": "medium_airport",
                        "YCUA": "small_airport",
                        "YCUE": "small_airport",
                        "YCWA": "small_airport",
                        "YCWI": "small_airport",
                        "YCWL": "small_airport",
                        "YCWR": "medium_airport",
                        "YCWY": "small_airport",
                        "YDAJ": "small_airport",
                        "YDAY": "small_airport",
                        "YDBI": "medium_airport",
                        "YDBR": "small_airport",
                        "YDBY": "medium_airport",
                        "YDDF": "small_airport",
                        "YDGA": "small_airport",
                        "YDGN": "small_airport",
                        "YDIX": "small_airport",
                        "YDKI": "small_airport",
                        "YDLK": "small_airport",
                        "YDLQ": "medium_airport",
                        "YDLT": "small_airport",
                        "YDLV": "small_airport",
                        "YDLW": "small_airport",
                        "YDMG": "small_airport",
                        "YDMN": "small_airport",
                        "YDNI": "small_airport",
                        "YDOR": "small_airport",
                        "YDPD": "small_airport",
                        "YDPO": "medium_airport",
                        "YDRA": "small_airport",
                        "YDRD": "small_airport",
                        "YDRH": "small_airport",
                        "YDRI": "small_airport",
                        "YDUN": "small_airport",
                        "YDVR": "small_airport",
                        "YDYS": "medium_airport",
                        "YE-WDA": "small_airport",
                        "YECH": "medium_airport",
                        "YECL": "small_airport",
                        "YEDA": "small_airport",
                        "YEEB": "small_airport",
                        "YEIN": "small_airport",
                        "YELD": "medium_airport",
                        "YELK": "small_airport",
                        "YEML": "medium_airport",
                        "YEQ": "small_airport",
                        "YERL": "small_airport",
                        "YERN": "small_airport",
                        "YESE": "small_airport",
                        "YESP": "medium_airport",
                        "YEVA": "small_airport",
                        "YEVD": "small_airport",
                        "YEXM": "small_airport",
                        "YFBS": "medium_airport",
                        "YFDF": "small_airport",
                        "YFIL": "small_airport",
                        "YFLI": "small_airport",
                        "YFLO": "small_airport",
                        "YFNE": "small_airport",
                        "YFRT": "medium_airport",
                        "YFRV": "small_airport",
                        "YFST": "small_airport",
                        "YFTZ": "medium_airport",
                        "YGAM": "small_airport",
                        "YGAY": "small_airport",
                        "YGBI": "small_airport",
                        "YGDH": "medium_airport",
                        "YGDI": "small_airport",
                        "YGDN": "small_airport",
                        "YGDS": "small_airport",
                        "YGDW": "small_airport",
                        "YGE": "seaplane_base",
                        "YGEL": "medium_airport",
                        "YGFN": "medium_airport",
                        "YGIA": "small_airport",
                        "YGIB": "small_airport",
                        "YGKL": "small_airport",
                        "YGLA": "medium_airport",
                        "YGLB": "medium_airport",
                        "YGLE": "small_airport",
                        "YGLG": "small_airport",
                        "YGLI": "medium_airport",
                        "YGLO": "small_airport",
                        "YGN": "seaplane_base",
                        "YGNF": "small_airport",
                        "YGNV": "small_airport",
                        "YGON": "small_airport",
                        "YGPT": "medium_airport",
                        "YGSC": "small_airport",
                        "YGTE": "medium_airport",
                        "YGTH": "medium_airport",
                        "YGTN": "small_airport",
                        "YGTO": "small_airport",
                        "YGYM": "small_airport",
                        "YHAW": "small_airport",
                        "YHAY": "medium_airport",
                        "YHBA": "medium_airport",
                        "YHBR": "small_airport",
                        "YHBY": "small_airport",
                        "YHDY": "small_airport",
                        "YHHY": "small_airport",
                        "YHID": "medium_airport",
                        "YHIL": "small_airport",
                        "YHLC": "medium_airport",
                        "YHMB": "small_airport",
                        "YHML": "medium_airport",
                        "YHOO": "small_airport",
                        "YHOT": "medium_airport",
                        "YHPN": "medium_airport",
                        "YHPV": "small_airport",
                        "YHSM": "medium_airport",
                        "YHTL": "small_airport",
                        "YHUG": "small_airport",
                        "YIDK": "small_airport",
                        "YIFL": "small_airport",
                        "YIFY": "small_airport",
                        "YIGM": "small_airport",
                        "YIKM": "small_airport",
                        "YINJ": "small_airport",
                        "YINN": "small_airport",
                        "YINW": "small_airport",
                        "YISF": "small_airport",
                        "YIVL": "medium_airport",
                        "YJAB": "small_airport",
                        "YJDA": "small_airport",
                        "YJIN": "small_airport",
                        "YJLC": "small_airport",
                        "YJNB": "small_airport",
                        "YKAL": "small_airport",
                        "YKBL": "small_airport",
                        "YKBR": "small_airport",
                        "YKBY": "small_airport",
                        "YKCA": "small_airport",
                        "YKCS": "small_airport",
                        "YKER": "medium_airport",
                        "YKII": "medium_airport",
                        "YKIR": "small_airport",
                        "YKKG": "small_airport",
                        "YKLA": "small_airport",
                        "YKLB": "small_airport",
                        "YKMB": "small_airport",
                        "YKML": "small_airport",
                        "YKMP": "medium_airport",
                        "YKNG": "small_airport",
                        "YKOW": "medium_airport",
                        "YKPR": "small_airport",
                        "YKRY": "medium_airport",
                        "YKSC": "medium_airport",
                        "YKT": "seaplane_base",
                        "YKUB": "small_airport",
                        "YKUR": "small_airport",
                        "YLAH": "small_airport",
                        "YLEC": "medium_airport",
                        "YLEO": "medium_airport",
                        "YLEV": "small_airport",
                        "YLFD": "small_airport",
                        "YLHI": "small_airport",
                        "YLHR": "medium_airport",
                        "YLHS": "small_airport",
                        "YLIM": "small_airport",
                        "YLIN": "small_airport",
                        "YLIS": "medium_airport",
                        "YLKN": "small_airport",
                        "YLLE": "small_airport",
                        "YLND": "small_airport",
                        "YLOK": "small_airport",
                        "YLOR": "small_airport",
                        "YLOV": "small_airport",
                        "YLP": "small_airport",
                        "YLRA": "small_airport",
                        "YLRD": "medium_airport",
                        "YLRE": "medium_airport",
                        "YLRS": "small_airport",
                        "YLST": "medium_airport",
                        "YLTN": "small_airport",
                        "YLTV": "medium_airport",
                        "YLZI": "small_airport",
                        "YMAA": "small_airport",
                        "YMAV": "medium_airport",
                        "YMAY": "medium_airport",
                        "YMBA": "medium_airport",
                        "YMBL": "small_airport",
                        "YMCO": "small_airport",
                        "YMCR": "small_airport",
                        "YMCT": "small_airport",
                        "YMDG": "medium_airport",
                        "YMDI": "small_airport",
                        "YMDS": "small_airport",
                        "YMEK": "medium_airport",
                        "YMEN": "medium_airport",
                        "YMER": "medium_airport",
                        "YMEU": "small_airport",
                        "YMGB": "small_airport",
                        "YMGD": "medium_airport",
                        "YMGN": "small_airport",
                        "YMGR": "small_airport",
                        "YMGT": "small_airport",
                        "YMGV": "small_airport",
                        "YMHB": "medium_airport",
                        "YMHO": "small_airport",
                        "YMHU": "small_airport",
                        "YMIA": "medium_airport",
                        "YMIN": "small_airport",
                        "YMIP": "small_airport",
                        "YMIT": "small_airport",
                        "YMJM": "small_airport",
                        "YMLS": "small_airport",
                        "YMLT": "medium_airport",
                        "YMMB": "medium_airport",
                        "YMMI": "small_airport",
                        "YMML": "large_airport",
                        "YMMU": "small_airport",
                        "YMND": "small_airport",
                        "YMNE": "medium_airport",
                        "YMNK": "small_airport",
                        "YMNS": "small_airport",
                        "YMNY": "small_airport",
                        "YMOG": "medium_airport",
                        "YMOO": "small_airport",
                        "YMOR": "medium_airport",
                        "YMOT": "small_airport",
                        "YMPA": "small_airport",
                        "YMQA": "small_airport",
                        "YMRB": "medium_airport",
                        "YMRE": "small_airport",
                        "YMRW": "small_airport",
                        "YMRY": "medium_airport",
                        "YMSF": "small_airport",
                        "YMTA": "small_airport",
                        "YMTB": "small_airport",
                        "YMTG": "medium_airport",
                        "YMTI": "small_airport",
                        "YMTO": "small_airport",
                        "YMUC": "small_airport",
                        "YMUG": "small_airport",
                        "YMUI": "small_airport",
                        "YMUK": "small_airport",
                        "YMUP": "small_airport",
                        "YMVG": "small_airport",
                        "YMWA": "small_airport",
                        "YMWT": "small_airport",
                        "YMWX": "small_airport",
                        "YMYB": "small_airport",
                        "YMYT": "small_airport",
                        "YNAP": "small_airport",
                        "YNAR": "medium_airport",
                        "YNBR": "medium_airport",
                        "YNGU": "medium_airport",
                        "YNHS": "small_airport",
                        "YNIC": "small_airport",
                        "YNKA": "small_airport",
                        "YNMN": "small_airport",
                        "YNPB": "small_airport",
                        "YNRC": "small_airport",
                        "YNRG": "small_airport",
                        "YNRM": "medium_airport",
                        "YNRV": "small_airport",
                        "YNSH": "small_airport",
                        "YNSM": "small_airport",
                        "YNT": "large_airport",
                        "YNTN": "medium_airport",
                        "YNUB": "small_airport",
                        "YNUL": "small_airport",
                        "YNUM": "small_airport",
                        "YNUT": "small_airport",
                        "YNWN": "medium_airport",
                        "YNYN": "small_airport",
                        "YOEN": "small_airport",
                        "YOI": "small_airport",
                        "YOLA": "small_airport",
                        "YOLD": "small_airport",
                        "YOLW": "small_airport",
                        "YOOD": "small_airport",
                        "YOOM": "small_airport",
                        "YORB": "small_airport",
                        "YORG": "medium_airport",
                        "YORV": "small_airport",
                        "YOSB": "small_airport",
                        "YOUY": "small_airport",
                        "YPAD": "large_airport",
                        "YPAG": "medium_airport",
                        "YPAM": "small_airport",
                        "YPBO": "medium_airport",
                        "YPCC": "medium_airport",
                        "YPDA": "small_airport",
                        "YPDI": "small_airport",
                        "YPDN": "medium_airport",
                        "YPDO": "small_airport",
                        "YPEC": "small_airport",
                        "YPGV": "medium_airport",
                        "YPIR": "medium_airport",
                        "YPJT": "medium_airport",
                        "YPKA": "medium_airport",
                        "YPKG": "medium_airport",
                        "YPKS": "medium_airport",
                        "YPKT": "medium_airport",
                        "YPKU": "medium_airport",
                        "YPLC": "medium_airport",
                        "YPLM": "medium_airport",
                        "YPMH": "small_airport",
                        "YPMP": "small_airport",
                        "YPMQ": "medium_airport",
                        "YPNG": "small_airport",
                        "YPOD": "medium_airport",
                        "YPPD": "medium_airport",
                        "YPPH": "large_airport",
                        "YPSH": "small_airport",
                        "YPTN": "medium_airport",
                        "YPWR": "small_airport",
                        "YPXM": "medium_airport",
                        "YQDI": "small_airport",
                        "YQJ": "seaplane_base",
                        "YQLP": "medium_airport",
                        "YQNS": "small_airport",
                        "YRBR": "small_airport",
                        "YRC": "seaplane_base",
                        "YRD": "small_airport",
                        "YREN": "medium_airport",
                        "YRMD": "small_airport",
                        "YRNG": "small_airport",
                        "YROB": "small_airport",
                        "YROE": "small_airport",
                        "YROI": "small_airport",
                        "YROM": "medium_airport",
                        "YRRB": "small_airport",
                        "YRSB": "small_airport",
                        "YRTI": "small_airport",
                        "YRTP": "small_airport",
                        "YRYH": "small_airport",
                        "YSAN": "small_airport",
                        "YSBK": "medium_airport",
                        "YSCB": "large_airport",
                        "YSCD": "small_airport",
                        "YSCH": "medium_airport",
                        "YSCN": "medium_airport",
                        "YSCO": "small_airport",
                        "YSCR": "small_airport",
                        "YSDU": "medium_airport",
                        "YSGE": "small_airport",
                        "YSGT": "small_airport",
                        "YSGW": "small_airport",
                        "YSHG": "small_airport",
                        "YSHK": "medium_airport",
                        "YSHR": "small_airport",
                        "YSHT": "medium_airport",
                        "YSII": "small_airport",
                        "YSMI": "medium_airport",
                        "YSMP": "small_airport",
                        "YSMR": "small_airport",
                        "YSNB": "medium_airport",
                        "YSNF": "medium_airport",
                        "YSNW": "medium_airport",
                        "YSPE": "small_airport",
                        "YSPK": "small_airport",
                        "YSPT": "small_airport",
                        "YSPV": "small_airport",
                        "YSRI": "medium_airport",
                        "YSRN": "small_airport",
                        "YSSY": "large_airport",
                        "YSTH": "small_airport",
                        "YSTI": "seaplane_base",
                        "YSTW": "medium_airport",
                        "YSVP": "small_airport",
                        "YSWG": "medium_airport",
                        "YSWH": "medium_airport",
                        "YSWL": "medium_airport",
                        "YTAA": "small_airport",
                        "YTAB": "small_airport",
                        "YTAM": "small_airport",
                        "YTAR": "small_airport",
                        "YTBF": "small_airport",
                        "YTBR": "small_airport",
                        "YTDR": "small_airport",
                        "YTEE": "small_airport",
                        "YTEF": "medium_airport",
                        "YTEM": "medium_airport",
                        "YTGA": "small_airport",
                        "YTGM": "medium_airport",
                        "YTHD": "small_airport",
                        "YTHY": "small_airport",
                        "YTIB": "medium_airport",
                        "YTKY": "small_airport",
                        "YTMO": "small_airport",
                        "YTMU": "medium_airport",
                        "YTMY": "small_airport",
                        "YTNB": "small_airport",
                        "YTNG": "medium_airport",
                        "YTNK": "medium_airport",
                        "YTOC": "small_airport",
                        "YTRE": "medium_airport",
                        "YTST": "small_airport",
                        "YTU": "closed",
                        "YTWB": "small_airport",
                        "YTY": "medium_airport",
                        "YUDA": "small_airport",
                        "YUNY": "small_airport",
                        "YUSL": "small_airport",
                        "YVRD": "small_airport",
                        "YVRS": "small_airport",
                        "YWAC": "small_airport",
                        "YWAL": "small_airport",
                        "YWAV": "small_airport",
                        "YWBL": "small_airport",
                        "YWBS": "small_airport",
                        "YWCA": "small_airport",
                        "YWCH": "small_airport",
                        "YWCK": "small_airport",
                        "YWDA": "small_airport",
                        "YWDG": "small_airport",
                        "YWDH": "medium_airport",
                        "YWDL": "small_airport",
                        "YWDV": "small_airport",
                        "YWGT": "medium_airport",
                        "YWHA": "medium_airport",
                        "YWHI": "small_airport",
                        "YWIT": "small_airport",
                        "YWKB": "medium_airport",
                        "YWLG": "medium_airport",
                        "YWLM": "medium_airport",
                        "YWLU": "medium_airport",
                        "YWMP": "small_airport",
                        "YWND": "small_airport",
                        "YWOL": "medium_airport",
                        "YWOR": "small_airport",
                        "YWRN": "medium_airport",
                        "YWSL": "medium_airport",
                        "YWTL": "small_airport",
                        "YWTN": "medium_airport",
                        "YWUD": "small_airport",
                        "YWWA": "small_airport",
                        "YWWG": "small_airport",
                        "YWWI": "small_airport",
                        "YWWL": "medium_airport",
                        "YWYM": "small_airport",
                        "YWYY": "medium_airport",
                        "YYA": "medium_airport",
                        "YYAL": "small_airport",
                        "YYKI": "small_airport",
                        "YYLR": "small_airport",
                        "YYMI": "small_airport",
                        "YYND": "small_airport",
                        "YYNG": "medium_airport",
                        "YYOR": "small_airport",
                        "YYTA": "small_airport",
                        "Z13": "small_airport",
                        "Z48": "small_airport",
                        "Z59": "seaplane_base",
                        "Z71": "seaplane_base",
                        "Z78": "seaplane_base",
                        "Z91": "small_airport",
                        "Z93": "small_airport",
                        "ZA-0021": "heliport",
                        "ZA-0069": "small_airport",
                        "ZA-0072": "small_airport",
                        "ZA-0094": "small_airport",
                        "ZA-0095": "small_airport",
                        "ZA-0096": "small_airport",
                        "ZA-0097": "small_airport",
                        "ZA-0098": "small_airport",
                        "ZA-0138": "small_airport",
                        "ZA-KIG": "small_airport",
                        "ZBAA": "large_airport",
                        "ZBAD": "large_airport",
                        "ZBCD": "medium_airport",
                        "ZBCF": "medium_airport",
                        "ZBCZ": "small_airport",
                        "ZBDH": "medium_airport",
                        "ZBDS": "medium_airport",
                        "ZBDT": "small_airport",
                        "ZBER": "medium_airport",
                        "ZBES": "medium_airport",
                        "ZBHD": "medium_airport",
                        "ZBHH": "large_airport",
                        "ZBHZ": "medium_airport",
                        "ZBK": "closed",
                        "ZBLA": "medium_airport",
                        "ZBLF": "medium_airport",
                        "ZBLL": "medium_airport",
                        "ZBMZ": "medium_airport",
                        "ZBNY": "large_airport",
                        "ZBOW": "medium_airport",
                        "ZBSJ": "medium_airport",
                        "ZBTJ": "large_airport",
                        "ZBTL": "medium_airport",
                        "ZBUC": "medium_airport",
                        "ZBUH": "small_airport",
                        "ZBUL": "small_airport",
                        "ZBXH": "medium_airport",
                        "ZBXT": "small_airport",
                        "ZBXZ": "small_airport",
                        "ZBYC": "small_airport",
                        "ZBYN": "large_airport",
                        "ZBYZ": "medium_airport",
                        "ZBZL": "medium_airport",
                        "ZDY": "small_airport",
                        "ZEN": "small_airport",
                        "ZGBH": "small_airport",
                        "ZGCD": "medium_airport",
                        "ZGCJ": "medium_airport",
                        "ZGDY": "small_airport",
                        "ZGGG": "large_airport",
                        "ZGHA": "large_airport",
                        "ZGHC": "medium_airport",
                        "ZGHU": "small_airport",
                        "ZGHY": "small_airport",
                        "ZGKL": "large_airport",
                        "ZGLG": "medium_airport",
                        "ZGMX": "small_airport",
                        "ZGNN": "large_airport",
                        "ZGOW": "medium_airport",
                        "ZGSD": "medium_airport",
                        "ZGSY": "medium_airport",
                        "ZGSZ": "large_airport",
                        "ZGWZ": "small_airport",
                        "ZGXN": "small_airport",
                        "ZGZH": "medium_airport",
                        "ZGZJ": "medium_airport",
                        "ZHAY": "small_airport",
                        "ZHCC": "large_airport",
                        "ZHES": "small_airport",
                        "ZHGH": "medium_airport",
                        "ZHHH": "large_airport",
                        "ZHLY": "small_airport",
                        "ZHNY": "small_airport",
                        "ZHSN": "medium_airport",
                        "ZHSS": "small_airport",
                        "ZHSY": "medium_airport",
                        "ZHXF": "small_airport",
                        "ZHYC": "medium_airport",
                        "ZIZ": "heliport",
                        "ZJHK": "large_airport",
                        "ZJQH": "medium_airport",
                        "ZJSY": "large_airport",
                        "ZKPY": "medium_airport",
                        "ZKSD": "medium_airport",
                        "ZKWS": "medium_airport",
                        "ZLAK": "small_airport",
                        "ZLDH": "medium_airport",
                        "ZLDL": "small_airport",
                        "ZLG": "closed",
                        "ZLGM": "medium_airport",
                        "ZLGY": "medium_airport",
                        "ZLHX": "medium_airport",
                        "ZLHZ": "small_airport",
                        "ZLIC": "small_airport",
                        "ZLJN": "medium_airport",
                        "ZLJQ": "medium_airport",
                        "ZLLL": "medium_airport",
                        "ZLLN": "medium_airport",
                        "ZLQY": "medium_airport",
                        "ZLSN": "medium_airport",
                        "ZLXH": "medium_airport",
                        "ZLXN": "medium_airport",
                        "ZLXY": "large_airport",
                        "ZLYA": "small_airport",
                        "ZLYL": "small_airport",
                        "ZLZW": "medium_airport",
                        "ZMAH": "medium_airport",
                        "ZMAT": "medium_airport",
                        "ZMBH": "medium_airport",
                        "ZMBN": "medium_airport",
                        "ZMBR": "small_airport",
                        "ZMBS": "small_airport",
                        "ZMBU": "medium_airport",
                        "ZMCD": "medium_airport",
                        "ZMCK": "large_airport",
                        "ZMD": "small_airport",
                        "ZMDN": "small_airport",
                        "ZMDZ": "medium_airport",
                        "ZMHH": "small_airport",
                        "ZMHU": "small_airport",
                        "ZMKD": "medium_airport",
                        "ZMMN": "medium_airport",
                        "ZMTG": "small_airport",
                        "ZMTL": "small_airport",
                        "ZMUB": "large_airport",
                        "ZMUG": "small_airport",
                        "ZMUL": "small_airport",
                        "ZNC": "small_airport",
                        "ZNU": "seaplane_base",
                        "ZPCW": "medium_airport",
                        "ZPDL": "medium_airport",
                        "ZPDQ": "medium_airport",
                        "ZPJH": "medium_airport",
                        "ZPJM": "medium_airport",
                        "ZPLJ": "medium_airport",
                        "ZPLX": "medium_airport",
                        "ZPPP": "large_airport",
                        "ZPSM": "small_airport",
                        "ZPWS": "small_airport",
                        "ZPZT": "small_airport",
                        "ZSAM": "large_airport",
                        "ZSAQ": "medium_airport",
                        "ZSBB": "medium_airport",
                        "ZSCG": "small_airport",
                        "ZSCN": "medium_airport",
                        "ZSFY": "medium_airport",
                        "ZSFZ": "large_airport",
                        "ZSGZ": "small_airport",
                        "ZSHC": "large_airport",
                        "ZSJD": "medium_airport",
                        "ZSJJ": "small_airport",
                        "ZSJN": "large_airport",
                        "ZSJU": "medium_airport",
                        "ZSLD": "medium_airport",
                        "ZSLG": "small_airport",
                        "ZSLQ": "medium_airport",
                        "ZSLY": "small_airport",
                        "ZSNB": "large_airport",
                        "ZSNJ": "large_airport",
                        "ZSOF": "medium_airport",
                        "ZSPD": "large_airport",
                        "ZSQD": "medium_airport",
                        "ZSQZ": "medium_airport",
                        "ZSRG": "medium_airport",
                        "ZSRZ": "medium_airport",
                        "ZSSS": "large_airport",
                        "ZSSZ": "medium_airport",
                        "ZSTX": "medium_airport",
                        "ZSWF": "medium_airport",
                        "ZSWH": "medium_airport",
                        "ZSWU": "medium_airport",
                        "ZSWX": "medium_airport",
                        "ZSWY": "medium_airport",
                        "ZSWZ": "large_airport",
                        "ZSXZ": "medium_airport",
                        "ZSYA": "medium_airport",
                        "ZSYC": "medium_airport",
                        "ZSYN": "small_airport",
                        "ZSYW": "medium_airport",
                        "ZSZS": "small_airport",
                        "ZUAL": "medium_airport",
                        "ZUAS": "medium_airport",
                        "ZUBD": "medium_airport",
                        "ZUBJ": "medium_airport",
                        "ZUCK": "large_airport",
                        "ZUDC": "medium_airport",
                        "ZUDX": "small_airport",
                        "ZUGH": "small_airport",
                        "ZUGU": "medium_airport",
                        "ZUGY": "large_airport",
                        "ZUJZ": "medium_airport",
                        "ZUKD": "medium_airport",
                        "ZUKJ": "medium_airport",
                        "ZULB": "small_airport",
                        "ZULP": "small_airport",
                        "ZULS": "medium_airport",
                        "ZULZ": "small_airport",
                        "ZUMH": "small_airport",
                        "ZUMT": "medium_airport",
                        "ZUMY": "medium_airport",
                        "ZUNC": "small_airport",
                        "ZUNP": "medium_airport",
                        "ZUNZ": "medium_airport",
                        "ZUPS": "medium_airport",
                        "ZUTC": "medium_airport",
                        "ZUTR": "small_airport",
                        "ZUUU": "large_airport",
                        "ZUWX": "small_airport",
                        "ZUXC": "medium_airport",
                        "ZUYB": "small_airport",
                        "ZUYI": "medium_airport",
                        "ZUZY": "medium_airport",
                        "ZWAK": "medium_airport",
                        "ZWAX": "medium_airport",
                        "ZWCM": "medium_airport",
                        "ZWFY": "medium_airport",
                        "ZWHM": "medium_airport",
                        "ZWKC": "small_airport",
                        "ZWKL": "medium_airport",
                        "ZWKM": "small_airport",
                        "ZWKN": "medium_airport",
                        "ZWNL": "medium_airport",
                        "ZWRQ": "medium_airport",
                        "ZWSC": "medium_airport",
                        "ZWSH": "medium_airport",
                        "ZWSS": "medium_airport",
                        "ZWTC": "small_airport",
                        "ZWTN": "medium_airport",
                        "ZWTP": "medium_airport",
                        "ZWWW": "large_airport",
                        "ZWYN": "small_airport",
                        "ZYAS": "medium_airport",
                        "ZYCC": "medium_airport",
                        "ZYCH": "medium_airport",
                        "ZYCY": "medium_airport",
                        "ZYDU": "medium_airport",
                        "ZYFY": "medium_airport",
                        "ZYHB": "large_airport",
                        "ZYHE": "medium_airport",
                        "ZYJL": "small_airport",
                        "ZYJM": "medium_airport",
                        "ZYJS": "medium_airport",
                        "ZYJX": "medium_airport",
                        "ZYJZ": "medium_airport",
                        "ZYLD": "medium_airport",
                        "ZYLS": "medium_airport",
                        "ZYMD": "medium_airport",
                        "ZYMH": "medium_airport",
                        "ZYQQ": "medium_airport",
                        "ZYSQ": "medium_airport",
                        "ZYTL": "large_airport",
                        "ZYTN": "medium_airport",
                        "ZYTX": "large_airport",
                        "ZYYJ": "medium_airport",
                        "ZYYK": "medium_airport",
                        "qfx": "heliport",
                        "rjns": "large_airport"
                    }
            }
        }
    }
}

In [ ]:
Now run this cell to send the object to the endpoint.

In [ ]:
x = requests.post(druid_host + '/druid/coordinator/v1/lookups/config', json=lookup_definition)

print (x.text)

Now run this to see what the status is.

You need to wait until the status is `"loaded":true` before going further in the notebook.

In [ ]:
x = requests.get(druid_host + '/druid/coordinator/v1/lookups/status/'+lookup_tier+'/'+lookup_name)
print(x.text)

Now let's query it

In [ ]:
sql='''
SELECT
    v AS "value",
    COUNT(*) AS "key_count"
FROM lookup."''' + lookup_name + '''"
GROUP BY 1
'''

display.sql(sql)

Now we can JOIN to it

In [ ]:
sql='''
SELECT
    a.Origin,
    b.v
FROM example-flights-lookup A
JOIN lookup."''' + lookup_name + '''" ON a.Origin = b.k
'''

display.sql(sql)

## Awesome!

The main body of your notebook goes here!

### This is a step

Here things get done

### And so is this!

Wow! Awesome!

## Summary

* You learned this
* Remember this

## Go further

* Try this out on your own data
* Solve for problem X that is't covered here

## Learn more

* Read the [full API documentation](https://druid.apache.org/docs/27.0.0/api-reference/lookups-api)
* Watch or read something cool from the community
* Do some exploratory stuff on your own

In [ ]:
# STANDARD CODE BLOCKS

# When just wanting to display some SQL results
display.sql(sql)

# When ingesting data:
sql_client.run_task(sql)
sql_client.wait_until_ready('wikipedia-en')
display.table('wikipedia-en')

# When you want to make an EXPLAIN look pretty
print(json.dumps(json.loads(sql_client.explain_sql(sql)['PLAN']), indent=2))

# When you want a simple plot
df = pd.DataFrame(sql_client.sql(sql))
df.plot(x='Tail_Number', y='Flights', marker='o')
plt.xticks(rotation=45, ha='right')
plt.gca().get_legend().remove()
plt.show()

# When you want to add some query context parameters
req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

# When you want to compare two different sets of results
df3 = df1.compare(df2, keep_equal=True)
df3